In [ ]:
## Import the Libraries
import pandas as pd
import numpy as np
import functools as ft
import os
import warnings
warnings.filterwarnings("ignore")
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import math
from datetime import date, timedelta
import time
from tqdm import tqdm
from datetime import datetime


periodMapping = {
                    'LTM'        : {"1Y" : {"S" : 0,   "F" : 252},
                                    "3Y" : {"S" : 252, "F" : 500},
                                    "5Y" : {"S" : 750, "F" : 500}},

                    'Momentum'   : {'1W' : 5, "1M" : 20, 
                                    "3M" : 62, "6M" : 125, 
                                    "1Y" : 250,"2Y" : 500,  
                                    "3Y" : 750},

                    'Volatility' : {'1W' : 5, "2W" : 10, 
                                    "1M" : 20, "3M" : 62, 
                                    "6M" : 125, "1Y" : 250, 
                                    "3Y" : 750},
                        
                    "Theme"      : {'1W' : 5, "2W" : 10, "2M" : 40,
                                    "1M" : 20, "3M" : 62,
                                    "1Y" : 250, "6M" : 125,
                                    "3Y" : 750}
                }

class EquityFactors:

    def __init__(self, noOfYears, correct, peer = "Theme"):
        ## Initialize the variables
        self.stockPriceData = None
        self.stockValueData = None
        self.sectorData = None
        self.benchmark = None
        self.stockFactors = None
        self.themeFactor = None
        self.companyMaster = None
        self.cashFlow = None
        self.profitLossGrowth = None
        self.profitLossQuality = None
        self.financialRatio = None
        self.appender = None
        self.financeBalanceSheet = None

        self.factorData = dict()
        self.recentFactorUpdate = dict()
        # self.__dataCursor = Data()
        self.rawFactorData = dict()

        self.noOfYears = noOfYears
        self.correct = correct
        self.peer = peer

    def uniqueFileName(self, filename):
        """
        Generates a unique filename by appending a counter to the base name if a file with 
        the specified filename already exists in the current directory.

        Args:
            filename (str): The desired filename.

        Returns:
            str: A unique filename that does not already exist in the current directory.
        """

        # Split the path into directory and filename
        directory, file_name = os.path.split(filename)
        
        # Split the filename into name and extension
        name, ext = os.path.splitext(file_name)
        
        # Initialize counter
        counter = 1
        
        # Generate the full path
        full_path = os.path.join(directory, file_name)
        
        # Check if the file already exists
        while os.path.exists(full_path):
            # If it does, create a new filename with the counter
            new_filename = f"{name}_{counter}{ext}"
            full_path = os.path.join(directory, new_filename)
            counter += 1
            
        return full_path

    def __readPriceData(self,):
        ''' Function  is used to read the price data '''
        ## Read the price Data.
        # self.stockPriceData = self.__dataCursor.fetch_price_data(no_of_years = self.noOfYears, inc_indices=True)
        self.stockPriceData = pd.read_csv("./price_data.csv")

        self.stockPriceData["Date"] = pd.to_datetime(self.stockPriceData["Date"])

        # if self.correct:
        #     # Replace The Old Symbol With the New Symbol
        #     mapping = pd.read_excel("company_master_mapping.xlsx")
        #     mapping = dict(zip(mapping["SYMBOL_NSE"], mapping["SYMBOL_CM"]))
        #     self.stockPriceData["Symbol"] = self.stockPriceData["Symbol"].replace(mapping)

        self.stockPriceData["Symbol"] = self.stockPriceData["Symbol"].str.strip()
        self.stockPriceData1 = self.stockPriceData.copy()
        self.stockPriceDataMom = self.stockPriceData.copy()
        ## Drop the Rows with NaN Values
        self.stockPriceData = self.stockPriceData.dropna()
        ## Remove the data of Erroneous Date.
        self.stockPriceData = self.stockPriceData[self.stockPriceData["Date"] != "2022-03-07"]#.reset_index(drop = True)

        ## Filter the data for the Benchmark data
        self.benchmark = self.stockPriceData[self.stockPriceData["Symbol"] == "NIFTY500"].reset_index(drop = True)
        ## Filter the necessary columns for benchmark
        self.benchmark = self.benchmark.filter(["Date","Close"])
        ## Calculate the daily percentage change for the benchmark and renaming the columns.
        self.benchmark["BenchmarkReturn"] = self.benchmark["Close"].pct_change()
        self.benchmark.rename(columns = {'Close' : "BenchmarkPrice"}, inplace = True)

        ## Fetch the ETF Indices List
        # self.etf = self.__dataCursor.fetch_data_from_database(table_name="Etf_Indices")
        self.etf = pd.read_csv("./etf_indices.csv")
        ## Remove the ETF Indices data from the stock data
        self.stockPriceData = self.stockPriceData[~self.stockPriceData['Symbol'].isin(self.etf['Symbol'])]

        self.priceDataLTM = self.stockPriceData.copy()
        ## Remove the data for the Saturday and Sunday and Sort the dataframe
        self.stockPriceData = self.stockPriceData[~self.stockPriceData["Date"].dt.day_name().isin(['Sunday', "Saturday"])]
        self.stockPriceData = self.stockPriceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        # ## Identify Companies which have been in Top-500 historically.
        # top500Companies  = self.stockPriceData.groupby(["Date"]).apply(lambda x: x.sort_values("Mcap", ascending=False).head(500), include_groups = False).reset_index(level = 0).reset_index(drop = True)
        # self.top500Companies = top500Companies["Symbol"].unique().tolist()
        # ## Identify Companies which have been in TOP-1000 historically.
        # top1000Companies  = self.stockPriceData.groupby(["Date"]).apply(lambda x: x.sort_values("Mcap", ascending=False).head(1000), include_groups=False).reset_index(level = 0).reset_index(drop = True)
        # self.top1000Companies = top1000Companies["Symbol"].unique().tolist()

        self.strategyUniverse = pd.read_csv("Universe.csv")
        # self.strategyUniverse = self.__dataCursor.fetch_data_from_database(table_name="universe_mcap_500", no_of_years=50)
        self.strategyUniverse["Date"] = pd.to_datetime(self.strategyUniverse["Date"])
        self.strategyUniverse["Peer"] = self.strategyUniverse[self.peer].copy()
        self.strategyUniverse.rename(columns = {"MCAP" : "Mcap"}, inplace = True)

        # sectordata = self.__dataCursor.fetch_data_from_database(table_name="SectorThemeGics")
        # self.strategyUniverse = pd.merge(self.strategyUniverse, sectordata, on = "Symbol", how = "left")
        # self.strategyUniverse[["Theme", "Sector"]] = self.strategyUniverse[["Theme", "Sector"]].fillna("Others")
        # self.strategyUniverse["Peer"] = self.strategyUniverse[self.peer].copy()

    def __readSectorData(self,):
        ## Inout the Sector Mapping data
        # self.sectorData = self.__dataCursor.fetch_data_from_database(table_name="SectorThemeGics")
        self.sectorData = pd.read_csv("./gics.csv")

    def __readCompanyMaster(self,):
        # self.companyMaster = self.__dataCursor.fetch_data_from_database(table_name = "Companymaster")
        self.companyMaster = pd.read_csv("./CM.csv")
        self.companyMaster = self.companyMaster[self.companyMaster["SERIES"] == "EQ"]

        self.companyMaster = self.companyMaster[["FINCODE", "SYMBOL"]].copy()
        self.companyMaster = self.companyMaster.dropna().reset_index(drop = True)
        self.companyMaster.rename(columns = {'SYMBOL' : "Symbol"}, inplace = True)

    def __readProfitLossGrowth(self,):
        # self.profitLossGrowth = self.__dataCursor.fetch_data_from_database(table_name = "Quarterly")
        self.profitLossGrowth = pd.read_csv("./Quarterly.csv")
        self.profitLossGrowth = self.profitLossGrowth[self.profitLossGrowth["Result_Type"] == "Q"].reset_index(drop = True)
        self.profitLossGrowth = self.profitLossGrowth.filter(items = ["Fincode", "Date_End",  "PAT", "OPERATING_PROFIT", "GROSS_PROFIT",
                                                                      "NET_SALES", "EPS_DILUTED", "PBT", 'Debt/Equity Ratio', 'Adj_eps_abs', "Dividend payout ratio"])
        self.profitLossGrowth.rename(columns = {'Fincode' : "FINCODE"}, inplace = True)

    def __readCashFlow(self, ):
        # self.cashFlow = self.__dataCursor.fetch_data_from_database(table_name = "Finance_cf")
        self.cashFlow = pd.read_csv("./Finance_Cf.csv")

    def __readValueData(self):
            ## Input the Company Valuation Data.

            # if self.correct:
            #     self.stockValueData = self.__dataCursor.fetch_data_from_database(table_name = "Value_RawData_Test", no_of_years = self.noOfYears)
            # else:
            #     self.stockValueData = self.__dataCursor.fetch_data_from_database(table_name = "Value_RawData", no_of_years = self.noOfYears)
            
            self.stockValueData = pd.read_csv("./Value_Data.csv")
            ## COnvert the Date column from string to Datetime format
            self.stockValueData["Date"] = pd.to_datetime(self.stockValueData["Date"])

            # Import the Nifty Valuation Fields
            # self.benchmarkValueData = self.__dataCursor.fetch_data_from_database(table_name = "Nifty_PE_PB", no_of_years = self.noOfYears)
            self.benchmarkValueData = pd.read_csv("./Nifty_PE_PB.csv")
            self.benchmarkValueData["Date"] = pd.to_datetime(self.benchmarkValueData["Date"])

    def __readProfitLossQuality(self,):
        # self.profitLossQuality = self.__dataCursor.fetch_data_from_database(table_name = "Finance_pl")
        self.profitLossQuality = pd.read_csv("./Finance_pl.csv")
        self.profitLossQuality = self.profitLossQuality[["FINCODE", "Year_end", "Profit_after_tax", "Net_sales", 
                                                        "Operating_profit", "Gross_profits", "Adj_Eps"]]

    def __readFinancialRatio(self,):
        # self.financialRatio = self.__dataCursor.fetch_data_from_database(table_name = "Finance_fr")

        self.financialRatio = pd.read_csv("./Finance_fr.csv")
        self.financialRatio = self.financialRatio[["FINCODE", "Year_end", "Inventory_Days", "Receivable_days", "Payable_days", "ROE", "ROCE", 
                                                   "Total_Debt_Equity", "Interest_Cover", "CEPS", "ROA", "FCF_Share", "Dividend_Payout_Per"]]
        self.financialRatio = self.financialRatio[self.financialRatio["Year_end"] >= 200012]
        self.financialRatio = self.financialRatio.reset_index(drop = True)

    def __readFinanceBalanceSheet(self,):
        # self.financeBalanceSheet = self.__dataCursor.fetch_data_from_database(table_name = "Finance_bs")
        self.financeBalanceSheet = pd.read_csv("./Finance_bs.csv")
        # self.financeBalanceSheet = self.financeBalanceSheet[["FINCODE", "Year_end", "Profit_after_tax", "Net_sales", 
        #                                                  "Operating_profit", "Gross_profits", "Adj_Eps"]]
        self.financeBalanceSheet.rename(columns = {"Fincode" : "FINCODE"}, inplace = True)

    def generate_LowVol(self,):

        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Calculate daily returns for each stock by symbol, based on "Close" prices
        priceData["Returns"] = priceData.groupby("Symbol")["Close"].pct_change()
        
        # Calculate squared returns for down days only; store in "DReturns" (for downside volatility)
        priceData["DReturns"] = np.square(np.where(priceData["Returns"] < 0, priceData["Returns"], 0))

        # Calculate 1-year rolling average of downside returns for each symbol and take the square root
        priceData["DownVol"] = priceData.groupby("Symbol")["DReturns"].transform(lambda x: x.rolling(252).mean())
        priceData["DownVol"] = np.sqrt(priceData["DownVol"])

        # Calculate 1-year rolling standard deviation of returns as low volatility measure
        priceData['LowVol'] = priceData.groupby("Symbol")["Returns"].transform(lambda x: x.rolling(window=252).std())

        # Calculate average volatility as the mean of LowVol and DownVol
        priceData["AvgVol"] = priceData[["LowVol", "DownVol"]].mean(axis=1)

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        # lowvol raw data
        self.rawFactorData['lowvol'] = priceData.copy()

        # Rank stocks daily within the top 500 by LowVol, DownVol, and AvgVol, assigning percentile ranks
        priceData[["LowVolRank", "DownVolRank", "AvgVolRank"]] = priceData.groupby("Date")[["LowVol", "DownVol", "AvgVol"]].rank(ascending=False, pct=True)
    
        # Compute average volatilities for each theme on each date
        themeScore = priceData.groupby(["Peer", "Date"])[["LowVol", "DownVol", "AvgVol"]].mean()
        
        # Rename columns to reflect theme-based volatility measures
        themeScore.columns = ["PeerLowVol", "PeerDownVol", "PeerAvgVol"]

        # Merge theme scores with price data
        priceData = pd.concat([priceData.set_index(["Peer", "Date"]), themeScore], axis=1).reset_index()

        # Rank theme-based volatilities across all themes daily
        priceData[["PeerLowVol", "PeerDownVol", "PeerAvgVol"]] = priceData.groupby(["Date"])[["PeerLowVol", "PeerDownVol", "PeerAvgVol"]].rank(ascending=False, pct=True)

        # Select relevant columns and rename to remove "Rank" suffix for final output
        df = priceData[["Date", "Symbol", "LowVolRank", "DownVolRank", "AvgVolRank", "PeerLowVol", "PeerDownVol", "PeerAvgVol"]].copy()
        df.columns = df.columns.str.replace("Rank", "")

        # Filter data from January 1, 2006, onwards and reset index
        df = df[df["Date"] >= "2006-01-01"].reset_index(drop=True)
 
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()

        # Iterate over each volatility-related column
        for col in ["LowVol", "DownVol", "AvgVol", "PeerLowVol", "PeerDownVol", "PeerAvgVol"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = df.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)
        scores.columns = scores.columns.str.replace("Peer", self.peer)

        ## Store the Factor Data in dictionary.
        self.factorData["LowVol"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["LowVol"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### LOWVOL COMPLETE ###")

        del priceData, themeScore, df, scores, result

    def generate_AM(self,):

        # Function to calculate log returns
        def calculate_log_returns(df):
            df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
            return df.dropna()

        # Function to calculate annualized standard deviation
        def calculate_annualized_std(df, window=252):
            return df['LogReturn'].rolling(window).std() * np.sqrt(window)

        # Function to calculate momentum ratios
        def calculate_momentum_ratios(series, period):
            return series / series.shift(period) - 1

        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Define periods for momentum ratios (in trading days)
        periods = {
            'MR1': 21,   # 1 month
            'MR2': 42,   # 2 months
            'MR3': 63,   # 3 months
            'MR6': 126,  # 6 months
            'MR12': 252, # 12 months
        }

        # Apply log return calculation
        priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
    
        # Calculate momentum ratios for each period
        for label, period in periods.items():
            priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

        # momentum raw data
        self.rawFactorData['momentum'] = priceData.copy()

        # Calculate annualized standard deviation
        priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

        # Normalize the momentum ratios by dividing by the annualized standard deviation
        for label in periods.keys():
            priceData[label] /= priceData['AnnualizedStd']

        priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)

        # Calculate the mean and std deviation of each momentum ratio across the universe
        for label in periods.keys():
            priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
            priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())

        # Calculate Z-scores for each period
        for label in periods.keys():
            priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']

        # Define specific combinations for which to calculate the final Z-scores
        metrics = ['Z_MR1', 'Z_MR2', 'Z_MR3', 'Z_MR6', 'Z_MR12']

        # Weighted average Z-score
        priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)

        # Normalized momentum score
        priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                1 + priceData[f'WtdZScore'],
                                                (1 - priceData[f'WtdZScore']) ** -1)

        ## Filter the Required Columns  
        priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Computing the Percentile Score of the Symbols on each Date
        priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())

        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)

        ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()

        # Iterate over each volatility-related column
        for col in ["Momentum", "PeerMomentum"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["AM"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["AM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### MOMENTUM COMPLETE ###")

        del priceData, result, temp, scores
    
    def generate_LTMA(self,):

        ## If Price is ot imported form Databse, then read it from DB.
        if self.stockPriceData is None:
            self.__readPriceData()
        
        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Sort the Dataframe by Symbol and Date, Reset the index
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        # Calculate 1-year, 3-year, and 5-year percentage changes in stock closing prices.
        for key in [3, 6, 12, 18]:
            priceData[f"{key}_return"] = priceData.groupby('Symbol')["Close"].pct_change(22*key)
            priceData[f"{key}_vol"] = priceData.groupby('Symbol')["Close"].transform(lambda x: x.pct_change().rolling(key*22).std())
            priceData[f"{key}_vol"] *= np.sqrt(252)
            priceData[f"{key}_sharpe"] = priceData[f"{key}_return"] / priceData[f"{key}_vol"]
            priceData.drop(columns = [f"{key}_return", f"{key}_vol"], inplace = True)

        ## Sort and reset the index
        priceData.sort_values("Date", inplace = True)
    
        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Assigning the rank to each 500 companies on particular date.
        for key in [3, 6, 12, 18]:
            priceData[f"{key}_Rank"]=priceData.groupby('Date')[f"{key}_sharpe"].rank(pct=True)

        ## Aggregate the rank for composite score.
        priceData["LTMA"] = priceData[[col for col in priceData.columns if "Rank" in col]].mean(axis = 1, skipna = False)

        ## Filter out the necessary columns
        priceData = priceData.filter(items = ["Date", "Symbol", "LTMA"])

        ## Convert the long from Dataframe to wide form dataframe
        priceData = priceData.pivot_table(index='Date',columns='Symbol',values='LTMA').reset_index()
        ## Shifting the Date column
        priceData["Date"] = priceData["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        priceData = priceData.melt(id_vars='Date', value_name = "LTMA")
        ## Drop na and reset the index
        priceData = priceData.dropna().reset_index(drop = True)
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["LTMA"] = priceData.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["LTMA"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys() if key != "Theme"], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### LTMA COMPLETE ###")

        del priceData

    def generate_LTM(self,):

        ## If Price is ot imported form Databse, then read it from DB.
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Set the maping dictionary for LTM Factor.
        mapper = periodMapping["LTM"]

        ## Function to calculate 3-year return after shifting by 100 days
        def rollingReturn(group, shift, period, key):
            group[f'{key}_Return'] = group['Close'].ffill().shift(shift).pct_change(period, fill_method = None)
            return group
            
        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Sort the Dataframe by Symbol and Date, Reset the index
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        # Calculate 1-year, 3-year, and 5-year percentage changes in stock closing prices.
        for key in mapper.keys():
            priceData = priceData.groupby('Symbol').apply(rollingReturn, shift = mapper[key]["S"], 
                                                            period = mapper[key]["F"], 
                                                            key = key, include_groups = False).reset_index(level = 0)

        ## Sort and reset the index
        priceData.sort_values("Date", inplace = True)
    
        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Assigning the rank to each 500 companies on particular date.
        for key in mapper.keys():
            priceData[f"{key}_Rank"]=priceData.groupby('Date')[f"{key}_Return"].rank(pct=True)
        
        ## Aggregate the rank for composite score.
        priceData["LTM"] = priceData[[col for col in priceData.columns if "Rank" in col]].mean(axis = 1, skipna = False)

        ## Filter out the necessary columns
        priceData = priceData.filter(items = ["Date", "Symbol", "LTM"])

        ## Convert the long from Dataframe to wide form dataframe
        priceData = priceData.pivot_table(index='Date',columns='Symbol',values='LTM').reset_index()
        
        ## Shifting the Date column
        priceData["Date"] = priceData["Date"].shift(-1)

        ## Fill NaN value of date with todays date.
        priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        priceData = priceData.melt(id_vars='Date', value_name = "LTM")

        ## Drop na and reset the index
        priceData = priceData.dropna().reset_index(drop = True)
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["LTM"] = priceData.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["LTM"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys() if key != "Theme"], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### LTM COMPLETE ###")

        del priceData

    def generate_EM(self):
        # RSI function
        def rsi(closes, n):
            diff_serie = closes.diff()
            gain = diff_serie.where(diff_serie > 0, 0)
            loss = -diff_serie.where(diff_serie < 0, 0)
            avg_gain = gain.rolling(window=n).mean()
            avg_loss = loss.rolling(window=n).mean()
            rs = avg_gain / avg_loss
            rsi = 100 - (100 / (1 + rs))
            return rsi.fillna(0)

        ################
        ## PRICE DATA ##
        ################
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
        
        ## Calculate the 50 and 200 day moving average
        priceData['200_ma'] = priceData.groupby('Symbol')['Close'].transform(lambda x: x.rolling(200).mean())
        priceData['50_ma'] = priceData.groupby('Symbol')['Close'].transform(lambda x: x.rolling(50).mean())

        # Calculate Close to 200DMA and 50DMA to 200DMA Ratio.
        priceData['200_ma_ratio'] = priceData['Close'] / priceData['200_ma']
        priceData['50_200_ratio'] = priceData['50_ma'] / priceData['200_ma']

        # Compute the RSI
        priceData['rsi'] = priceData.groupby('Symbol')['Close'].transform(lambda x: rsi(x, 14))

        ## Compute the rank for each of the sub-factors
        priceData['200_ma_ratio_score'] = priceData.groupby('Date')['200_ma_ratio'].rank(ascending=False, pct=True)
        priceData['50_200_score'] = priceData.groupby('Date')['50_200_ratio'].rank(ascending=False, pct=True)
        priceData['rsi_score'] = priceData.groupby('Date')['rsi'].rank(ascending=False, pct=True)

        # Calculate final scores and sor the dataframe
        priceData['finalScore'] = priceData[['200_ma_ratio_score', '50_200_score', 'rsi_score']].sum(axis = 1, skipna = False)

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Rank the top500 companies and filter the necessary columns
        priceData['AM_New'] = priceData.groupby('Date')['finalScore'].rank(ascending=False, pct=True)
        priceData = priceData.filter(items= ['Date','Symbol','AM_New'])

        ## Filter the data after year 2006 and sor the dataframe.
        priceData = priceData[priceData["Date"] >= "2006-01-01"].copy()
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
    
        ## Convert the long from Dataframe to wide form dataframe
        priceData = priceData.pivot_table(index='Date',columns='Symbol',values='AM_New').reset_index()
        ## Shifting the Date column
        priceData["Date"] = priceData["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())
        ## Re-converting the wide form dataframe to long form dataframe
        priceData = priceData.melt(id_vars='Date', value_name= "EM")
        ## Drop na and reset the index
        priceData = priceData.dropna().reset_index(drop = True)

        ## Store the Factor Data in dictionary.
        self.factorData["EM"] = priceData.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["EM"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### EM COMPLETE ###")

        del priceData

    def generate_Dividend(self, valueYieldCall = False):

        def previousQuarter(currDate):
            if pd.Period(currDate,freq = "Q").end_time.date() == currDate.date():
                return currDate
            else:
                return currDate - pd.offsets.QuarterEnd()
            
        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossGrowth is None:
            self.__readProfitLossGrowth() 
    
        if self.stockPriceData is None:
            self.__readPriceData()
            
        # Fetch the rebalance dates for the last 100 years from the "GrowthDate" table in the database
        # rebalanceDates = self.__dataCursor.fetch_data_from_database(table_name = "GrowthDate", no_of_years=100)
        rebalanceDates = pd.read_csv('growth_date_latest.csv')
        rebalanceDates['Date'] = pd.to_datetime(rebalanceDates['Date'])

        # Create a copy of the strategy universe (likely a dataframe or similar structure) to work with for company composition
        composition  = self.strategyUniverse.copy()

        # Extract the quarter and date values from the rebalanceDates dataframe
        rebalanceQtr = list(rebalanceDates['Quarter'])
        rebalanceDate = list(rebalanceDates['Date'])

        # Create a dictionary mapping each date to its corresponding quarter and vice versa
        qtrDateDict = dict(zip(rebalanceDate, rebalanceQtr))
        dateQtrDict = dict(zip(rebalanceQtr, rebalanceDate))

        # Add a new column 'Quarter' to the composition dataframe by mapping the 'Date' column to the respective quarter using qtrDateDict
        composition['Quarter'] = composition['Date'].map(qtrDateDict)

        # Filter the necessary columns ('FINCODE', 'Date_End', 'Dividend payout ratio', 'PAT') from the profitLossGrowth dataframe
        div_qtr = self.profitLossGrowth.filter(items=["FINCODE", "Date_End", "Dividend payout ratio", "PAT"])

        # Merge div_qtr with companyMaster on the 'FINCODE' column to include company information
        div_qtr = pd.merge(div_qtr, self.companyMaster, on="FINCODE", how="inner")

        # Convert the 'Date_End' column to datetime format
        div_qtr["Date_End"] = pd.to_datetime(div_qtr['Date_End'], format="%Y%m")

        # Shift 'Date_End' to the respective month-end date
        div_qtr["MonthEnd"] = div_qtr["Date_End"] + pd.offsets.MonthEnd()

        # Create a new column 'QuarterEnd' to calculate the end of the financial quarter based on the 'MonthEnd'
        div_qtr["QuarterEnd"] = div_qtr["MonthEnd"].apply(previousQuarter) 

        # Convert the 'QuarterEnd' to the Indian Financial Year format, which ends in March
        div_qtr["YearQuarter"] = div_qtr["QuarterEnd"].dt.to_period("Q-MAR")

        # Extract the quarter and year from 'YearQuarter' and create a new 'Quarter' column
        div_qtr["Quarter"] = div_qtr["YearQuarter"].astype(str).str[-2:] + div_qtr["YearQuarter"].astype(str).str[:4]

        # Extract the year from 'YearQuarter' and create a 'Year' column
        div_qtr["Year"] = div_qtr["YearQuarter"].astype(str).str[:4].astype(int)

        # Sort the dataframe by 'YearQuarter' in ascending order for chronological order
        div_qtr.sort_values("YearQuarter", ascending=True, inplace=True)
        div_qtr.reset_index(drop=True, inplace=True)

        # Calculate the dividend by multiplying 'PAT' (Profit After Tax) by the 'Dividend payout ratio'
        div_qtr['dividend'] = div_qtr['PAT'] * div_qtr['Dividend payout ratio']

        # Sort by 'Symbol' and 'MonthEnd' for proper chronological order within each company
        div_qtr.sort_values(['Symbol', 'MonthEnd'], inplace=True)

        # Calculate the rolling dividend over the last 4 quarters for each company
        div_qtr['Rollingdividend'] = div_qtr.groupby('Symbol')['dividend'].transform(lambda x: x.rolling(4).sum())

        # Sort the dataframe again by 'Symbol' and 'YearQuarter', and reset the index
        div_qtr.sort_values(["Symbol", "YearQuarter"], inplace=True)
        div_qtr.reset_index(drop=True, inplace=True)

        # Map the 'Quarter' to its respective 'Date' using the dateQtrDict for final data
        div_qtr['Date'] = div_qtr['Quarter'].map(dateQtrDict)

        # Merge the 'div_qtr' with the 'composition' dataframe on 'Symbol', 'Date', and 'Quarter' to get the final composition
        div_qtr = pd.merge(div_qtr, composition, on=['Symbol', 'Date', 'Quarter'])

        # Extract the year from the 'Date' column and assign it to a new 'Year' column
        div_qtr['Year'] = div_qtr['Date'].dt.year

        # Sort the dataframe by 'Date' and 'Symbol' for easier analysis and reset the index
        div_qtr = div_qtr.sort_values(["Date", "Symbol"]).reset_index(drop=True)

        # Create a dataframe 'divRank' containing only relevant columns like dividend, PAT, and Rollingdividend
        divRank = div_qtr.filter(items=["Date", "Symbol", "dividend", "PAT", 'Dividend payout ratio', 'Rollingdividend'])

        # Map the 'divRank' to the 'composition' dataframe on 'Date' and 'Symbol' to align dividends with composition details
        divRank = pd.merge(composition, divRank, on=['Date', 'Symbol'], how='left')

        # Sort the 'divRank' dataframe by 'Symbol' and 'Date' for proper chronological order
        divRank = divRank.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        # Forward fill missing values in 'dividend', 'PAT', 'Dividend payout ratio', and 'Rollingdividend' within each 'Symbol'
        divRank[['dividend', 'PAT', 'Dividend payout ratio', 'Rollingdividend']] = divRank.groupby('Symbol')[['dividend', 'PAT', 'Dividend payout ratio', 'Rollingdividend']].ffill()

        # Calculate the dividend yield for each company as (Rollingdividend / Market Capitalization) * 100
        divRank['yield'] = (divRank['Rollingdividend'] / divRank['Mcap']) * 100

        # Calculate the 4-quarter rolling average of the dividend yield for each company
        divRank["yield"] = divRank.groupby("Symbol")["yield"].transform(lambda x: x.rolling(4).mean())

        # Final dataframe 'divRankFinal' contains 'Date', 'Symbol', and 'Dividend' columns, with no missing values
        divRankFinal = divRank[['Date', 'Symbol', 'yield']].dropna()

        # dividend raw data
        self.rawFactorData['dividend'] = divRankFinal.copy()

        # Rename the 'yield' column to 'Dividend' for better clarity
        divRankFinal.rename(columns={'yield': 'Dividend'}, inplace=True)

        if valueYieldCall:
            return divRankFinal

        ## Convert the long from Dataframe to wide form dataframe
        divRankFinal = divRankFinal.pivot_table(index='Date',columns='Symbol',values='Dividend').reset_index()

        ## Shifting the Date column
        divRankFinal["Date"] = divRankFinal["Date"].shift(-1)
        
        ## Fill NaN value of date with todays date.
        divRankFinal.iloc[-1, divRankFinal.columns.get_loc("Date")] = pd.to_datetime(date.today())
        
        ## Re-converting the wide form dataframe to long form dataframe
        divRankFinal = divRankFinal.melt(id_vars='Date', value_name= "Dividend")

        ## Drop na and reset the index
        divRankFinal = divRankFinal.dropna().reset_index(drop = True)
        divRankFinal = divRankFinal[divRankFinal["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        self.div= divRankFinal.copy()

        divRankFinal['Dividend']=divRankFinal.groupby('Date')['Dividend'].rank(pct=True)

        ## Store the Factor Data in dictionary.
        self.factorData["Dividend"] = divRankFinal.copy()
        
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["Dividend"] = divRankFinal[divRankFinal["Date"] == divRankFinal["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### DIVIDEND COMPLETE ###")

        del div_qtr, divRank, divRankFinal

    def generate_Growth(self,valueYieldCall = False):

        ## FUnction to get the previous Quarter Date
        def previousQuarter(currDate):
            if pd.Period(currDate,freq = "Q").end_time.date() == currDate.date():
                return currDate
            else:
                return currDate - pd.offsets.QuarterEnd()

        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossGrowth is None:
            self.__readProfitLossGrowth()

        if self.cashFlow is None:
            self.__readCashFlow()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()


        ######################
        ## DATA PREPARATION ##
        ######################
        ## List of the Rebalance Dates
        # rebalanceDates = self.__dataCursor.fetch_data_from_database(table_name = "GrowthDate", no_of_years=100)
        rebalanceDates = pd.read_csv('growth_date_latest.csv')
        rebalanceDates['Date'] = pd.to_datetime(rebalanceDates['Date'])
        
        # Identify Companies which have been in Top-500 historically
        composition  = self.strategyUniverse.copy()
        
        ## List of Quarter and Dates
        rebalanceQtr=list(rebalanceDates['Quarter'])
        rebalanceDate=list(rebalanceDates['Date'])
        
        ## Mappinf Dictionary of Date to Quarter and vice versa
        qtrDateDict=dict(zip(rebalanceDate,rebalanceQtr))
        dateQtrDict=dict(zip(rebalanceQtr,rebalanceDate))

        ## Quarter column in PriceData
        composition['Quarter']=composition['Date'].map(qtrDateDict)

        ############################
        ## OPERATION ON CASH DATA ##
        ############################
        
        ## Import the Cash data
        cf_yearly = self.cashFlow.filter(items = ["FINCODE", "Year_end", "Cash_from_Operation"])
        ## Map the Company Names
        cf_yearly = pd.merge(cf_yearly, self.companyMaster, how = "inner", on = "FINCODE")
        ## Convert the string into Datetime
        cf_yearly["Year_end"] = pd.to_datetime(cf_yearly['Year_end'],format="%Y%m")
        cf_yearly["Year"] = cf_yearly["Year_end"].dt.year
        ## Shift the Date Year
        cf_yearly['Year'] = np.where(cf_yearly['Year_end'].dt.month == 12, 
                                    (cf_yearly['Year'] + 1), 
                                    (cf_yearly['Year']))
        cf_yearly["Year"] = cf_yearly["Year"] + 1
        ## Sort the dataframe
        cf_yearly.sort_values('Year',inplace=True)
        cf_yearly.reset_index(drop = True, inplace = True)

        self.cf_yearly = cf_yearly.copy()

        ###################################
        ## OPERATION ON PROFIT/LOSS DATA ##
        ###################################
        ## Read the Fundamental data of the comapnies - Growth Related
        pl_yearly = self.profitLossGrowth.drop(columns = [ 'Debt/Equity Ratio', 'Adj_eps_abs', 'Dividend payout ratio']).copy()
        ## Convert the string into Datetime
        pl_yearly["Date_End"] = pd.to_datetime(pl_yearly['Date_End'],format="%Y%m")

        ## Merge the Fundamental data with master list of the companies 
        pl_yearly = pd.merge(pl_yearly, self.companyMaster, on = "FINCODE", how = "inner")

        ## Calculate the Operating_Margin and Gross_Margin
        pl_yearly["Operating_Margin"] = pl_yearly["OPERATING_PROFIT"].div(pl_yearly["NET_SALES"])
        pl_yearly["Gross_Margin"] = pl_yearly["GROSS_PROFIT"].div(pl_yearly["NET_SALES"])

        # ## Shift Date to respective Month End date.
        pl_yearly["MonthEnd"] = pl_yearly["Date_End"] + pd.offsets.MonthEnd()
        pl_yearly["QuarterEnd"] = pl_yearly["MonthEnd"].apply(previousQuarter) 

        ## Convert the Dates as per Indian Financial Quarter
        pl_yearly["YearQuarter"] = pl_yearly["QuarterEnd"].dt.to_period("Q-MAR")
        pl_yearly["Quarter"] = pl_yearly["YearQuarter"].astype(str).str[-2:] + pl_yearly["YearQuarter"].astype(str).str[:4]
        pl_yearly["Year"] =pl_yearly["YearQuarter"].astype(str).str[:4].astype(int)

        ## sort the Data Frame based on Quarter
        pl_yearly.sort_values("YearQuarter", ascending = True, inplace = True)
        pl_yearly.reset_index(drop = True, inplace = True)

        self.pl_yearly = pl_yearly.copy()

        ###############
        ## COMBINING ##
        ###############
        ## Combine the Profit loss and Cash data
        data = pd.merge(pl_yearly, cf_yearly, on = ['FINCODE',"Symbol", "Year"])

        ## List of Factors
        factorUniverse = ['PAT','OPERATING_PROFIT','GROSS_PROFIT','NET_SALES','EPS_DILUTED','PBT','Operating_Margin','Gross_Margin','Cash_from_Operation']
        ## SUbste the columns
        data = data[["Year", "YearQuarter", "Quarter", "Symbol", "Date_End"] + factorUniverse]

        # growth raw data
        self.rawFactorData['growth'] = data.copy()

        ## Sort the DataFrame
        data.sort_values(["Symbol", "YearQuarter"], inplace = True)
        data.reset_index(drop = True, inplace = True)

        ## Calculating the Change in factors over a year
        year_chg = lambda ser: (ser - ser.shift(4))/ser.abs().shift(4)
        data[[f"{c}_change" for c in factorUniverse]] = data.groupby("Symbol", group_keys=False)[factorUniverse].apply(year_chg)
    
        ## Mapping the Date column to Quarter Date
        data['Date']=data['Quarter'].map(dateQtrDict)
        
        data=pd.merge(data,composition,on=['Symbol','Date','Quarter'])
        data['Year']=data['Date'].dt.year

        ## Compute the Absolute / Peer and Historical Rank
        data = data.sort_values(["Date", "Symbol"]).reset_index(drop = True)

        if valueYieldCall:
            return data

        data.drop(columns = ["Date_End"], inplace = True)

        factorUniverse = [f"{col}_change" for col in factorUniverse]
        data["AbsRank"] = data.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        data["PeerRank"] = data.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        data = data.sort_values(["Symbol", "Date"]).reset_index(drop = True)
        data["HistRank"] = data.groupby("Symbol")[factorUniverse].rolling(window = 12, min_periods = 4).rank(pct = True).mean(axis = 1).reset_index(drop = True)
        growthRank = data.filter(items = ["Date", "Symbol", "AbsRank", "PeerRank", "HistRank"])
        growthRank["Rank"] = growthRank[[ "AbsRank", "PeerRank", "HistRank"]].mean(axis = 1)
        
        ## Mapping the Growth score to each date
        growthRank = pd.merge(composition,growthRank,on=['Date','Symbol'],how='left')
        growthRank = growthRank.sort_values(["Symbol",  "Date"]).reset_index(drop = True)

        ## Forward fill the growth score, as growth is for each quarter
        growthRank[['AbsRank','PeerRank','HistRank','FinalRank']]=growthRank.groupby('Symbol')[['AbsRank','PeerRank','HistRank','Rank']].ffill()
        growthRank['Growth'] = growthRank['PeerRank']*0.95 + growthRank['AbsRank']*0.05

        ## Convert the long from Dataframe to wide form dataframe
        growthRank = growthRank.pivot_table(index='Date',columns='Symbol',values='Growth').reset_index()
        ## Shifting the Date column
        growthRank["Date"] = growthRank["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        growthRank.iloc[-1, growthRank.columns.get_loc("Date")] = pd.to_datetime(date.today())
        ## Re-converting the wide form dataframe to long form dataframe
        growthRank = growthRank.melt(id_vars='Date', value_name= "Growth")
        ## Drop na and reset the index
        growthRank = growthRank.dropna().reset_index(drop = True)
        growthRank = growthRank[growthRank["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["Growth"] = growthRank.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["Growth"] = growthRank[growthRank["Date"] == growthRank["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### GROWTH COMPLETE ###")

    def generate_ValueYield(self):

        ## Fetch the Stock Value Data
        if self.stockValueData is None:
            self.__readValueData()

        ## Fetch the 
        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        
        ### VALUE ###
        # Filter the rows for Top-500 companies historically
        valueData = self.stockValueData[self.stockValueData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].copy()

        ## List of Fundamental Factor Value.
        fundamentalValueFactor = ["EV_EBITDA", "PS", "PB", "PE"]
        ## Replace -ve value with NaN value for all the fundamental value factor
        for factor in fundamentalValueFactor:
            valueData[factor] = np.where(valueData[factor] <= 0, np.nan, valueData[factor])
            valueData[factor] = 1/valueData[factor]

        ## Sort and reset the index
        valueData.sort_values(["Symbol", "Date"], inplace = True)
        valueData.reset_index(drop = True, inplace = True)


        ### Dividend ###
        dividendData = self.generate_Dividend(valueYieldCall = True)

        ## EPS ###
        epsData = self.generate_Growth(valueYieldCall = True)

        ## ROE
        roe = self.generate_QualityQuarter(valueYieldCall=True)
        roe["Date_End"] = pd.to_datetime(roe['Date_End'],format="%Y%m")

        growth = pd.merge(epsData[["Date", "Date_End", "Symbol","EPS_DILUTED_change"]], 
                          roe[["Date_End", "Symbol", "ROE_ttm"]], on = ["Date_End", "Symbol"], how = "left")

        final=pd.merge(valueData,dividendData,on=['Date','Symbol'])
        final=pd.merge(final,growth,on=['Date','Symbol'],how='left')

        final = final.sort_values(["Symbol","Date"]).reset_index(drop = True)
        final[['ROE_ttm', "EPS_DILUTED_change"]]=final.groupby('Symbol')[['ROE_ttm', "EPS_DILUTED_change"]].ffill()
        final['PEG']=final['PE']/final['EPS_DILUTED_change']

        factorUniverse = ['EV_EBITDA', 'PS','PB', 'PE', 'Dividend','PEG','ROE_ttm']

        final = pd.merge(self.strategyUniverse, final[["Date", "Symbol"] + factorUniverse], on = ["Date","Symbol"], how = "left")

        # valueyield raw data
        self.rawFactorData['value_yield'] = final.copy()

        final["AbsRank"] = final.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        final["PeerRank"] = final.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        final['ValueYield'] = final['PeerRank']*0.95 + final['AbsRank']*0.05
        final['ValueABS'] = final['PeerRank']*0.05 + final['AbsRank']*0.95        

        result = list()

        # Iterate over each volatility-related column
        for col in ["ValueYield", "ValueABS"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = final.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))


        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        ## Store the Factor Data in dictionary.
        self.factorData["ValueYield"] = scores.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ValueYield"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### ValueYield COMPLETE ###")

    def generate_QualityAnnual(self,):

        ## Function to calculate the volatility of the ratio
        def calculateVol(series):
            _max = series.rolling(window = 5, min_periods = 3).max()
            _min = series.rolling(window = 5, min_periods = 3).min()
            _mean = series.rolling(window = 5, min_periods = 3).mean() 
            return (_max - _min)/(_mean)

        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossQuality is None:
            self.__readProfitLossQuality()

        if self.financialRatio is None:
            self.__readFinancialRatio()

        if self.cashFlow is None:
            self.__readCashFlow()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        ## Read the Rebalance Dates
        # rebalanceDates = self.__dataCursor.fetch_data_from_database(table_name = "QualityDate", no_of_years = 100)
        rebalanceDates = pd.read_csv('QualityDate.csv')
        rebalanceDates["Date"] = pd.to_datetime(rebalanceDates["Date"])
        rebalances = rebalanceDates["Date"].tolist()

        ## Mapping the Data Receive Year
        priceData = pd.merge(self.strategyUniverse, rebalanceDates, on = "Date", how = "left")
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
        
        ## Filter the dataframe for the rebalance dates.
        priceDataTop500Rebal = priceData[priceData["Date"].isin(rebalances)]

        ############################
        ### PROFIT and LOSS DATA ###
        ############################
        ## Mapping Company Symbol to Cash Flow Data
        profitLoss = pd.merge(self.profitLossQuality, self.companyMaster, on = ["FINCODE"], how = "inner")

        ## Calculating the Operating Margin and Gross Margin
        profitLoss["OperatingMargin"] = profitLoss["Operating_profit"].div(profitLoss["Net_sales"])
        profitLoss["GrossMargin"] = profitLoss["Gross_profits"].div(profitLoss["Net_sales"])

        ## Convert the Date String to python datetime
        profitLoss["Year_end"] = pd.to_datetime(profitLoss["Year_end"], format = "%Y%m")
        profitLoss["Year"]  = profitLoss["Year_end"].dt.year
        profitLoss["Year"] = np.where(profitLoss["Year_end"].dt.month == 12, profitLoss["Year"]+1, profitLoss["Year"])

        ## Drop the duplicates and Keep the last record
        profitLoss = profitLoss.sort_values(["Year_end", "Symbol"])#.drop_duplicates(["Year", "Symbol"], keep = "last")

        ## Sort the dataframe by Year column and reset the dataframe
        profitLoss.sort_values("Year", inplace = True)
        profitLoss.reset_index(drop = True, inplace = True)
        
        ############################
        ### FINANCIAL RATIO DATA ###
        ############################
        ## Replace NaN value with 0
        financialRatio = self.financialRatio.copy()
        financialRatio["Payable_days"] = financialRatio["Payable_days"].fillna(0)
        ## Calculating the Working Capital Days
        financialRatio["WC_Days"] = financialRatio["Inventory_Days"] + financialRatio["Receivable_days"] - financialRatio["Payable_days"]
        ## Mapping COmpany Symbol to Financial Ratio data
        financialRatio = pd.merge(financialRatio, self.companyMaster, on = ["FINCODE"], how = "inner")

        ## Convert the Date string to python datetime
        financialRatio["Year_end"] = pd.to_datetime(financialRatio.Year_end, format = "%Y%m")
        financialRatio["Year"] = financialRatio["Year_end"].dt.year
        financialRatio["Year"] = np.where(financialRatio["Year_end"].dt.month == 12, financialRatio["Year"] + 1, financialRatio["Year"])

        ## Drop the duplicates and Keep the last record
        financialRatio = financialRatio.sort_values(["Year_end", "Symbol"])#.drop_duplicates(["Year", "Symbol"], keep = "last")

        ## Sort the dataframe by Year column and reset the dataframe
        financialRatio.sort_values("Year", inplace = True)
        financialRatio.reset_index(drop = True, inplace = True)

        ######################
        ### CASH FLOW DATA ###
        ######################
        ## Mapping Company Symbol to Cash Flow Data
        cashFlow = self.cashFlow[["FINCODE", "Year_end","Cash_from_Operation"]].copy()
        cashFlow = pd.merge(cashFlow, self.companyMaster, on = ["FINCODE"], how = "inner")

        ## Convert the Date String to python datetime
        cashFlow["Year_end"] = pd.to_datetime(cashFlow["Year_end"], format = "%Y%m")
        cashFlow["Year"]  = cashFlow["Year_end"].dt.year
        cashFlow["Year"] = np.where(cashFlow["Year_end"].dt.month == 12, cashFlow["Year"]+1, cashFlow["Year"])

        ## Drop the duplicates and Keep the last record
        cashFlow = cashFlow.sort_values(["Year_end", "Symbol"])#.drop_duplicates(["Year", "Symbol"], keep = "last")
        ## Sort the dataframe by Year column and reset the dataframe
        cashFlow.sort_values("Year", inplace = True)
        cashFlow.reset_index(drop = True, inplace = True)

        ## Merge the dataframes of Profit and Loss, Cash Flow and Financial Ratios into one dataframe
        data = ft.reduce(lambda left, right: pd.merge(left, right, on=['Symbol','Year','FINCODE']), [profitLoss, financialRatio, cashFlow])
        ## Calculating Cash Flow Operation (CFO) BY EBITDA  and Free cash Flow (FCF) COnversion ratio.
        data["CFO_By_EBITDA"] = data["Cash_from_Operation"].div(data["Operating_profit"])
        data["FCF_Conversion"] = data["FCF_Share"].div(data["Adj_Eps"])

        ## Sort the dataframe and reset the index
        data.sort_values(["Symbol", "Year"], inplace = True)
        data.reset_index(drop = True, inplace = True)

        ## Calculating the Volatility of Selected Financial ratio ("ROE", "Adj_EPS", "CEPS", "OperatingMargin")
        tempFactors = ['ROE', 'Adj_Eps', 'CEPS', 'OperatingMargin']
        data[[f"Vol_{value}" for value in tempFactors]] = data.groupby("Symbol", group_keys = False)[tempFactors].apply(calculateVol)

        ## Calculating the Percentage of Selected Financila Ratio ("Net_Sales", "Oprating_profit", "Adj_Eps", "Cash_from_operation", "FCF_Share")
        tempFactors = ["Net_sales", "Operating_profit", "Adj_Eps", "Cash_from_Operation", "FCF_Share"]
        data[[f"{value}_Growth" for value in tempFactors]] = data.groupby("Symbol", group_keys = False)[tempFactors].apply(lambda ser: ser.pct_change())

        ## Consolidated List of individual factors to create the Consolidated Quality Factor
        factorUniverse = ["FCF_Share", "OperatingMargin", "GrossMargin", "CFO_By_EBITDA", "ROE", "ROCE",
                        "Total_Debt_Equity", "Interest_Cover", "ROA", "Inventory_Days", "Receivable_days",
                        "WC_Days", "FCF_Conversion","Vol_ROE", "Vol_Adj_Eps", "Vol_CEPS", "Vol_OperatingMargin",
                        "Dividend_Payout_Per"]

        # "Growth"
        ## Indivodual facrtor in universe are divided in two list, one to be ranked in ascending order and other list to be ranked in descending order.
        ascendingFactorUniverse = ["CFO_By_EBITDA", "Dividend_Payout_Per", 
                                "FCF_Conversion",  "FCF_Share", "GrossMargin", "Interest_Cover", "OperatingMargin", "ROA",
                                "ROCE", "ROE"]

        descendingFactorUniverse = ["Inventory_Days", "Receivable_days", "WC_Days", "Total_Debt_Equity", 
                                    "Vol_ROE", "Vol_Adj_Eps", "Vol_CEPS", "Vol_OperatingMargin"]
        
        ## Filter the Necessary columns
        data = data.filter(items = ["Symbol", 'Year'] + factorUniverse)

        # quality raw data
        self.rawFactorData['quality'] = data.copy()
        
        ## Mapping the Factor to Stock price Data.
        rank = pd.merge(priceDataTop500Rebal, data, on = ["Symbol", "Year"])

        ## Sort the Dataframe and reset the index
        rank.sort_values("Date", inplace = True)
        rank.reset_index(drop = True, inplace = True)

        ## Individual Absolute rank for Ascending and Descending Factor
        absAscRank = rank.groupby("Year")[ascendingFactorUniverse].rank(ascending = True, pct = True)
        absDescRank = rank.groupby("Year")[descendingFactorUniverse].rank(ascending = False, pct = True)
        ## Take the mean of individual factor to get Aboslute rank for each stock
        rank["AbsoluteRank"] = pd.concat([absAscRank, absDescRank],axis = 1).mean(axis = 1)

        ## Individual Peer rank for Ascending and Descending Factor
        peerAscRank = rank.groupby(["Year", "Sector"])[ascendingFactorUniverse].rank(ascending = True, pct = True)
        peerDescRank = rank.groupby(["Year", "Sector"])[descendingFactorUniverse].rank(ascending = False, pct = True)
        ## Take the mean of individual factor to get Peer rank for each stock
        rank["PeerRank"] = pd.concat([peerAscRank, peerDescRank],axis = 1).mean(axis = 1)

        ## Mapping the Rank against The dates of each stock.
        quality = pd.merge(priceData, rank[["Date", "Symbol", "AbsoluteRank", "PeerRank"]], on = ["Date", "Symbol"], how = "left")

         ## Sort the dataframe and reset the index
        quality.sort_values(["Symbol", "Date"], inplace = True)
        quality.reset_index(drop = True, inplace = True)
    
        ## Forward fill the Rank
        quality[["AbsoluteRank", "PeerRank"]] = quality.groupby("Symbol", group_keys = False)[["AbsoluteRank", "PeerRank"]].ffill()
    
        ## Selecting the relevant columns
        quality = quality.filter(["Date", "Symbol", "AbsoluteRank", "PeerRank"])

        ## Calculate the Weighted Quality Factor Score
        quality["QualityFactor"] = quality["AbsoluteRank"] * 0.05 + quality["PeerRank"] * 0.95
        ## Selecting the relevant columns
        quality = quality.filter(["Date", "Symbol", "QualityFactor"])

        ## Convert the long from Dataframe to wide form dataframe
        quality = quality.pivot_table(index='Date',columns='Symbol',values='QualityFactor').reset_index()
        ## Shifting the Date column
        quality["Date"] = quality["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        quality.iloc[-1, quality.columns.get_loc("Date")] = pd.to_datetime(date.today())
        ## Re-converting the wide form dataframe to long form dataframe
        quality = quality.melt(id_vars='Date', value_name= "QualityAnnual")
        ## Drop na and reset the index
        quality = quality.dropna().reset_index(drop = True)
        quality = quality[quality["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["QualityAnnual"] = quality.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["QualityAnnual"] = quality[quality["Date"] == quality["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys() ], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### QUALITY ANNUAL COMPLETE ###")
            
    def generate_QualityQuarter(self, valueYieldCall = False):

        # Quarterly Networth Calculation
        def fill_networth(row, prev_networth):
            if pd.isna(row['NetWorth']):
                return prev_networth + row['PAT']
            else:
                return row['NetWorth']

        def fill_networth_column(df):
            df = df.sort_values(by=['FINCODE', 'Date_End']) 
            df['NetWorth_Filled'] = np.nan
            
            for fincode in df['FINCODE'].unique():
                prev_networth = None
                for i, row in df[df['FINCODE'] == fincode].iterrows():
                    if prev_networth is None:
                        prev_networth = row['NetWorth']
                    else:
                        df.at[i, 'NetWorth_Filled'] = fill_networth(row, prev_networth)
                        prev_networth = df.at[i, 'NetWorth_Filled']
            
            return df
        
        #  EPS Growth Calculation
        def calculate_yoy_eps_growth(eps):
            """Calculates YoY EPS Growth based on the provided rules."""
            growth = []
            for i in range(len(eps)):
                if i < 4:
                    growth.append(np.nan) 
                else:
                    prev_eps = eps.iloc[i - 4]  
                    curr_eps = eps.iloc[i]      
                    if prev_eps > 0:
                        growth.append((curr_eps - prev_eps) / prev_eps)
                    elif prev_eps < 0:
                        growth.append(-(curr_eps - prev_eps) / prev_eps)
                    else:
                        growth.append(np.nan)  
            return pd.Series(growth, index=eps.index)  

        # Calculate 5-year mean and std deviation for each quarter separately
        def calc_quarterly_stats(group):
            group = group.sort_values('Date_End')
            group['Mean_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=5, min_periods=1).mean()
            group['Std_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=5, min_periods=1).std()
            return group
        
        # Weighted Average Z Quality Score
        def calculate_weighted_avg_z(row):
            if row['Sector'] == 'Bank':
                return (1/2) * row['Z_ROE_ttm_fin'] - (1/2) * abs(row['Z_EPS_Growth_Variability_fin'])
            else:
                return (1/3) * row['Z_ROE_ttm'] - (1/3) * abs(row['Z_DE']) - (1/3) * abs(row['Z_EPS_Growth_Variability'])
        
        # Define a function to check for negatives in the last 16 quarters
        def has_negative_in_last_4_years(series):
            return series.rolling(window=16, min_periods=16).apply(lambda x: (x < 0).any(), raw=True)

        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossGrowth is None:
            self.__readProfitLossGrowth()

        if self.financeBalanceSheet is None:
            self.__readFinanceBalanceSheet()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()


        price_data = self.stockPriceData.copy()
        top_500 = self.strategyUniverse.copy()

        # rebalDates = self.__dataCursor.fetch_data_from_database(table_name="GrowthDate", no_of_years=50)
        rebalDates = pd.read_csv('growth_date_latest.csv')
        rebalDates['Date'] = pd.to_datetime(rebalDates['Date'])
        Rebalance_Qtr=list(rebalDates['Quarter'])
        Rebalance_Date=list(rebalDates['Date'])
        Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))
        date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))

        top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)

        ## Financial Balance Sheet Modfication
        financialBalanceSheet = self.financeBalanceSheet[["Year_end", "FINCODE", "Share_Capital", "Reserve"]]
        financialBalanceSheet["NetWorth"] = financialBalanceSheet[["Share_Capital", "Reserve"]].sum(axis = 1)
        financialBalanceSheet  = pd.merge(financialBalanceSheet, self.companyMaster[["FINCODE", "Symbol"]], on = "FINCODE")
        financialBalanceSheet = financialBalanceSheet[["Year_end", "FINCODE", "Symbol", "NetWorth"]]

        ## PAT Modification
        pat = self.profitLossGrowth[["Date_End", "FINCODE", "PAT"]]
        pat["PAT"] /= 10

        patBS = pd.merge(pat, financialBalanceSheet, left_on = ["Date_End", "FINCODE"], 
                right_on = ["Year_end", "FINCODE"], how = "left")
        patBS['Symbol'] = patBS.groupby('FINCODE', group_keys=False)['Symbol'].apply(lambda x: x.fillna(method='ffill'))
        patBS.drop(columns = ["Year_end"], inplace = True)

        # Apply the function
        patBS = fill_networth_column(patBS)
        # Fill any remaining NaN in the original NetWorth column with the filled values
        patBS['NetWorth'] = patBS['NetWorth'].combine_first(patBS['NetWorth_Filled'])
        patBS.drop(columns=['NetWorth_Filled'], inplace=True)  
        patBS = patBS.dropna()

        # Calculating ROE from scratch
        patBS['ROE'] = patBS.groupby('Symbol', group_keys=False).apply(lambda x: x['PAT']/x['NetWorth'])
        patBS['ROE_ttm'] = patBS.groupby('Symbol', group_keys=False)['ROE'].apply(lambda x: x.rolling(window=4).sum())

        self.patBS = patBS.copy()

        roe_ttm = patBS[['FINCODE', 'Date_End', 'ROE_ttm']].copy()

        if valueYieldCall:
            return  patBS[['FINCODE', "Symbol", 'Date_End', 'ROE_ttm']]

        quality = self.profitLossGrowth[['FINCODE','Date_End','Debt/Equity Ratio', 'Adj_eps_abs']]
        # quality = factor.profitLossGrowth[['FINCODE','Date_End','Debt/Equity Ratio', 'Adj_eps_abs']]
        quality = pd.merge(quality, roe_ttm, on=['FINCODE', 'Date_End'])
        pl_quality = pd.merge(quality, self.companyMaster, on = "FINCODE", how='inner')

        self.pl_quality = pl_quality.copy()
    
        # Getting required Quality ratios
        pl_quality = pl_quality[['FINCODE', "Symbol", 'Date_End', 'Debt/Equity Ratio', 'ROE_ttm', 'Adj_eps_abs']].reset_index(drop=True)

        # Removing the rows with NaN Symbol Names
        pl_quality = pl_quality[pl_quality["Symbol"].notna()]

        # Filtering Stocks which have been historically in top 500 universe only.
        pl_quality = pl_quality[pl_quality["Symbol"].isin(top_500.Symbol.unique())]

        # Calculate Adjusted EPS TTM
        pl_quality['Adj_eps_abs_TTM'] = pl_quality.groupby('Symbol')['Adj_eps_abs'].transform(lambda x: x.rolling(4).sum())

        # Apply the function to create a flag for exclusion
        pl_quality['exclude_flag'] = pl_quality.groupby('Symbol')['Adj_eps_abs_TTM'].transform(has_negative_in_last_4_years)

        # Remove symbols with fewer than 16 quarters of data
        # Count the number of quarters for each symbol
        symbol_quarter_counts = pl_quality.groupby('Symbol').size()

        # Create a list of symbols with at least 16 quarters
        valid_symbols = symbol_quarter_counts[symbol_quarter_counts >= 16].index

        # Filter the DataFrame to keep only rows with valid symbols
        pl_quality = pl_quality[pl_quality['Symbol'].isin(valid_symbols)]

        # Filter out rows based on the exclude flag
        b_filtered = pl_quality[pl_quality['exclude_flag'] != 1].reset_index(drop=True)

        # Drop the intermediate column used for filtering
        b_filtered = b_filtered.drop(columns=['exclude_flag'])

        # Mapping GICS to each Symbol
        df = pd.merge(b_filtered, self.sectorData[['Symbol', 'Sector']], on='Symbol', how='left')

        # Calculating ROE for TTM for the past 4 Years years
        df['ROE_ttm'] = df.groupby('Symbol')['ROE_ttm'].transform(lambda x : x.rolling(16).mean())

        # Create a new column for quarter information
        df['Quarter'] = pd.to_datetime(df['Date_End'], format='%Y%m').dt.quarter

        # Calculating YoY EPS Growth for each quarter
        df['YoY_EPS_Growth'] = df.groupby('Symbol')['Adj_eps_abs'].transform(calculate_yoy_eps_growth)

        # Apply function to each SYMBOL and Quarter group
        df = df.groupby(['Symbol', 'Quarter'], group_keys=False).apply(calc_quarterly_stats)

        # Dropping Quarter column after calculation to keep the original dataframe structure
        df.drop(columns=['Quarter'], inplace=True)

        # Calculating rolling 5-year mean and std deviation for EPS growth
        df['Mean_YoY_EPS_Growth'] = df.groupby('Symbol')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=20).mean())
        df['Std_YoY_EPS_Growth'] = df.groupby('Symbol')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=20).std())

        # Calulating EPS Growth Variability
        df['EPS_Growth_Variability'] = df['Mean_YoY_EPS_Growth'].div(df['Std_YoY_EPS_Growth'])

        # Split data into financial and non-financial sectors
        financials_df = df[df['Sector'] == 'Bank']
        non_financials_df = df[df['Sector'] != 'Bank']

        # Calculate mean and std for financials
        financials_df['mean_roe_fin'] = financials_df.groupby('Date_End')['ROE_ttm'].transform('mean')
        financials_df['std_roe_fin'] = financials_df.groupby('Date_End')['ROE_ttm'].transform('std')

        financials_df['mean_eps_growth_variability_fin'] = financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
        financials_df['std_eps_growth_variability_fin'] = financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

        # Calculate mean and std for non-financials
        non_financials_df['mean_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('mean')
        non_financials_df['std_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('std')

        non_financials_df['mean_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('mean')
        non_financials_df['std_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('std')

        non_financials_df['mean_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
        non_financials_df['std_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

        # Combine the financials and non-financials dataframes back together
        df = pd.concat([financials_df, non_financials_df]).reset_index(drop=True)

        # Z-Score Calculation
        df['Z_ROE_ttm'] = (df['ROE_ttm'] - df['mean_roe']) / df['std_roe']
        df['Z_ROE_ttm_fin'] = (df['ROE_ttm'] - df['mean_roe_fin']) / df['std_roe_fin']
        df['Z_DE'] = (df['Debt/Equity Ratio'] - df['mean_de']) / df['std_de']
        df['Z_EPS_Growth_Variability'] = (df['EPS_Growth_Variability'] - df['mean_eps_growth_variability']) / df['std_eps_growth_variability']
        df['Z_EPS_Growth_Variability_fin'] = (df['EPS_Growth_Variability'] - df['mean_eps_growth_variability_fin']) / df['std_eps_growth_variability_fin']

        df['WeightedAvgZ'] = df.apply(calculate_weighted_avg_z, axis=1)

        # Calculate Normalized Quality Score
        df['Quality_Score'] = np.where(df['WeightedAvgZ'] >= 0, 
                                    1 + df['WeightedAvgZ'], 
                                    (1 - df['WeightedAvgZ'])**-1)

        df = df[['Date_End', 'Symbol', 'Sector','Quality_Score']].dropna().reset_index(drop=True).dropna()

        df['Quality_pct_rank'] = df.groupby('Date_End', group_keys=False)['Quality_Score'].apply(lambda x : x.rank(pct=True))
        # GrowthDate = self.__dataCursor.fetch_data_from_database(table_name='GrowthDate', no_of_years=25)[['Date', 'Qtr']]
        GrowthDate = pd.read_csv('growth_date_latest.csv')[['Date', 'Qtr']]
        GrowthDate['Date'] = pd.to_datetime(GrowthDate['Date'])
        GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')

        # GrowthDate Mapping  
        final_df =  pd.merge(df, GrowthDate, left_on='Date_End', right_on='Qtr').drop(columns=['Date_End', 'Qtr'])
        final_df = final_df[['Date', 'Symbol', 'Sector', 'Quality_Score', 'Quality_pct_rank']].sort_values(by='Date').reset_index(drop=True)
        
        price_data['Date'] = pd.to_datetime(price_data['Date'])
        merged_df = pd.merge(final_df[['Date', 'Symbol', 'Quality_pct_rank']], price_data, on=['Date', 'Symbol'], how='outer')

        merged_df['Quality_pct_rank'] = merged_df.groupby('Symbol', group_keys=False)['Quality_pct_rank'].apply(lambda x: x.fillna(method='ffill'))

        merged_df = merged_df.groupby("Date").apply(lambda x: x.sort_values("Mcap", ascending = False).head(500)).reset_index(drop = True)

        merged_df["Quality_pct_rank"] = merged_df.groupby("Date")["Quality_pct_rank"].rank(pct = True)

        qualityfinal = merged_df[['Date','Symbol','Quality_pct_rank']]

        top500 = qualityfinal.filter(["Symbol", "Date", "Quality_pct_rank"])

        ## Convert the long from Dataframe to wide form dataframe
        top500 = top500.pivot_table(index='Date',columns='Symbol',values='Quality_pct_rank').reset_index()
        ## Shifting the Date column
        top500["Date"] = top500["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        top500.iloc[-1, top500.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        top500 = top500.melt(id_vars='Date', value_name = "QualityQuarter")
        ## Drop na and reset the index
        top500 = top500.dropna().reset_index(drop = True)
        top500 = top500[top500["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

         ## Store the Factor Data in dictionary.
        self.factorData["QualityQuarter"] = top500.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["QualityQuarter"] = top500[top500["Date"] == top500["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### QUALITY QUARTER COMPLETE ###")

    def generate_ValueYieldNoPeg(self):

        ## Fetch the Stock Value Data
        if self.stockValueData is None:
            self.__readValueData()

        ## Fetch the 
        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        
        ### VALUE ###
        # Filter the rows for Top-500 companies historically
        valueData = self.stockValueData[self.stockValueData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].copy()

        ## List of Fundamental Factor Value.
        fundamentalValueFactor = ["EV_EBITDA", "PS", "PB", "PE"]
        ## Replace -ve value with NaN value for all the fundamental value factor
        for factor in fundamentalValueFactor:
            valueData[factor] = np.where(valueData[factor] <= 0, np.nan, valueData[factor])
            valueData[factor] = 1/valueData[factor]

        ## Sort and reset the index
        valueData.sort_values(["Symbol", "Date"], inplace = True)
        valueData.reset_index(drop = True, inplace = True)


        ### Dividend ###
        dividendData = self.generate_Dividend(valueYieldCall = True)

        ## EPS ###
        epsData = self.generate_Growth(valueYieldCall = True)

        ## ROE
        roe = self.generate_QualityQuarter(valueYieldCall=True)
        roe["Date_End"] = pd.to_datetime(roe['Date_End'],format="%Y%m")

        growth = pd.merge(epsData[["Date", "Date_End", "Symbol","EPS_DILUTED_change"]], 
                          roe[["Date_End", "Symbol", "ROE_ttm"]], on = ["Date_End", "Symbol"], how = "left")

        final=pd.merge(valueData,dividendData,on=['Date','Symbol'])
        final=pd.merge(final,growth,on=['Date','Symbol'],how='left')

        final = final.sort_values(["Symbol","Date"]).reset_index(drop = True)
        final[['ROE_ttm', "EPS_DILUTED_change"]]=final.groupby('Symbol')[['ROE_ttm', "EPS_DILUTED_change"]].ffill()
        final['PEG']=final['PE']/final['EPS_DILUTED_change']

        factorUniverse = ['EV_EBITDA', 'PS','PB', 'PE', 'Dividend','ROE_ttm']

        final = pd.merge(self.strategyUniverse, final[["Date", "Symbol"] + factorUniverse], on = ["Date","Symbol"], how = "left")

        final["AbsRank"] = final.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        final["PeerRank"] = final.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        final['ValueYieldNoPeg'] = final['PeerRank']*0.95 + final['AbsRank']*0.05
        final['ValueABSNoPeg'] = final['PeerRank']*0.05 + final['AbsRank']*0.95        

        result = list()

        # Iterate over each volatility-related column
        for col in ["ValueYieldNoPeg", "ValueABSNoPeg"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = final.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))


        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        ## Store the Factor Data in dictionary.
        self.factorData["ValueYieldNoPeg"] = scores.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ValueYieldNoPeg"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### ValueYieldNoPEG COMPLETE ###")

    def generate_ValueYieldExDiv(self):

        ## Fetch the Stock Value Data
        if self.stockValueData is None:
            self.__readValueData()

        ## Fetch the 
        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        
        ### VALUE ###
        # Filter the rows for Top-500 companies historically
        valueData = self.stockValueData[self.stockValueData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].copy()

        ## List of Fundamental Factor Value.
        fundamentalValueFactor = ["EV_EBITDA", "PS", "PB", "PE"]
        ## Replace -ve value with NaN value for all the fundamental value factor
        for factor in fundamentalValueFactor:
            valueData[factor] = np.where(valueData[factor] <= 0, np.nan, valueData[factor])
            valueData[factor] = 1/valueData[factor]

        ## Sort and reset the index
        valueData.sort_values(["Symbol", "Date"], inplace = True)
        valueData.reset_index(drop = True, inplace = True)


        ### Dividend ###
        dividendData = self.generate_Dividend(valueYieldCall = True)

        ## EPS ###
        epsData = self.generate_Growth(valueYieldCall = True)

        ## ROE
        roe = self.generate_QualityQuarter(valueYieldCall=True)
        roe["Date_End"] = pd.to_datetime(roe['Date_End'],format="%Y%m")

        growth = pd.merge(epsData[["Date", "Date_End", "Symbol","EPS_DILUTED_change"]], 
                          roe[["Date_End", "Symbol", "ROE_ttm"]], on = ["Date_End", "Symbol"], how = "left")

        final=pd.merge(valueData,dividendData,on=['Date','Symbol'])
        final=pd.merge(final,growth,on=['Date','Symbol'],how='left')

        final = final.sort_values(["Symbol","Date"]).reset_index(drop = True)
        final[['ROE_ttm', "EPS_DILUTED_change"]]=final.groupby('Symbol')[['ROE_ttm', "EPS_DILUTED_change"]].ffill()
        final['PEG']=final['PE']/final['EPS_DILUTED_change']

        factorUniverse = ['EV_EBITDA', 'PS','PB', 'PE','PEG','ROE_ttm']

        final = pd.merge(self.strategyUniverse, final[["Date", "Symbol"] + factorUniverse], on = ["Date","Symbol"], how = "left")

        final["AbsRank"] = final.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        final["PeerRank"] = final.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        final['ValueYieldExDiv'] = final['PeerRank']*0.95 + final['AbsRank']*0.05
        final['ValueABSExDiv'] = final['PeerRank']*0.05 + final['AbsRank']*0.95        

        result = list()

        # Iterate over each volatility-related column
        for col in ["ValueYieldExDiv", "ValueABSExDiv"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = final.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))


        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        ## Store the Factor Data in dictionary.
        self.factorData["ValueYieldExDiv"] = scores.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ValueYieldExDiv"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### ValueYieldExDiv COMPLETE ###")

    def generate_ShortAM(self,):

        # Function to calculate log returns
        def calculate_log_returns(df):
            df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
            return df.dropna()

        # Function to calculate annualized standard deviation
        def calculate_annualized_std(df, window=252):
            return df['LogReturn'].rolling(window).std() * np.sqrt(window)

        # Function to calculate momentum ratios
        def calculate_momentum_ratios(series, period):
            return series / series.shift(period) - 1

        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Define periods for momentum ratios (in trading days)
        periods = {
            'MR1': 21,   # 1 month
            'MR2': 42,   # 2 months
            'MR3': 63,   # 3 months
        }

        # Apply log return calculation
        priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
    
        # Calculate momentum ratios for each period
        for label, period in periods.items():
            priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

        # Calculate annualized standard deviation
        priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

        # Normalize the momentum ratios by dividing by the annualized standard deviation
        for label in periods.keys():
            priceData[label] /= priceData['AnnualizedStd']

        priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)

        # Calculate the mean and std deviation of each momentum ratio across the universe
        for label in periods.keys():
            priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
            priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())

        # Calculate Z-scores for each period
        for label in periods.keys():
            priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']

        # Define specific combinations for which to calculate the final Z-scores
        metrics = ['Z_MR1', 'Z_MR2', 'Z_MR3']

        # Weighted average Z-score
        priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)

        # Normalized momentum score
        priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                1 + priceData[f'WtdZScore'],
                                                (1 - priceData[f'WtdZScore']) ** -1)

        ## Filter the Required Columns  
        priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Computing the Percentile Score of the Symbols on each Date
        priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())

        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)

        ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()

        # Iterate over each volatility-related column
        for col in ["Momentum", "PeerMomentum"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)
        scores.rename(columns = {"AM" : "ShortAM", f"{self.peer}AM" : f"Short{self.peer}AM"}, inplace = True)

        self.x = scores.copy()

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["ShortAM"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ShortAM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### SHORT MOMENTUM COMPLETE ###")

        del priceData, result, temp, scores
    
    def generate_MidAM(self,):

        # Function to calculate log returns
        def calculate_log_returns(df):
            df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
            return df.dropna()

        # Function to calculate annualized standard deviation
        def calculate_annualized_std(df, window=252):
            return df['LogReturn'].rolling(window).std() * np.sqrt(window)

        # Function to calculate momentum ratios
        def calculate_momentum_ratios(series, period):
            return series / series.shift(period) - 1

        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Define periods for momentum ratios (in trading days)
        periods = {
            'MR3': 63,   # 3 months
            'MR6': 6*22,   # 3 months
        }

        # Apply log return calculation
        priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
    
        # Calculate momentum ratios for each period
        for label, period in periods.items():
            priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

        # Calculate annualized standard deviation
        priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

        # Normalize the momentum ratios by dividing by the annualized standard deviation
        for label in periods.keys():
            priceData[label] /= priceData['AnnualizedStd']

        priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)

        # Calculate the mean and std deviation of each momentum ratio across the universe
        for label in periods.keys():
            priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
            priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())

        # Calculate Z-scores for each period
        for label in periods.keys():
            priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']

        # Define specific combinations for which to calculate the final Z-scores
        metrics = ['Z_MR3', 'Z_MR6']

        # Weighted average Z-score
        priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)

        # Normalized momentum score
        priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                1 + priceData[f'WtdZScore'],
                                                (1 - priceData[f'WtdZScore']) ** -1)

        ## Filter the Required Columns  
        priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Computing the Percentile Score of the Symbols on each Date
        priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())

        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)

        ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()

        # Iterate over each volatility-related column
        for col in ["Momentum", "PeerMomentum"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)
        scores.rename(columns = {"AM" : "MidAM", f"{self.peer}AM" : f"Mid{self.peer}AM"}, inplace = True)

        self.x = scores.copy()

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["MidAM"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["MidAM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### MID MOMENTUM COMPLETE ###")

        del priceData, result, temp, scores

    def generate_LongAM(self,):

            # Function to calculate log returns
            def calculate_log_returns(df):
                df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
                return df.dropna()

            # Function to calculate annualized standard deviation
            def calculate_annualized_std(df, window=252):
                return df['LogReturn'].rolling(window).std() * np.sqrt(window)

            # Function to calculate momentum ratios
            def calculate_momentum_ratios(series, period):
                return series / series.shift(period) - 1

            ################
            ## PRICE DATA ##
            ################
            # Check if stock price data is loaded; if not, read it
            if self.stockPriceData is None:
                self.__readPriceData()

            ## Drop the Unnecessary COlumns
            priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
            
            ## Filter the Symbol which are present in strategy universe
            priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

            # Define periods for momentum ratios (in trading days)
            periods = {
                'MR12': 252,   # 12 months
                'MR6': 6*22,   # 6 months
            }

            # Apply log return calculation
            priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
        
            # Calculate momentum ratios for each period
            for label, period in periods.items():
                priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

            # Calculate annualized standard deviation
            priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

            # Normalize the momentum ratios by dividing by the annualized standard deviation
            for label in periods.keys():
                priceData[label] /= priceData['AnnualizedStd']

            priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)

            # Calculate the mean and std deviation of each momentum ratio across the universe
            for label in periods.keys():
                priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
                priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())

            # Calculate Z-scores for each period
            for label in periods.keys():
                priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']

            # Define specific combinations for which to calculate the final Z-scores
            metrics = ['Z_MR12', 'Z_MR6']

            # Weighted average Z-score
            priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)

            # Normalized momentum score
            priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                    1 + priceData[f'WtdZScore'],
                                                    (1 - priceData[f'WtdZScore']) ** -1)

            ## Filter the Required Columns  
            priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])

            ## Merge the Computed Metrics against the symbol on each day's universe
            priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

            ## Computing the Percentile Score of the Symbols on each Date
            priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

            ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
            priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())

            ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
            priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)

            ## Filter the data after year 2006
            priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
            
            # Initialize an empty list to store the transformed data for each volatility column
            result = list()

            # Iterate over each volatility-related column
            for col in ["Momentum", "PeerMomentum"]:

                # Filter the dataframe to keep only the "Symbol", "Date", and current column
                temp = priceData.filter(["Symbol", "Date", col])

                # Pivot the table to have dates as rows and symbols as columns, with values from the current column
                temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

                # Shift the "Date" column up by one row to align data with the next date
                temp["Date"] = temp["Date"].shift(-1)

                # Set the last row's "Date" to today's date to capture current data
                temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

                # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
                temp = temp.melt(id_vars='Date', value_name=col)

                # Drop rows with missing values and reset the index
                temp = temp.dropna().reset_index(drop=True)

                # Filter data to include only records from January 1, 2006, onwards and reset the index
                temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

                # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
                result.append(temp.set_index(["Date", "Symbol"]))

            
            # Concatenate all transformed columns along the columns axis to create a combined dataframe
            scores = pd.concat(result, axis=1).reset_index()
            scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)
            scores.rename(columns = {"AM" : "LongAM", f"{self.peer}AM" : f"Long{self.peer}AM"}, inplace = True)

            self.x = scores.copy()

            # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
            scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

            ## Store the Factor Data in dictionary.
            self.factorData["LongAM"] = scores.copy()

            ## Filter the data for latest update or values.
            self.recentFactorUpdate["LongAM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
            
            ## Combining the Factors in one Dataframe
            self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
            self.stockFactors.reset_index(inplace = True)

            print("### LONG MOMENTUM COMPLETE ###")

            del priceData, result, temp, scores

    def generate_MultiAM(self,):

            # Function to calculate log returns
            def calculate_log_returns(df):
                df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
                return df.dropna()

            # Function to calculate annualized standard deviation
            def calculate_annualized_std(df, window=252):
                return df['LogReturn'].rolling(window).std() * np.sqrt(window)

            # Function to calculate momentum ratios
            def calculate_momentum_ratios(series, period):
                return series / series.shift(period) - 1

            ################
            ## PRICE DATA ##
            ################
            # Check if stock price data is loaded; if not, read it
            if self.stockPriceData is None:
                self.__readPriceData()

            ## Drop the Unnecessary COlumns
            priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
            
            ## Filter the Symbol which are present in strategy universe
            priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

            # Define periods for momentum ratios (in trading days)
            periods = {
                'MR3': 63,   # 3 months
                'MR6': 6*22,   # 3 months
            }

            # Apply log return calculation
            priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
        
            # Calculate momentum ratios for each period
            for label, period in periods.items():
                priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

            # Calculate annualized standard deviation
            priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

            # Normalize the momentum ratios by dividing by the annualized standard deviation
            for label in periods.keys():
                priceData[label] /= priceData['AnnualizedStd']

            priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)

            # Calculate the mean and std deviation of each momentum ratio across the universe
            for label in periods.keys():
                priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
                priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())

            # Calculate Z-scores for each period
            for label in periods.keys():
                priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']

            # Define specific combinations for which to calculate the final Z-scores
            metrics = ['Z_MR3', 'Z_MR6']

            # Weighted average Z-score
            priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)

            # Normalized momentum score
            priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                    1 + priceData[f'WtdZScore'],
                                                    (1 - priceData[f'WtdZScore']) ** -1)

            ## Filter the Required Columns  
            priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])

            ## Merge the Computed Metrics against the symbol on each day's universe
            priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

            ## Computing the Percentile Score of the Symbols on each Date
            priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

            ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
            priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())

            ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
            priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)

            ## Filter the data after year 2006
            priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
            
            # Initialize an empty list to store the transformed data for each volatility column
            result = list()

            # Iterate over each volatility-related column
            for col in ["Momentum", "PeerMomentum"]:

                # Filter the dataframe to keep only the "Symbol", "Date", and current column
                temp = priceData.filter(["Symbol", "Date", col])

                # Pivot the table to have dates as rows and symbols as columns, with values from the current column
                temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

                # Shift the "Date" column up by one row to align data with the next date
                temp["Date"] = temp["Date"].shift(-1)

                # Set the last row's "Date" to today's date to capture current data
                temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

                # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
                temp = temp.melt(id_vars='Date', value_name=col)

                # Drop rows with missing values and reset the index
                temp = temp.dropna().reset_index(drop=True)

                # Filter data to include only records from January 1, 2006, onwards and reset the index
                temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

                # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
                result.append(temp.set_index(["Date", "Symbol"]))

            
            # Concatenate all transformed columns along the columns axis to create a combined dataframe
            scores = pd.concat(result, axis=1).reset_index()
            scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)
            scores.rename(columns = {"AM" : "MidAM", f"{self.peer}AM" : f"Mid{self.peer}AM"}, inplace = True)

            self.x = scores.copy()

            # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
            scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

            ## Store the Factor Data in dictionary.
            self.factorData["MidAM"] = scores.copy()

            ## Filter the data for latest update or values.
            self.recentFactorUpdate["MidAM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
            
            ## Combining the Factors in one Dataframe
            self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
            self.stockFactors.reset_index(inplace = True)

            print("### MID MOMENTUM COMPLETE ###")

            del priceData, result, temp, scores

    def generate_UltraShortAM(self,):
 
        # Function to calculate log returns
        def calculate_log_returns(df):
            df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
            return df.dropna()
 
        # Function to calculate annualized standard deviation
        def calculate_annualized_std(df, window=252):
            return df['LogReturn'].rolling(window).std() * np.sqrt(window)
 
        # Function to calculate momentum ratios
        def calculate_momentum_ratios(series, period):
            return series / series.shift(period) - 1
 
        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()
 
        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)
 
        # Define periods for momentum ratios (in trading days)
        periods = {
            'MR4': 5*4,   # 4 Weeks
            'MR2': 5*2,   # 2 Weeks
            'MR1': 5*1,   # 1 Weeks
        }
 
        # Apply log return calculation
        priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
    
        # Calculate momentum ratios for each period
        for label, period in periods.items():
            priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))
 
        # Calculate annualized standard deviation
        priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)
 
        # Normalize the momentum ratios by dividing by the annualized standard deviation
        for label in periods.keys():
            priceData[label] /= priceData['AnnualizedStd']
 
        priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)
 
        # Calculate the mean and std deviation of each momentum ratio across the universe
        for label in periods.keys():
            priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
            priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())
 
        # Calculate Z-scores for each period
        for label in periods.keys():
            priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']
 
        # Define specific combinations for which to calculate the final Z-scores
        metrics = ['Z_MR1', 'Z_MR2', "Z_MR4"]
 
        # Weighted average Z-score
        priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)
 
        # Normalized momentum score
        priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                1 + priceData[f'WtdZScore'],
                                                (1 - priceData[f'WtdZScore']) ** -1)
 
        ## Filter the Required Columns  
        priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])
 
        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")
 
        ## Computing the Percentile Score of the Symbols on each Date
        priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)
 
        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())
 
        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)
 
        ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()
 
        # Iterate over each volatility-related column
        for col in ["Momentum", "PeerMomentum"]:
 
            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])
 
            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()
 
            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)
 
            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())
 
            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)
 
            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)
 
            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()
 
            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))
 
        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)
        scores.rename(columns = {"AM" : "UltraShortAM", f"{self.peer}AM" : f"UltraShort{self.peer}AM"}, inplace = True)
 
        self.x = scores.copy()
 
        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)
 
        ## Store the Factor Data in dictionary.
        self.factorData["UltraShortAM"] = scores.copy()
 
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["UltraShortAM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)
 
        print("### ULTRA SHORT MOMENTUM COMPLETE ###")
 
        del priceData, result, temp, scores

    def generate_Beta(self):
        
        ################
        ## PRICE DATA ##
        ################
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        # priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)
        # priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        # data = consol_factor.stockPriceData.copy()
        priceData.rename(columns={'Close': 'Price'}, inplace=True)
        
        # nifty = d.fetch_data_from_database(table_name='Nifty', no_of_years=25)
        nifty = pd.read_csv('Nifty.csv', parse_dates=['Date'])
        nifty.drop(columns={'NIFTY'}, inplace=True)
        nifty.rename(columns={'NIFTY500': 'Nifty'}, inplace=True)

        bench1 = nifty.copy()
        bench1["BenchReturn"] = bench1["Nifty"].pct_change()
        
        priceData = pd.merge(priceData, bench1, on = ["Date"], how = "left")

        def rollBeta(group, window = 252):
        
            group["Change"] = group["Price"].pct_change()
            group["1Y_Beta"] = group["Change"].rolling(window = window).cov(group["BenchReturn"]) / group["BenchReturn"].rolling(window = window).var()
        
            return group
        
        priceData = priceData.groupby("Symbol").apply(rollBeta, window = 250, include_groups=False).reset_index().drop(columns='level_1')
        priceData = priceData[["Date", "Symbol", "1Y_Beta"]]
        # df_beta = beta.copy()
        # beta_latest = df_beta[df_beta['Date'] == df_beta['Date'].max()].reset_index(drop=True)
        # beta_latest = beta_latest[['Symbol', '1Y_Beta']]
        # df_beta = df_beta[['Date', 'Symbol', '1Y_Beta']]
        # df_beta = date_shift(df_beta)

        # appender_final = pd.merge(appender, df_beta, how='left', on=['Date', 'Symbol'])
        
        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        #high beta, low beta
        # print(priceData.groupby('Date')['Symbol'].count().tail(10))
        priceData['HighBeta'] = priceData.groupby('Date')['1Y_Beta'].rank(pct=True)
        priceData['LowBeta'] = 1- priceData['HighBeta']
        priceData.drop(columns=['1Y_Beta'], inplace=True)

         ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each beta column
        result = list()

        # Iterate over each beta-related column
        for col in ["HighBeta", "LowBeta"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        # scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["Beta"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["Beta"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### BETA COMPLETE ###")

        del priceData, result, temp, scores
        

        # ## Filter the data after year 2006 and sor the dataframe.
        # priceData = priceData[priceData["Date"] >= "2006-01-01"].copy()
        # priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
    
        # ## Convert the long from Dataframe to wide form dataframe
        # priceData = priceData.pivot_table(index='Date',columns='Symbol',values='AM_New').reset_index()
        # ## Shifting the Date column
        # priceData["Date"] = priceData["Date"].shift(-1)
        # ## Fill NaN value of date with todays date.
        # priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())
        # ## Re-converting the wide form dataframe to long form dataframe
        # priceData = priceData.melt(id_vars='Date', value_name= "EM")
        # ## Drop na and reset the index
        # priceData = priceData.dropna().reset_index(drop = True)

        # ## Store the Factor Data in dictionary.
        # self.factorData["EM"] = priceData.copy()
        # ## Filter the data for latest update or values.
        # self.recentFactorUpdate["EM"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        # ## Combining the Factors in one Dataframe
        # self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        # self.stockFactors.reset_index(inplace = True)

        # print("### EM COMPLETE ###")

        # del priceData

    def generate_TrendAntitrendMR(self):
        ################
        ## PRICE DATA ##
        ################
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume"]).copy()

        price_data = priceData.copy()
        price_data_500 = price_data.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
        price_data_500 = price_data[price_data['Symbol'].isin(price_data_500['Symbol'])]
        # theme_df = pd.read_excel('SectorMapping.xlsx')
        # theme_df = d.fetch_data_from_database(table_name = 'SectorThemeGics')
        theme_df = pd.read_csv('gics.csv')
        theme_df = theme_df[['Symbol', 'Theme']]  # Ensure only relevant columns are kept
        df = price_data_500[['Date', 'Symbol', 'Close', 'Mcap']].merge(theme_df, on='Symbol', how='inner')
        df['Theme'] = df['Theme'].fillna('Others')
        df.set_index('Date', inplace=True)

        # Function to calculate log returns
        def calculate_log_returns(df):
            df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
            return df.dropna()

        # Function to calculate annualized standard deviation
        def calculate_annualized_std(df, window=252):
            return df['LogReturn'].rolling(window).std() * np.sqrt(window)

        # Function to calculate momentum ratios
        def calculate_momentum_ratios(series, period):
            return series / series.shift(period) - 1

        # Define periods for momentum ratios (in trading days)
        periods = {
            'MR1': 21,   # 1 month
            'MR3': 63,   # 3 months
            'MR6': 126,  # 6 months
            'MR12': 252, # 12 months
        }
        # Apply log return calculation
        df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns)

        # Calculate momentum ratios for each period
        for label, period in periods.items():
            df[label] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

        # Calculate annualized standard deviation
        df['AnnualizedStd'] = df.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

        # Normalize the momentum ratios by dividing by the annualized standard deviation
        for label in periods.keys():
            df[label] /= df['AnnualizedStd']

        # Reset index
        df = df.reset_index()

        # Calculate the mean and std deviation of each momentum ratio across the universe
        for label in periods.keys():
            df[f'mu_{label}'] = df.groupby('Date')[label].transform(lambda x: x.mean())
            df[f'sigma_{label}'] = df.groupby('Date')[label].transform(lambda x: x.std())

        # Calculate Z-scores for each period
        for label in periods.keys():
            df[f'Z_{label}'] = (df[label] - df[f'mu_{label}']) / df[f'sigma_{label}']

        #For short, medium and long run cells 1,2,3 and the last cell(this one)

        # Define the combinations for different momentum strategies
        momentum_strategies = {
            'MomentumShort': ['MR1', 'MR3'],
            'MomentumLong': ['MR6', 'MR12']
        }

        # Calculate weighted average Z-scores and normalized momentum scores for each strategy
        for strategy_name, combination in momentum_strategies.items():
            # Create Z-score column labels
            z_columns = [f'Z_{label}' for label in combination]
            
            # Calculate equal weights
            weights = np.ones(len(z_columns)) / len(z_columns)
            
            # Calculate weighted average Z-score
            df[f'WeightedAvgZ_{strategy_name}'] = df[z_columns].dot(weights)
            
            # Calculate normalized momentum score
            df[f'Score_{strategy_name}'] = np.where(
                df[f'WeightedAvgZ_{strategy_name}'] >= 0,
                1 + df[f'WeightedAvgZ_{strategy_name}'],
                (1 - df[f'WeightedAvgZ_{strategy_name}']) ** -1
            )

        # Process each strategy and create CSV files
        for strategy_name in momentum_strategies.keys():
            # Create a copy of the dataframe with only relevant columns
            strategy_df = df.copy()
            
            # Filter for top 500 stocks by market cap
            strategy_df = strategy_df.groupby('Date', group_keys=False).apply(
                lambda x: x.sort_values(by='Mcap', ascending=False).head(500)
            )
        strategy_df = strategy_df[['Date', 'Symbol', 'Score_MomentumShort', 'Score_MomentumLong']]
        strategy_df = strategy_df.groupby('Date', group_keys=False).apply(
            lambda x: x.assign(
                TrendMR=x['Score_MomentumLong'].rank(pct=True, method='min', ascending=False),
                Short_Rank=x['Score_MomentumShort'].rank(pct=True, method='min', ascending=True),
                Long_Rank=x['Score_MomentumLong'].rank(pct=True, method='min', ascending=True)
            ).assign(
                AntiTrend_MR=lambda df: (df['Short_Rank'] - df['Long_Rank']) / 2,
                AntiTrendMR=lambda df: df['AntiTrend_MR'].rank(pct=True, method='min', ascending=False)  # Lower scores get higher ranks
            )
        )
        strategy_df = strategy_df[['Date','Symbol','TrendMR','AntiTrendMR']].reset_index(drop=True)
        # strategy_df = date_shift(strategy_df)
        # return strategy_df

        # trend_antitrend_MR_df = trend_antitrend_MR(consol_factor.stockPriceData.copy())
        # latest_trend_mr_df = strategy_df[strategy_df['Date'] == strategy_df['Date'].max()].reset_index(drop=True)
        # strategy_df = strategy_df[['Date', 'Symbol','TrendMR','AntiTrendMR']].reset_index(drop=True)
        # appender_final = pd.merge(appender_final, trend_antitrend_MR_df, how='left', on=['Date', 'Symbol'])

        priceData = strategy_df.copy()

         ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each beta column
        result = list()

        # Iterate over each beta-related column
        for col in ["TrendMR", "AntiTrendMR"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        # scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["Trend"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["Trend"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### TREND COMPLETE ###")

        del priceData, result, temp, scores, strategy_df
        
    def generate_ShiftedAM(self):
        ################
        ## PRICE DATA ##
        ################
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        top_1000 =  priceData.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
        price_data_500 = priceData[priceData['Symbol'].isin(top_1000['Symbol'])]
        df = top_1000[['Date', 'Symbol', 'Close', 'Mcap']]
        df.set_index('Date', inplace=True)

        # Shift the Close price by 21 days to get the starting point for the annualized standard deviation calculation
        df['Close_shifted'] = df.groupby('Symbol')['Close'].shift(21)

        # Function to calculate log returns based on the shifted Close (21 days back)
        def calculate_log_returns_shifted(df):
            df['LogReturn_shifted'] = np.log(df['Close_shifted'] / df['Close_shifted'].shift(1))
            return df

        # Apply the shifted log return calculation for each symbol group
        df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns_shifted)

        # Function to calculate annualized standard deviation over a 252-day rolling window from the 21-day shifted close
        def calculate_annualized_std_shifted(df):
            df['AnnualizedStd_Shifted'] = df['LogReturn_shifted'].rolling(window=252).std() * np.sqrt(252)
            return df

        # Calculate annualized standard deviation based on the 21-day shifted close prices for each symbol group
        df = df.groupby('Symbol', group_keys=False).apply(calculate_annualized_std_shifted)

        # Drop any NaN values generated from the shifting process
        # df = df.dropna(subset=['AnnualizedStd_Shifted'])

        # Display the DataFrame
        df.head()

        # Define custom momentum calculation based on rebalancing months with custom intervals
        def calculate_custom_momentum_intervals(df, label, start_month, end_month):
            shifted_df = df.copy()
            
            # Calculate close prices shifted by the specified start and end months
            shifted_df[f'{label}_start'] = shifted_df.groupby('Symbol')['Close'].shift(start_month * 21)  # T-1 month back
            shifted_df[f'{label}_end'] = shifted_df.groupby('Symbol')['Close'].shift(end_month * 21)      # T-7 or T-13 months back
            
            # Calculate the momentum ratio based on specified months and normalize by annualized standard deviation
            shifted_df[label] = (
                (shifted_df[f'{label}_start'] / shifted_df[f'{label}_end'] - 1) / shifted_df['AnnualizedStd_Shifted']
            )

            return shifted_df
            # return shifted_df.dropna()  # Keep shifted columns by not dropping them here

        # Define pairs for each momentum ratio (start and end month)
        momentum_intervals = {'MR6': (1, 7), 'MR12': (1, 13)}

        # Calculate custom momentum ratios based on the specified intervals
        for label, (start_month, end_month) in momentum_intervals.items():
            df = calculate_custom_momentum_intervals(df, label, start_month, end_month)

        # Reset index for further calculations
        df = df.reset_index()

        # Calculate the mean and standard deviation of each momentum ratio across the universe on each date
        for label in momentum_intervals.keys():
            df[f'mu_{label}'] = df.groupby('Date')[label].transform('mean')
            df[f'sigma_{label}'] = df.groupby('Date')[label].transform('std')

        # Calculate Z-scores for each period
        for label in momentum_intervals.keys():
            df[f'Z_{label}'] = (df[label] - df[f'mu_{label}']) / df[f'sigma_{label}']
        comb_labels = ['Z_MR6', 'Z_MR12'] 
        comb_weights = [0.5, 0.5] 
        # comb_labels = ['Z_MR6'] 
        # comb_weights = [1] 
        # Calculate weighted average Z-score based on combination of momentum scores
        df['WeightedAvgZ_comb'] = df[comb_labels].dot(comb_weights)
        # Calculate Normalized Momentum Score based on Weighted Average Z-score
        df['NormalizedMomentumScore'] = np.where(
            df['WeightedAvgZ_comb'] >= 0,
            1 + df['WeightedAvgZ_comb'],
            (1 - df['WeightedAvgZ_comb']) ** -1
        )

        # Filter for the top 1000 stocks by market cap on each date
        df = df.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))

        # Reset index after grouping
        df = df.reset_index(drop=True)
        momdf = df[['Date','Symbol','Mcap','NormalizedMomentumScore']]
        momdf['Shifted_Mom_Rank'] = df.groupby('Date')['NormalizedMomentumScore'].rank(pct=True, ascending=True)
        momdf = momdf.sort_values(by=['Date','Shifted_Mom_Rank'], ascending=[True,False])
        momdf['Date'] = momdf.groupby('Symbol')['Date'].shift(-1)
        momdf = momdf.dropna(subset=['Date'])
        momdf = momdf[['Date','Symbol', 'Shifted_Mom_Rank']]
        momdf = momdf.dropna()
        momdf.rename(columns={'Shifted_Mom_Rank': 'ShiftedAM'}, inplace=True)

        ## Filter the data after year 2006 and sor the dataframe.
        priceData = momdf.copy()
        priceData = priceData[priceData["Date"] >= "2006-01-01"].copy()
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
    
        ## Convert the long from Dataframe to wide form dataframe
        priceData = priceData.pivot_table(index='Date',columns='Symbol',values='ShiftedAM').reset_index()
        ## Shifting the Date column
        priceData["Date"] = priceData["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())
        ## Re-converting the wide form dataframe to long form dataframe
        priceData = priceData.melt(id_vars='Date', value_name= "ShiftedAM")
        ## Drop na and reset the index
        priceData = priceData.dropna().reset_index(drop = True)

        ## Store the Factor Data in dictionary.
        self.factorData["ShiftedAM"] = priceData.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ShiftedAM"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### SHIFTED AM COMPLETE ###")

        del priceData, momdf, df, top_1000, price_data_500

    def generate_ValuePrice(self):
        ################
        ## PRICE DATA ##
        ################
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        # print(priceData.columns)
        # 52 week low
        price_data = priceData.copy()
        # price_data.rename(columns={'Price': 'Close'}, inplace=True)
        price_data = price_data[['Date', 'Symbol', 'Close']].reset_index(drop=True)

        def calculate_distance52Low(df):  
            df['52W_Low'] = df.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=252, min_periods=1).min())
            df['percentage_change_52WLow'] = ((df['52W_Low'] / df['Close']) - 1).mul(100)
            # df = df[df['Date'] == df['Date'].iloc[-1]].reset_index(drop=True)
            df = df[['Date', 'Symbol', 'percentage_change_52WLow']]
            return df

        priceData = calculate_distance52Low(price_data.copy(deep=True))
        # latest_52WLow = date_shift(latest_52WLow)

        # print(priceData.columns)
        # priceData = latest_52WLow.copy()
        # ValuePrice
        # appender_final = pd.merge(appender_final, latest_52WLow, how='left', on=['Date', 'Symbol'])
        priceData['ValuePrice'] = priceData.groupby('Date')['percentage_change_52WLow'].rank(pct=True, ascending=True)
        # appender_final['ValuePrice'] = appender_final['percentage_change_52WLow'].rank(pct=True, ascending=True)
        priceData.drop(columns={'percentage_change_52WLow'}, inplace=True)

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Computing the Percentile Score of the Symbols on each Date
        # priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["SectorValuePrice"] = priceData.groupby(["Date", "Sector"])["ValuePrice"].transform(lambda x: x.mean())

        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["SectorValuePrice"] = priceData.groupby(["Date"])["SectorValuePrice"].rank(ascending = True, pct = True)

        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["ThemeValuePrice"] = priceData.groupby(["Date", "Theme"])["ValuePrice"].transform(lambda x: x.mean())

        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["ThemeValuePrice"] = priceData.groupby(["Date"])["ThemeValuePrice"].rank(ascending = True, pct = True)
        
         ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each beta column
        result = list()

        # Iterate over each beta-related column
        for col in ["ValuePrice", "SectorValuePrice", "ThemeValuePrice"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        # scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["ValuePrice"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ValuePrice"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### VALUEPRICE COMPLETE ###")

        del priceData, result, temp, scores

    def generate_3DQuality(self):

        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossGrowth is None:
            self.__readProfitLossGrowth()

        if self.financeBalanceSheet is None:
            self.__readFinanceBalanceSheet()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()


        price_data = self.stockPriceData.copy()
        top_500 = self.strategyUniverse.copy()

        # mapping = pd.read_excel("company_master_mapping.xlsx")
        # price_data = d.fetch_price_data(no_of_years=20)
        # price_data = pd.read_csv('stockPriceData-3.csv')
        # Convert 'Date' to datetime type
        # price_data['Date'] = pd.to_datetime(price_data['Date'])
        # Create a dictionary mapping from SYMBOL_NSE to SYMBOL_CM
        # mapping = dict(zip(mapping["SYMBOL_NSE"], mapping["SYMBOL_CM"]))
        # price_data["Symbol"] = price_data["Symbol"].replace(mapping)
        # top_500 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
        # Filter price data for dates after '2006-01-01' and drop duplicates
        # price_data.drop_duplicates(['Date', 'Symbol'], inplace=True)

        #company master swapping
        # company_master = d.fetch_data_from_database(table_name='Companymaster')
        # company_master = pd.read_csv('Companymaster_3_2_2025.csv')
        # company_master["SYMBOL"] = company_master["SYMBOL"].replace(mapping)
        company_master = self.companyMaster.copy()
        company_master.columns = company_master.columns.str.upper()
        # yearly = pd.read_csv('Finance_pl_3_2_2025.csv')
        # Quaterly = pd.read_csv('Quarterly_3_2_2025.csv')
        # Finance_bs =  pd.read_csv('Finance_bs_3_2_2025.csv')[[ 'Year_end', 'Fincode', 'Share_Capital', 'Reserve']]
        Finance_bs = self.financeBalanceSheet[["Year_end", "FINCODE", "Share_Capital", "Reserve"]].copy()
        Finance_bs.rename(columns={'FINCODE': 'Fincode'}, inplace=True)
        # rebal_dates = self.__dataCursor.fetch_data_from_database(table_name="GrowthDate", no_of_years=50)
        rebal_dates = pd.read_csv('growth_date_latest.csv')
        rebal_dates["Date"] = pd.to_datetime(rebal_dates["Date"])
        rebal=rebal_dates['Date']
        Rebalance_Qtr=list(rebal_dates['Quarter'])
        Rebalance_Date=list(rebal_dates['Date'])
        Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))
        date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))
        top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)
        index_constituents_data= top_500
        # X = Quaterly[Quaterly['Result_Type'] == 'Q']
        # print(X.columns)
        # Calculating Networth
        # X = self.profitLossGrowth[["Date_End", "FINCODE", "PAT"]].copy()
        X = self.profitLossGrowth.copy()
        X.rename(columns={'FINCODE': 'Fincode'}, inplace=True)

        Finance_bs['NetWorth'] = Finance_bs['Share_Capital'] + Finance_bs['Reserve']
        Finance_bs2 = Finance_bs[['Year_end', 'Fincode', 'NetWorth']]
        Finance_bs3 = pd.merge(Finance_bs2, company_master[['FINCODE', 'SYMBOL']], left_on='Fincode', right_on='FINCODE')
        Finance_bs3 = Finance_bs3[['Year_end', 'FINCODE', 'SYMBOL', 'NetWorth']]

        # Revisd PAT in units
        pat_df= X[['Date_End', 'Fincode', 'PAT']]
        pat_df['PAT'] = pat_df['PAT']/10

        # Calculating Networth on quarterly basis
        p =pd.merge(pat_df, Finance_bs3, left_on=['Date_End','Fincode'], right_on=['Year_end','FINCODE'], how='left')
        p['SYMBOL'] = p.groupby('Fincode', group_keys=False)['SYMBOL'].apply(lambda x: x.fillna(method='ffill'))
        p = p.drop(columns=['Year_end', 'FINCODE'])

        # Quarterly Networth Calculation
        def fill_networth(row, prev_networth):
            if pd.isna(row['NetWorth']):
                return prev_networth + row['PAT']
            else:
                return row['NetWorth']

        def fill_networth_column(df):
            df = df.sort_values(by=['Fincode', 'Date_End'])
            df['NetWorth_Filled'] = np.nan
            for fincode in df['Fincode'].unique():
                prev_networth = None
                for i, row in df[df['Fincode'] == fincode].iterrows():
                    if prev_networth is None:
                        prev_networth = row['NetWorth']
                    else:
                        df.at[i, 'NetWorth_Filled'] = fill_networth(row, prev_networth)
                        prev_networth = df.at[i, 'NetWorth_Filled']
            return df

        # Apply the function
        p = fill_networth_column(p)
        # Fill any remaining NaN in the original NetWorth column with the filled values
        p['NetWorth'] = p['NetWorth'].combine_first(p['NetWorth_Filled'])
        p.drop(columns=['NetWorth_Filled'], inplace=True)
        p = p.dropna()

        # Calculating ROE from scratch
        p['ROE'] = p.groupby('SYMBOL', group_keys=False).apply(lambda x: x['PAT']/x['NetWorth'])
        p['ROE_ttm'] = p.groupby('SYMBOL', group_keys=False)['ROE'].apply(lambda x: x.rolling(window=4).sum())
        roe_ttm = p[['Fincode', 'Date_End', 'ROE_ttm']]

        quality = X[['Fincode','Date_End','Debt/Equity Ratio', 'Adj_eps_abs']]
        quality = pd.merge(quality, roe_ttm, on=['Fincode', 'Date_End'])
        pl_quality = pd.merge(quality, company_master, how='inner', left_on='Fincode', right_on='FINCODE')
        a = pl_quality[['Fincode', 'Date_End', 'Debt/Equity Ratio', 'ROE_ttm', 'SYMBOL', 'Adj_eps_abs']].reset_index(drop=True)

        # Removing the rows with NaN Symbol Names
        a = a[a.SYMBOL.notna()]

        # Filtering Stocks which have been historically in top 500 universe only.
        b = a[a.SYMBOL.isin(top_500.Symbol.unique())]

        # Filter out stocks with fewer than 4 quarters of history
        symbol_quarter_counts = b.groupby('SYMBOL').size()
        valid_symbols = symbol_quarter_counts[symbol_quarter_counts >= 4].index
        b = b[b['SYMBOL'].isin(valid_symbols)]

        # Define a function to check for 2 consecutive quarters of negative EPS within a 1-year period
        def has_2_consecutive_negative_in_last_1_year(series):
            return series.rolling(window=4, min_periods=4).apply(
                lambda x: any((x[i] < 0 and x[i + 1] < 0) for i in range(len(x) - 1)), raw=True
            )

        # Apply the function to create a flag for exclusion
        b['exclude_flag'] = b.groupby('SYMBOL')['Adj_eps_abs'].transform(has_2_consecutive_negative_in_last_1_year)
        # Filter out rows based on the exclude flag
        b_filtered = b[b['exclude_flag'] != 1].reset_index(drop=True)
        # Drop the intermediate column used for filtering
        b_filtered = b_filtered.drop(columns=['exclude_flag'])
        # Calculate Adjusted EPS TTM
        b_filtered['Adj_eps_abs_TTM'] = b_filtered.groupby('SYMBOL')['Adj_eps_abs'].transform(lambda x: x.rolling(4).sum())
        # SectorThemeGICS = pd.read_excel('SectorMapping.xlsx')
        SectorThemeGICS = self.sectorData.copy()
        df = pd.merge(b_filtered, SectorThemeGICS[['Symbol', 'Sector']], left_on='SYMBOL', right_on='Symbol', how='left')
        # Calculating ROE for TTM for the past 3 Years years
        df['ROE_ttm'] = df.groupby('Symbol')['ROE_ttm'].transform(lambda x : x.rolling(12).mean())
        # Create a new column for quarter information
        df['Quarter'] = pd.to_datetime(df['Date_End'], format='%Y%m').dt.quarter

        #  EPS Growth Calculation
        def calculate_yoy_eps_growth(eps):
            """Calculates YoY EPS Growth based on the provided rules."""
            growth = []
            for i in range(len(eps)):
                if i < 4:
                    growth.append(np.nan)
                else:
                    prev_eps = eps.iloc[i - 4] 
                    curr_eps = eps.iloc[i]     
                    if prev_eps > 0:
                        growth.append((curr_eps - prev_eps) / prev_eps)
                    elif prev_eps < 0:
                        growth.append(-(curr_eps - prev_eps) / prev_eps)
                    else:
                        growth.append(np.nan) 
            return pd.Series(growth, index=eps.index) 

        # Calculating YoY EPS Growth for each quarter
        df['YoY_EPS_Growth'] = df.groupby('SYMBOL')['Adj_eps_abs'].transform(calculate_yoy_eps_growth)

        # Calculate 3-year mean and std deviation for each quarter separately
        def calc_quarterly_stats(group):
            group = group.sort_values('Date_End')
            group['Mean_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=3, min_periods=1).mean()
            group['Std_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=3, min_periods=1).std()
            return group

        # Apply function to each SYMBOL and Quarter group
        df = df.groupby(['SYMBOL', 'Quarter'], group_keys=False).apply(calc_quarterly_stats)

        # Dropping Quarter column after calculation to keep the original dataframe structure
        df.drop(columns=['Quarter'], inplace=True)

        # Calculating rolling 3-year mean and std deviation for EPS growth
        df['Mean_YoY_EPS_Growth'] = df.groupby('SYMBOL')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=12,min_periods=4).mean())
        df['Std_YoY_EPS_Growth'] = df.groupby('SYMBOL')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=12,min_periods=4).std())

        # Calulating EPS Growth Variability
        df['EPS_Growth_Variability'] = df['Mean_YoY_EPS_Growth'].div(df['Std_YoY_EPS_Growth'])

        # Split data into three sectors: Bank, Finance, and Others
        bank_df = df[df['Sector'] == 'Bank']
        finance_df = df[df['Sector'] == 'Finance']
        non_financials_df = df[~df['Sector'].isin(['Bank', 'Finance'])]

        # Calculate metrics for Bank sector
        bank_df['mean_roe_bank'] = bank_df.groupby('Date_End')['ROE_ttm'].transform('mean')
        bank_df['std_roe_bank'] = bank_df.groupby('Date_End')['ROE_ttm'].transform('std')
        bank_df['mean_eps_growth_variability_bank'] = bank_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
        bank_df['std_eps_growth_variability_bank'] = bank_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

        # Calculate metrics for Finance sector
        finance_df['mean_roe_fin'] = finance_df.groupby('Date_End')['ROE_ttm'].transform('mean')
        finance_df['std_roe_fin'] = finance_df.groupby('Date_End')['ROE_ttm'].transform('std')
        finance_df['mean_eps_growth_variability_fin'] = finance_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
        finance_df['std_eps_growth_variability_fin'] = finance_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

        # Calculate metrics for non-financials
        non_financials_df['mean_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('mean')
        non_financials_df['std_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('std')
        non_financials_df['mean_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('mean')
        non_financials_df['std_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('std')
        non_financials_df['mean_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
        non_financials_df['std_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

        # Z-Score calculations for Bank sector
        bank_df['Z_ROE_ttm_bank'] = (bank_df['ROE_ttm'] - bank_df['mean_roe_bank']) / bank_df['std_roe_bank']
        bank_df['Z_EPS_Growth_Variability_bank'] = (bank_df['EPS_Growth_Variability'] - bank_df['mean_eps_growth_variability_bank']) / bank_df['std_eps_growth_variability_bank']

        # Z-Score calculations for Finance sector
        finance_df['Z_ROE_ttm_fin'] = (finance_df['ROE_ttm'] - finance_df['mean_roe_fin']) / finance_df['std_roe_fin']
        finance_df['Z_EPS_Growth_Variability_fin'] = (finance_df['EPS_Growth_Variability'] - finance_df['mean_eps_growth_variability_fin']) / finance_df['std_eps_growth_variability_fin']

        # Z-Score calculations for non-financials
        non_financials_df['Z_ROE_ttm'] = (non_financials_df['ROE_ttm'] - non_financials_df['mean_roe']) / non_financials_df['std_roe']
        non_financials_df['Z_DE'] = (non_financials_df['Debt/Equity Ratio'] - non_financials_df['mean_de']) / non_financials_df['std_de']
        non_financials_df['Z_EPS_Growth_Variability'] = (non_financials_df['EPS_Growth_Variability'] - non_financials_df['mean_eps_growth_variability']) / non_financials_df['std_eps_growth_variability']

        # Weighted Average Z Quality Score calculation
        def calculate_weighted_avg_z(row):
            if row['Sector'] == 'Bank':
                return (1/2) * row['Z_ROE_ttm_bank'] - (1/2) * abs(row['Z_EPS_Growth_Variability_bank'])
            elif row['Sector'] == 'Finance':
                return (1/2) * row['Z_ROE_ttm_fin'] - (1/2) * abs(row['Z_EPS_Growth_Variability_fin'])
            else:
                return (1/3) * row['Z_ROE_ttm'] - (1/3) * abs(row['Z_DE']) - (1/3) * abs(row['Z_EPS_Growth_Variability'])
            
        # Combine all dataframes
        df_combined = pd.concat([bank_df, finance_df, non_financials_df]).reset_index(drop=True)
        # Calculate weighted average Z-score
        df_combined['WeightedAvgZ'] = df_combined.apply(calculate_weighted_avg_z, axis=1)
        # Calculate Normalized Quality Score
        df_combined['Quality_Score'] = np.where(df_combined['WeightedAvgZ'] >= 0,
                                            1 + df_combined['WeightedAvgZ'],
                                            (1 - df_combined['WeightedAvgZ'])**-1)
        # df_combined
        # Final dataframe with required columns
        # df = df_combined[['Date_End', 'Symbol', 'Sector', 'Quality_Score']].dropna().reset_index(drop=True)
        # First, let's calculate within-sector quality scores
        # We need to recalculate Z-scores within each sector for each date
        def calculate_within_sector_quality_score(df):
            df_result = df.copy()
            # Calculate within-sector Z-scores for each sector
            for sector in df['Sector'].unique():
                sector_mask = df['Sector'] == sector
                sector_data = df[sector_mask].copy()
                if sector in ['Bank', 'Finance']:
                    # For Bank and Finance sectors - calculate within-sector means and stds
                    sector_data['mean_roe_sector'] = sector_data.groupby('Date_End')['ROE_ttm'].transform('mean')
                    sector_data['std_roe_sector'] = sector_data.groupby('Date_End')['ROE_ttm'].transform('std')
                    sector_data['mean_eps_growth_variability_sector'] = sector_data.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
                    sector_data['std_eps_growth_variability_sector'] = sector_data.groupby('Date_End')['EPS_Growth_Variability'].transform('std')
                    # Calculate within-sector Z-scores
                    sector_data['Z_ROE_ttm_sector'] = (sector_data['ROE_ttm'] - sector_data['mean_roe_sector']) / sector_data['std_roe_sector']
                    sector_data['Z_EPS_Growth_Variability_sector'] = (sector_data['EPS_Growth_Variability'] - sector_data['mean_eps_growth_variability_sector']) / sector_data['std_eps_growth_variability_sector']
                    # Calculate within-sector quality score
                    sector_data['Within_Sector_Quality_Score'] = (1/2) * sector_data['Z_ROE_ttm_sector'] - (1/2) * abs(sector_data['Z_EPS_Growth_Variability_sector'])
                else:  # Non-financial sectors
                    # Calculate within-sector means and stds for non-financials
                    sector_data['mean_roe_sector'] = sector_data.groupby('Date_End')['ROE_ttm'].transform('mean')
                    sector_data['std_roe_sector'] = sector_data.groupby('Date_End')['ROE_ttm'].transform('std')
                    sector_data['mean_de_sector'] = sector_data.groupby('Date_End')['Debt/Equity Ratio'].transform('mean')
                    sector_data['std_de_sector'] = sector_data.groupby('Date_End')['Debt/Equity Ratio'].transform('std')
                    sector_data['mean_eps_growth_variability_sector'] = sector_data.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
                    sector_data['std_eps_growth_variability_sector'] = sector_data.groupby('Date_End')['EPS_Growth_Variability'].transform('std')                
                    # Calculate within-sector Z-scores
                    sector_data['Z_ROE_ttm_sector'] = (sector_data['ROE_ttm'] - sector_data['mean_roe_sector']) / sector_data['std_roe_sector']
                    sector_data['Z_DE_sector'] = (sector_data['Debt/Equity Ratio'] - sector_data['mean_de_sector']) / sector_data['std_de_sector']
                    sector_data['Z_EPS_Growth_Variability_sector'] = (sector_data['EPS_Growth_Variability'] - sector_data['mean_eps_growth_variability_sector']) / sector_data['std_eps_growth_variability_sector']
                    # Calculate within-sector quality score
                    sector_data['Within_Sector_Quality_Score'] = (1/3) * sector_data['Z_ROE_ttm_sector'] - (1/3) * abs(sector_data['Z_DE_sector']) - (1/3) * abs(sector_data['Z_EPS_Growth_Variability_sector'])
                # Update the result dataframe
                df_result.loc[sector_mask, 'Within_Sector_Quality_Score'] = sector_data['Within_Sector_Quality_Score']
            return df_result

        # Calculate within-sector quality scores
        df_combined = calculate_within_sector_quality_score(df_combined)
        # Handle any NaN or infinite values that might arise from division by zero
        df_combined['Within_Sector_Quality_Score'] = df_combined['Within_Sector_Quality_Score'].replace([np.inf, -np.inf], np.nan)
        # Normalize within-sector quality scores to create within-sector normalized scores
        def normalize_within_sector_scores(df):
            df_result = df.copy()
            df_result['Within_Sector_Normalized_Score'] = np.where(
                df_result['Within_Sector_Quality_Score'] >= 0,
                1 + df_result['Within_Sector_Quality_Score'],
                (1 - df_result['Within_Sector_Quality_Score'])**-1
            )
            return df_result

        df_combined = normalize_within_sector_scores(df_combined)
        df = df_combined[['Date_End', 'Symbol', 'Sector', 'Quality_Score', 'Within_Sector_Normalized_Score']].dropna().reset_index(drop=True)
        df['MeanQScore'] = ((1/2) * df['Quality_Score'] + (1/2) * (df['Within_Sector_Normalized_Score']))
        # df

        # GrowthDate = self.__dataCursor.fetch_data_from_database(table_name="GrowthDate", no_of_years=50)[['Date', 'Qtr']]
        GrowthDate = pd.read_csv('growth_date_latest.csv', parse_dates=['Date'])[['Date', 'Qtr']]
        GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')
        # GrowthDate Mapping 
        final_df =  pd.merge(df, GrowthDate, left_on='Date_End', right_on='Qtr').drop(columns=['Date_End', 'Qtr'])
        final_df = final_df[['Date', 'Symbol', 'Sector', 'MeanQScore']].sort_values(by='Date').reset_index(drop=True)
        # final_df

        # Merging Quality Ranks with price data universe
        price_data['Date'] = pd.to_datetime(price_data['Date'])
        final_df['Date'] = pd.to_datetime(final_df['Date'])
        merged_df = pd.merge(final_df[['Date', 'Symbol', 'MeanQScore']], price_data, on=['Date', 'Symbol'], how='outer')
        merged_df =  merged_df.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
        merged_df['Quality_pct_rank'] = merged_df.groupby('Date', group_keys=False)['MeanQScore'].apply(lambda x : x.rank(pct=True))
        merged_df[['Quality_pct_rank']] = merged_df.groupby('Symbol', group_keys=False)[['Quality_pct_rank']].apply(lambda x: x.fillna(method='ffill'))
        qualityfinal = merged_df[['Date','Symbol','Quality_pct_rank']]
        qualityfinal.rename(columns={'Quality_pct_rank': '3DQuality'}, inplace=True)
        # qualityfinal
        top500 = qualityfinal.filter(["Symbol", "Date","3DQuality"])

        ## Convert the long from Dataframe to wide form dataframe
        top500 = top500.pivot_table(index='Date',columns='Symbol',values='3DQuality').reset_index()
        ## Shifting the Date column
        top500["Date"] = top500["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        top500.iloc[-1, top500.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        top500 = top500.melt(id_vars='Date', value_name = "3DQuality")
        ## Drop na and reset the index
        top500 = top500.dropna().reset_index(drop = True)
        top500 = top500[top500["Date"] >= "2006-01-01"].reset_index(drop = True).copy()
        # top500
        ## Store the Factor Data in dictionary.
        self.factorData["3DQuality"] = top500.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["3DQuality"] = top500[top500["Date"] == top500["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### 3D QUALITY COMPLETE ###")

        del price_data, top500, company_master, Finance_bs, rebal_dates, pat_df, p, roe_ttm, quality, pl_quality, a, b, b_filtered, SectorThemeGICS, df, bank_df, finance_df, non_financials_df, df_combined, merged_df, qualityfinal

    def generate_3DValue(self):
        ## Fetch the Stock Value Data
        if self.stockValueData is None:
            self.__readValueData()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()


        price_data = self.stockPriceData.copy()
        top_500 = self.strategyUniverse.copy()

        # table = pd.read_csv('value_raw_data_latest.csv')
        # table['Date'] = pd.to_datetime(table['Date'])
        table = self.stockValueData.copy()

        # price_data = pd.read_csv('stockPriceData-3.csv')
        price_data=price_data[price_data['Date']>='2006']
        price_data.drop_duplicates(['Date','Symbol'],inplace=True)
        # price_data['Date'] = pd.to_datetime(price_data['Date'])
        
        table = pd.merge(price_data,table, on= ['Date','Symbol'])
        # Filter top 500 companies by market cap for each date
        table = table.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
        # Keep only the relevant columns
        table = table[['Date', 'Symbol', 'PB', 'PE','Mcap']]
        # Replace non-positive values with NaN for factors
        factor = ['PB', 'PE']
        table[factor] = table[factor].where(table[factor] > 0, np.nan)

        # Calculate mean and standard deviation for each factor grouped by date
        table['mean_PB'] = table.groupby('Date')['PB'].transform('mean')
        table['mean_PE'] = table.groupby('Date')['PE'].transform('mean')
        table['STD_PB'] = table.groupby('Date')['PB'].transform('std')
        table['STD_PE'] = table.groupby('Date')['PE'].transform('std')

        # Calculate inverted Z-Scores (higher values -> lower Z-scores)
        table['Z_PE'] = -(table['PE'] - table['mean_PE']) / table['STD_PE']
        table['Z_PB'] = -(table['PB'] - table['mean_PB']) / table['STD_PB']

        # Calculate Weighted Average Z-Score
        table['WeightedAvgZ'] = ((1/2) * table['Z_PE']) + ((1/2) * table['Z_PB'])

        # Calculate Normalized Value Score
        table['Value_score_cross'] = np.where(table['WeightedAvgZ'] >= 0,
                                        1 + table['WeightedAvgZ'],
                                        (1 - table['WeightedAvgZ'])**-1)

        # Filter, rank, and finalize the table
        table = table[['Date', 'Symbol', 'Value_score_cross', 'PB', 'PE', 'Mcap']].reset_index(drop=True)
        table['Value_cross_rank'] = table.groupby('Date', group_keys=False)['Value_score_cross'].apply(lambda x: x.rank(pct=True, ascending=True))
        # Filter top 500 companies by market cap for each date
        table = table.groupby('Date', group_keys=False).apply(
            lambda x: x.sort_values(by='Mcap', ascending=False).head(500)
        )
        # Keep only relevant columns
        table = table[['Date', 'Symbol', 'PB', 'PE','Mcap','Value_score_cross','Value_cross_rank']]
        # Replace non-positive values with NaN for factors
        factors = ['PB', 'PE']
        table[factors] = table[factors].where(table[factors] > 0, np.nan)
        # Merge with sector mapping data
        # SectorThemeGICS = pd.read_excel('SectorMapping.xlsx')
        SectorThemeGICS = self.sectorData.copy()
        table = pd.merge(table, SectorThemeGICS[['Symbol', 'Sector']], on='Symbol', how='left')
        # Calculate sector-wise mean and std
        for factor in factors:
            table[f'mean_{factor}Peer'] = table.groupby(['Date', 'Sector'])[factor].transform('mean')
            table[f'STD_{factor}Peer'] = table.groupby(['Date', 'Sector'])[factor].transform('std')

        # Calculate inverted Z-scores for sector peers
        table['Z_PBPeer'] = -(table['PB'] - table['mean_PBPeer']) / table['STD_PBPeer']
        table['Z_PEPeer'] = -(table['PE'] - table['mean_PEPeer']) / table['STD_PEPeer']

        # Calculate weighted average Z-score (equal weights)
        table['WeightedAvgZPeer'] = ((1/2) * table['Z_PEPeer'] + (1/2) * table['Z_PBPeer'])

        # Calculate normalized value score for sector peers
        table['Value_score_Peer'] = np.where(
            table['WeightedAvgZPeer'] >= 0,
            1 + table['WeightedAvgZPeer'],
            (1 - table['WeightedAvgZPeer'])**-1
        )

        # Calculate percentile rank within sector for each date
        table['Value_pct_rank_Peer'] = table.groupby(['Date', 'Sector'], group_keys=False)['Value_score_Peer'].apply(lambda x: x.rank(pct=True, ascending=True))

        # Finalize and clean table
        table = table[['Date', 'Symbol', 'Sector', 'PB', 'PE','Value_score_cross','Value_cross_rank','Value_score_Peer', 'Value_pct_rank_Peer','Mcap']].reset_index(drop=True)
        table = table.sort_values(['Symbol', 'Date'])
        window = 252*3  # rolling window size (adjust for your data frequency)
        # Replace non-positive values
        for factor in ['PB', 'PE']:
            table[factor] = table[factor].where(table[factor] > 0, np.nan)

        # Rolling metrics per stock
        for factor in ['PB', 'PE']:
            table[f'mean_{factor}'] = table.groupby('Symbol')[factor].transform(lambda x: x.rolling(window).mean())
            table[f'STD_{factor}'] = table.groupby('Symbol')[factor].transform(lambda x: x.rolling(window).std())

        # Calculate Z-scores (inverted, so lower valuation → higher score)
        for factor in ['PB', 'PE']:
            table[f'Z_{factor}'] = -(table[factor] - table[f'mean_{factor}']) / table[f'STD_{factor}']

        # Weighted average Z
        table['WeightedAvgZ_time'] = 0.5 * table['Z_PE'] + 0.5 * table['Z_PB']

        # Normalized Value Score (same as cross-section formula)
        table['Value_score_time'] = np.where(table['WeightedAvgZ_time'] >= 0,
                                            1 + table['WeightedAvgZ_time'],
                                            (1 - table['WeightedAvgZ_time'])**-1)
        table['Value_time_rank'] = table.groupby('Date', group_keys=False)['Value_score_cross'].apply(lambda x: x.rank(pct=True, ascending=True))

        # Finalize: drop NaNs from rolling window
        table = table[['Date', 'Symbol', 'Sector','Value_score_cross','Value_cross_rank','Value_score_Peer', 'Value_pct_rank_Peer','Value_score_time','Value_time_rank','Mcap']].reset_index(drop=True)
        # Step 1: Calculate mean score
        table['mean_score'] = table[['Value_score_cross', 'Value_score_Peer', 'Value_score_time']].mean(axis=1)
        # Step 2: Calculate percentile rank (higher is better)
        table['mean_score_percentile'] = table['mean_score'].rank(pct=True)
        table = table[['Date','Symbol','mean_score_percentile']]
        table.rename(columns={'mean_score_percentile': '3DValue'}, inplace=True)

        # value final
        top500 = table.filter(["Symbol", "Date","3DValue"]).copy()

        ## Convert the long from Dataframe to wide form dataframe
        top500 = top500.pivot_table(index='Date',columns='Symbol',values='3DValue').reset_index()
        ## Shifting the Date column
        top500["Date"] = top500["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        top500.iloc[-1, top500.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        top500 = top500.melt(id_vars='Date', value_name = "3DValue")
        ## Drop na and reset the index
        top500 = top500.dropna().reset_index(drop = True)
        top500 = top500[top500["Date"] >= "2006-01-01"].reset_index(drop = True).copy()
        # top500
        ## Store the Factor Data in dictionary.
        self.factorData["3DValue"] = top500.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["3DValue"] = top500[top500["Date"] == top500["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### 3D VALUE COMPLETE ###")

        del price_data, top500, table, SectorThemeGICS

    def generate_AllFactors(self,save_appender = False):
        # FUnction call for LTM Factor
        self.generate_LTM()
        # ## Function call for Momentum Factor
        # self.generate_Momentum()
        # ## Function call for Theme Factor
        # self.generate_Theme()
        # ## Function call for Volatility Factor
        # self.generate_Volatility()
        ## Function call for Value Factor
        self.generate_ValueYield()
        # Function call for Growth Factor
        self.generate_Growth()
        ## Function call for Quality Factor
        self.generate_QualityAnnual()
        ## Function call for Quality Factor
        self.generate_QualityQuarter()
        ## Function call for EM Factor
        self.generate_EM()
        # Function call for Dividend Factor
        self.generate_Dividend()
        ## Function call for LTMA Factor
        self.generate_LTMA()
        # # Function call for Low Vol Factor
        self.generate_LowVol()
        ## Function call for AM Factor
        self.generate_AM()
        ## Function call for UltraShort AM Factor
        self.generate_UltraShortAM()
        ## Function call for Short AM Factor
        self.generate_ShortAM()
        ## Function call for AM Factor
        self.generate_ValueYieldNoPeg()
        ## Function call for AM Factor
        self.generate_ValueYieldExDiv()
        ## Function call for Mid AM Factor
        self.generate_MidAM()
        ## Function call for Long AM Factor
        self.generate_LongAM()
        ## Function call for Multi AM Factor
        self.generate_MultiAM()

        self.generate_Beta()

        self.generate_TrendAntitrendMR()

        self.generate_ShiftedAM()

        self.generate_3DQuality()

        self.generate_3DValue()

        # Extract unique dates from the 'Date' column of the strategyUniverse DataFrame.
        # Drop duplicates and reset the index to create a clean, unique list of dates.
        dates = self.strategyUniverse[["Date"]].drop_duplicates().reset_index(drop=True)

        # Create a new column 'DateO' which is the next date (shifted upwards by one row).
        # This maps each date to the following date in the dataset.
        dates["DateO"] = dates["Date"].shift(-1)

        # Create a dictionary to map each date to its corresponding next date ('DateO').
        dateMapper = dict(zip(dates["Date"], dates["DateO"]))

        # Make a copy of the original strategyUniverse DataFrame to avoid modifying it directly.
        strategyUniverse = self.strategyUniverse.copy()

        # Map the 'DateO' column in the copied DataFrame using the dateMapper.
        strategyUniverse["DateO"] = strategyUniverse["Date"].replace(dateMapper)

        # Fill any missing values in 'DateO' (last row) with today's date.
        strategyUniverse["DateO"] = strategyUniverse["DateO"].fillna(pd.to_datetime(date.today()))

        # Drop the original 'Date' column as 'DateO' will now replace it.
        strategyUniverse.drop(columns=["Date"], inplace=True)

        # Rename the 'DateO' column back to 'Date' to maintain consistency in column names.
        strategyUniverse.rename(columns={'DateO': "Date"}, inplace=True)

        # Reorder the columns so that the 'Date' column is the last one in the DataFrame.
        strategyUniverse = strategyUniverse[list(strategyUniverse.columns[-1:]) + list(strategyUniverse.columns[:-2])]

        ## Generating the Appender file only when all the factors are generated.
        self.appender = pd.DataFrame()
        for key in self.factorData.keys():
            if len(self.appender) == 0:
                self.appender = pd.merge(strategyUniverse,  self.factorData[key], on = ["Date", "Symbol"], how = "left")
            else:
                self.appender = pd.merge(self.appender, self.factorData[key], on = ["Date", "Symbol"], how= "left")

        ## Save the appender file to csv only if the flag is True
        if save_appender:
            ## If there is no directory then create the directory
            if not os.path.isdir("./DailyAppender"):
                os.mkdir("./DailyAppender")

            # self.appender.to_csv(self.uniqueFileName(f"./DailyAppender/Appender_{date.today().strftime('%d%b%Y')}.csv"), index = False)
            self.appender.to_csv("./DailyAppender/Appender_latest.csv", index = False)

        print("### ALL THE FACTORS ARE GENERATED ###")    



In [2]:
# QualityDate = d.fetch_data_from_database(table_name = "QualityDate", no_of_years = 100)
# QualityDate.to_csv('QualityDate.csv', index=False)
# growth_date_latest = d.fetch_data_from_database(table_name = "GrowthDate", no_of_years=100)
# growth_date_latest.to_csv('growth_date_latest.csv', index=False)
# Finance_bs = d.fetch_data_from_database(table_name = "Finance_bs")
# Finance_bs.to_csv("./Finance_bs.csv", index=False)
# Finance_fr = d.fetch_data_from_database(table_name = "Finance_fr")
# Finance_fr.to_csv("./Finance_fr.csv", index=False)
# Finance_pl = d.fetch_data_from_database(table_name = "Finance_pl")
# Finance_pl.to_csv('Finance_pl.csv', index=False)
# # self.profitLossQuality = pd.read_csv("./Finance_pl.csv")
# Nifty_PE_PB = d.fetch_data_from_database(table_name = "Nifty_PE_PB", no_of_years = 30)
# Nifty_PE_PB.to_csv('Nifty_PE_PB.csv', index=False)
# # self.benchmarkValueData = pd.read_csv("./RAW_DATA/Nifty_PE_PB.csv")
# Value_Data_test = d.fetch_data_from_database(table_name = "Value_RawData_Test", no_of_years = 30)
# Value_Data_test.to_csv('Value_Data.csv', index=False)
# Finance_Cf = d.fetch_data_from_database(table_name = "Finance_cf")
# Finance_Cf.to_csv('Finance_Cf.csv', index=False)
# Quarterly = d.fetch_data_from_database(table_name = "Quarterly")
# Quarterly.to_csv('Quarterly.csv', index=False)
# CM = d.fetch_data_from_database(table_name = "Companymaster")
# CM.to_csv('CM.csv', index=False)
# price_data = d.fetch_price_data(no_of_years = 30, inc_indices=True)
# price_data.to_csv('price_data.csv', index=False)
# etf_indices = d.fetch_data_from_database(table_name="Etf_Indices")
# etf_indices.to_csv('etf_indices.csv', index=False)
# Universe = d.fetch_data_from_database(table_name="universe_mcap_500", no_of_years=50)
# Universe.to_csv('Universe.csv', index=False)
# gics = d.fetch_data_from_database(table_name="SectorThemeGics")
# gics.to_csv('gics.csv', index=False)

In [3]:
## Import the Libraries
import pandas as pd
import numpy as np

from datetime import date

import functools as ft

import os
import warnings
warnings.filterwarnings("ignore")

periodMapping = {
                    'LTM'        : {"1Y" : {"S" : 0,   "F" : 252},
                                    "3Y" : {"S" : 252, "F" : 500},
                                    "5Y" : {"S" : 750, "F" : 500}},

                    'Momentum'   : {'1W' : 5, "1M" : 20, 
                                    "3M" : 62, "6M" : 125, 
                                    "1Y" : 250,"2Y" : 500,  
                                    "3Y" : 750},

                    'Volatility' : {'1W' : 5, "2W" : 10, 
                                    "1M" : 20, "3M" : 62, 
                                    "6M" : 125, "1Y" : 250, 
                                    "3Y" : 750},
                        
                    "Theme"      : {'1W' : 5, "2W" : 10, "2M" : 40,
                                    "1M" : 20, "3M" : 62,
                                    "1Y" : 250, "6M" : 125,
                                    "3Y" : 750}
                }


class EquityFactorsConsol:

    def __init__(self, noOfYears, correct, peer = "Theme"):
        ## Initialize the variables
        self.stockPriceData = None
        self.stockValueData = None
        self.sectorData = None
        self.benchmark = None
        self.stockFactors = None
        self.themeFactor = None
        self.companyMaster = None
        self.cashFlow = None
        self.profitLossGrowth = None
        self.profitLossQuality = None
        self.financialRatio = None
        self.appender = None
        self.financeBalanceSheet = None

        self.factorData = dict()
        self.recentFactorUpdate = dict()
        # self.__dataCursor = Data()

        self.noOfYears = noOfYears
        self.correct = correct
        self.peer = peer

    def uniqueFileName(self, filename):
        """
        Generates a unique filename by appending a counter to the base name if a file with 
        the specified filename already exists in the current directory.

        Args:
            filename (str): The desired filename.

        Returns:
            str: A unique filename that does not already exist in the current directory.
        """

        # Split the path into directory and filename
        directory, file_name = os.path.split(filename)
        
        # Split the filename into name and extension
        name, ext = os.path.splitext(file_name)
        
        # Initialize counter
        counter = 1
        
        # Generate the full path
        full_path = os.path.join(directory, file_name)
        
        # Check if the file already exists
        while os.path.exists(full_path):
            # If it does, create a new filename with the counter
            new_filename = f"{name}_{counter}{ext}"
            full_path = os.path.join(directory, new_filename)
            counter += 1
            
        return full_path

    def __readPriceData(self,):
        ''' Function  is used to read the price data '''
        ## Read the price Data.
        # self.stockPriceData = self.__dataCursor.fetch_price_data(no_of_years = self.noOfYears, inc_indices=True)
        self.stockPriceData = pd.read_csv("./price_data.csv")

        self.stockPriceData["Date"] = pd.to_datetime(self.stockPriceData["Date"])

        # if self.correct:
        #     # Replace The Old Symbol With the New Symbol
        #     mapping = pd.read_excel("company_master_mapping.xlsx")
        #     mapping = dict(zip(mapping["SYMBOL_NSE"], mapping["SYMBOL_CM"]))
        #     self.stockPriceData["Symbol"] = self.stockPriceData["Symbol"].replace(mapping)

        self.stockPriceData["Symbol"] = self.stockPriceData["Symbol"].str.strip()
        self.stockPriceData1 = self.stockPriceData.copy()
        self.stockPriceDataMom = self.stockPriceData.copy()
        ## Drop the Rows with NaN Values
        self.stockPriceData = self.stockPriceData.dropna()
        ## Remove the data of Erroneous Date.
        self.stockPriceData = self.stockPriceData[self.stockPriceData["Date"] != "2022-03-07"]#.reset_index(drop = True)

        ## Filter the data for the Benchmark data
        self.benchmark = self.stockPriceData[self.stockPriceData["Symbol"] == "NIFTY500"].reset_index(drop = True)
        ## Filter the necessary columns for benchmark
        self.benchmark = self.benchmark.filter(["Date","Close"])
        ## Calculate the daily percentage change for the benchmark and renaming the columns.
        self.benchmark["BenchmarkReturn"] = self.benchmark["Close"].pct_change()
        self.benchmark.rename(columns = {'Close' : "BenchmarkPrice"}, inplace = True)

        ## Fetch the ETF Indices List
        # self.etf = self.__dataCursor.fetch_data_from_database(table_name="Etf_Indices")
        self.etf = pd.read_csv("./etf_indices.csv")
        ## Remove the ETF Indices data from the stock data
        self.stockPriceData = self.stockPriceData[~self.stockPriceData['Symbol'].isin(self.etf['Symbol'])]

        self.priceDataLTM = self.stockPriceData.copy()
        ## Remove the data for the Saturday and Sunday and Sort the dataframe
        self.stockPriceData = self.stockPriceData[~self.stockPriceData["Date"].dt.day_name().isin(['Sunday', "Saturday"])]
        self.stockPriceData = self.stockPriceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        # ## Identify Companies which have been in Top-500 historically.
        # top500Companies  = self.stockPriceData.groupby(["Date"]).apply(lambda x: x.sort_values("Mcap", ascending=False).head(500), include_groups = False).reset_index(level = 0).reset_index(drop = True)
        # self.top500Companies = top500Companies["Symbol"].unique().tolist()
        # ## Identify Companies which have been in TOP-1000 historically.
        # top1000Companies  = self.stockPriceData.groupby(["Date"]).apply(lambda x: x.sort_values("Mcap", ascending=False).head(1000), include_groups=False).reset_index(level = 0).reset_index(drop = True)
        # self.top1000Companies = top1000Companies["Symbol"].unique().tolist()

        self.strategyUniverse = pd.read_csv("Universe.csv")
        # self.strategyUniverse = self.__dataCursor.fetch_data_from_database(table_name="universe_mcap_500", no_of_years=50)
        self.strategyUniverse["Date"] = pd.to_datetime(self.strategyUniverse["Date"])
        self.strategyUniverse["Peer"] = self.strategyUniverse[self.peer].copy()
        self.strategyUniverse.rename(columns = {"MCAP" : "Mcap"}, inplace = True)

        # sectordata = self.__dataCursor.fetch_data_from_database(table_name="SectorThemeGics")
        # self.strategyUniverse = pd.merge(self.strategyUniverse, sectordata, on = "Symbol", how = "left")
        # self.strategyUniverse[["Theme", "Sector"]] = self.strategyUniverse[["Theme", "Sector"]].fillna("Others")
        # self.strategyUniverse["Peer"] = self.strategyUniverse[self.peer].copy()

    def __readSectorData(self,):
        ## Inout the Sector Mapping data
        # self.sectorData = self.__dataCursor.fetch_data_from_database(table_name="SectorThemeGics")
        self.sectorData = pd.read_csv("./gics.csv")

    def __readCompanyMaster(self,):
        # self.companyMaster = self.__dataCursor.fetch_data_from_database(table_name = "Companymaster")
        self.companyMaster = pd.read_csv("./CM.csv")
        self.companyMaster = self.companyMaster[self.companyMaster["SERIES"] == "EQ"]

        self.companyMaster = self.companyMaster[["FINCODE", "SYMBOL"]].copy()
        self.companyMaster = self.companyMaster.dropna().reset_index(drop = True)
        self.companyMaster.rename(columns = {'SYMBOL' : "Symbol"}, inplace = True)

    def __readProfitLossGrowth(self,):
        # self.profitLossGrowth = self.__dataCursor.fetch_data_from_database(table_name = "Quarterly_Cons")
        self.profitLossGrowth = pd.read_csv("./Quarterly_Cons.csv")
        self.profitLossGrowth = self.profitLossGrowth[self.profitLossGrowth["Result_Type"] == "Q"].reset_index(drop = True)
        self.profitLossGrowth = self.profitLossGrowth.filter(items = ["Fincode", "Date_End",  "PAT", "OPERATING_PROFIT", "GROSS_PROFIT",
                                                                      "NET_SALES", "EPS_DILUTED", "PBT", 'Debt/Equity Ratio', 'Adj_eps_abs', "Dividend payout ratio"])
        self.profitLossGrowth.rename(columns = {'Fincode' : "FINCODE"}, inplace = True)

    def __readCashFlow(self, ):
        # self.cashFlow = self.__dataCursor.fetch_data_from_database(table_name = "Finance_cons_cf")
        self.cashFlow = pd.read_csv("./Finance_cons_cf.csv")

    def __readValueData(self):
            ## Input the Company Valuation Data.

            # if self.correct:
            #     self.stockValueData = self.__dataCursor.fetch_data_from_database(table_name = "Value_RawData_Test", no_of_years = self.noOfYears)
            # else:
            #     self.stockValueData = self.__dataCursor.fetch_data_from_database(table_name = "Value_RawData", no_of_years = self.noOfYears)
            
            self.stockValueData = pd.read_csv("./Value_Data.csv")
            ## COnvert the Date column from string to Datetime format
            self.stockValueData["Date"] = pd.to_datetime(self.stockValueData["Date"])

            # Import the Nifty Valuation Fields
            # self.benchmarkValueData = self.__dataCursor.fetch_data_from_database(table_name = "Nifty_PE_PB", no_of_years = self.noOfYears)
            self.benchmarkValueData = pd.read_csv("./Nifty_PE_PB.csv")
            self.benchmarkValueData["Date"] = pd.to_datetime(self.benchmarkValueData["Date"])

    def __readProfitLossQuality(self,):
        # self.profitLossQuality = self.__dataCursor.fetch_data_from_database(table_name = "Finance_cons_pl")
        self.profitLossQuality = pd.read_csv("./Finance_cons_pl.csv")
        self.profitLossQuality = self.profitLossQuality[["FINCODE", "Year_end", "Profit_after_tax", "Net_sales", 
                                                        "Operating_profit", "Gross_profits", "Adj_Eps"]]

    def __readFinancialRatio(self,):
        # self.financialRatio = self.__dataCursor.fetch_data_from_database(table_name = "Finance_cons_fr")

        self.financialRatio = pd.read_csv("./Finance_cons_fr.csv")
        self.financialRatio = self.financialRatio[["FINCODE", "Year_end", "Inventory_Days", "Receivable_days", "Payable_days", "ROE", "ROCE", 
                                                   "Total_Debt_Equity", "Interest_Cover", "CEPS", "ROA", "FCF_Share", "Dividend_Payout_Per"]]
        self.financialRatio = self.financialRatio[self.financialRatio["Year_end"] >= 200012]
        self.financialRatio = self.financialRatio.reset_index(drop = True)

    def __readFinanceBalanceSheet(self,):
        # self.financeBalanceSheet = self.__dataCursor.fetch_data_from_database(table_name = "Finance_cons_bs")
        self.financeBalanceSheet = pd.read_csv("./Finance_cons_bs.csv")
        # self.financeBalanceSheet = self.financeBalanceSheet[["FINCODE", "Year_end", "Profit_after_tax", "Net_sales", 
        #                                                  "Operating_profit", "Gross_profits", "Adj_Eps"]]
        self.financeBalanceSheet.rename(columns = {"Fincode" : "FINCODE"}, inplace = True)

    def generate_LowVol(self,):

        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Calculate daily returns for each stock by symbol, based on "Close" prices
        priceData["Returns"] = priceData.groupby("Symbol")["Close"].pct_change()
        
        # Calculate squared returns for down days only; store in "DReturns" (for downside volatility)
        priceData["DReturns"] = np.square(np.where(priceData["Returns"] < 0, priceData["Returns"], 0))

        # Calculate 1-year rolling average of downside returns for each symbol and take the square root
        priceData["DownVol"] = priceData.groupby("Symbol")["DReturns"].transform(lambda x: x.rolling(252).mean())
        priceData["DownVol"] = np.sqrt(priceData["DownVol"])

        # Calculate 1-year rolling standard deviation of returns as low volatility measure
        priceData['LowVol'] = priceData.groupby("Symbol")["Returns"].transform(lambda x: x.rolling(window=252).std())

        # Calculate average volatility as the mean of LowVol and DownVol
        priceData["AvgVol"] = priceData[["LowVol", "DownVol"]].mean(axis=1)

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        # Rank stocks daily within the top 500 by LowVol, DownVol, and AvgVol, assigning percentile ranks
        priceData[["LowVolRank", "DownVolRank", "AvgVolRank"]] = priceData.groupby("Date")[["LowVol", "DownVol", "AvgVol"]].rank(ascending=False, pct=True)
    
        # Compute average volatilities for each theme on each date
        themeScore = priceData.groupby(["Peer", "Date"])[["LowVol", "DownVol", "AvgVol"]].mean()
        
        # Rename columns to reflect theme-based volatility measures
        themeScore.columns = ["PeerLowVol", "PeerDownVol", "PeerAvgVol"]

        # Merge theme scores with price data
        priceData = pd.concat([priceData.set_index(["Peer", "Date"]), themeScore], axis=1).reset_index()

        # Rank theme-based volatilities across all themes daily
        priceData[["PeerLowVol", "PeerDownVol", "PeerAvgVol"]] = priceData.groupby(["Date"])[["PeerLowVol", "PeerDownVol", "PeerAvgVol"]].rank(ascending=False, pct=True)

        # Select relevant columns and rename to remove "Rank" suffix for final output
        df = priceData[["Date", "Symbol", "LowVolRank", "DownVolRank", "AvgVolRank", "PeerLowVol", "PeerDownVol", "PeerAvgVol"]].copy()
        df.columns = df.columns.str.replace("Rank", "")

        # Filter data from January 1, 2006, onwards and reset index
        df = df[df["Date"] >= "2006-01-01"].reset_index(drop=True)
 
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()

        # Iterate over each volatility-related column
        for col in ["LowVol", "DownVol", "AvgVol", "PeerLowVol", "PeerDownVol", "PeerAvgVol"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = df.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)
        scores.columns = scores.columns.str.replace("Peer", self.peer)

        ## Store the Factor Data in dictionary.
        self.factorData["LowVol"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["LowVol"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### LOWVOL COMPLETE ###")

        del priceData, themeScore, df, scores, result

    def generate_AM(self,):

        # Function to calculate log returns
        def calculate_log_returns(df):
            df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
            return df.dropna()

        # Function to calculate annualized standard deviation
        def calculate_annualized_std(df, window=252):
            return df['LogReturn'].rolling(window).std() * np.sqrt(window)

        # Function to calculate momentum ratios
        def calculate_momentum_ratios(series, period):
            return series / series.shift(period) - 1

        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Define periods for momentum ratios (in trading days)
        periods = {
            'MR1': 21,   # 1 month
            'MR2': 42,   # 2 months
            'MR3': 63,   # 3 months
            'MR6': 126,  # 6 months
            'MR12': 252, # 12 months
        }

        # Apply log return calculation
        priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
    
        # Calculate momentum ratios for each period
        for label, period in periods.items():
            priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

        # Calculate annualized standard deviation
        priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

        # Normalize the momentum ratios by dividing by the annualized standard deviation
        for label in periods.keys():
            priceData[label] /= priceData['AnnualizedStd']

        priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)

        # Calculate the mean and std deviation of each momentum ratio across the universe
        for label in periods.keys():
            priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
            priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())

        # Calculate Z-scores for each period
        for label in periods.keys():
            priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']

        # Define specific combinations for which to calculate the final Z-scores
        metrics = ['Z_MR1', 'Z_MR2', 'Z_MR3', 'Z_MR6', 'Z_MR12']

        # Weighted average Z-score
        priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)

        # Normalized momentum score
        priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                1 + priceData[f'WtdZScore'],
                                                (1 - priceData[f'WtdZScore']) ** -1)

        ## Filter the Required Columns  
        priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Computing the Percentile Score of the Symbols on each Date
        priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())

        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)

        ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()

        # Iterate over each volatility-related column
        for col in ["Momentum", "PeerMomentum"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["AM"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["AM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### MOMENTUM COMPLETE ###")

        del priceData, result, temp, scores
    
    def generate_LTMA(self,):

        ## If Price is ot imported form Databse, then read it from DB.
        if self.stockPriceData is None:
            self.__readPriceData()
        
        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Sort the Dataframe by Symbol and Date, Reset the index
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        # Calculate 1-year, 3-year, and 5-year percentage changes in stock closing prices.
        for key in [3, 6, 12, 18]:
            priceData[f"{key}_return"] = priceData.groupby('Symbol')["Close"].pct_change(22*key)
            priceData[f"{key}_vol"] = priceData.groupby('Symbol')["Close"].transform(lambda x: x.pct_change().rolling(key*22).std())
            priceData[f"{key}_vol"] *= np.sqrt(252)
            priceData[f"{key}_sharpe"] = priceData[f"{key}_return"] / priceData[f"{key}_vol"]
            priceData.drop(columns = [f"{key}_return", f"{key}_vol"], inplace = True)

        ## Sort and reset the index
        priceData.sort_values("Date", inplace = True)
    
        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Assigning the rank to each 500 companies on particular date.
        for key in [3, 6, 12, 18]:
            priceData[f"{key}_Rank"]=priceData.groupby('Date')[f"{key}_sharpe"].rank(pct=True)

        ## Aggregate the rank for composite score.
        priceData["LTMA"] = priceData[[col for col in priceData.columns if "Rank" in col]].mean(axis = 1, skipna = False)

        ## Filter out the necessary columns
        priceData = priceData.filter(items = ["Date", "Symbol", "LTMA"])

        ## Convert the long from Dataframe to wide form dataframe
        priceData = priceData.pivot_table(index='Date',columns='Symbol',values='LTMA').reset_index()
        ## Shifting the Date column
        priceData["Date"] = priceData["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        priceData = priceData.melt(id_vars='Date', value_name = "LTMA")
        ## Drop na and reset the index
        priceData = priceData.dropna().reset_index(drop = True)
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["LTMA"] = priceData.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["LTMA"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys() if key != "Theme"], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### LTMA COMPLETE ###")

        del priceData

    def generate_LTM(self,):

        ## If Price is ot imported form Databse, then read it from DB.
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Set the maping dictionary for LTM Factor.
        mapper = periodMapping["LTM"]

        ## Function to calculate 3-year return after shifting by 100 days
        def rollingReturn(group, shift, period, key):
            group[f'{key}_Return'] = group['Close'].ffill().shift(shift).pct_change(period, fill_method = None)
            return group
            
        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Sort the Dataframe by Symbol and Date, Reset the index
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)

        # Calculate 1-year, 3-year, and 5-year percentage changes in stock closing prices.
        for key in mapper.keys():
            priceData = priceData.groupby('Symbol').apply(rollingReturn, shift = mapper[key]["S"], 
                                                            period = mapper[key]["F"], 
                                                            key = key, include_groups = False).reset_index(level = 0)

        ## Sort and reset the index
        priceData.sort_values("Date", inplace = True)
    
        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Assigning the rank to each 500 companies on particular date.
        for key in mapper.keys():
            priceData[f"{key}_Rank"]=priceData.groupby('Date')[f"{key}_Return"].rank(pct=True)
        
        ## Aggregate the rank for composite score.
        priceData["LTM"] = priceData[[col for col in priceData.columns if "Rank" in col]].mean(axis = 1, skipna = False)

        ## Filter out the necessary columns
        priceData = priceData.filter(items = ["Date", "Symbol", "LTM"])

        ## Convert the long from Dataframe to wide form dataframe
        priceData = priceData.pivot_table(index='Date',columns='Symbol',values='LTM').reset_index()
        
        ## Shifting the Date column
        priceData["Date"] = priceData["Date"].shift(-1)

        ## Fill NaN value of date with todays date.
        priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        priceData = priceData.melt(id_vars='Date', value_name = "LTM")

        ## Drop na and reset the index
        priceData = priceData.dropna().reset_index(drop = True)
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["LTM"] = priceData.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["LTM"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys() if key != "Theme"], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### LTM COMPLETE ###")

        del priceData

    def generate_EM(self):
        # RSI function
        def rsi(closes, n):
            diff_serie = closes.diff()
            gain = diff_serie.where(diff_serie > 0, 0)
            loss = -diff_serie.where(diff_serie < 0, 0)
            avg_gain = gain.rolling(window=n).mean()
            avg_loss = loss.rolling(window=n).mean()
            rs = avg_gain / avg_loss
            rsi = 100 - (100 / (1 + rs))
            return rsi.fillna(0)

        ################
        ## PRICE DATA ##
        ################
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
        
        ## Calculate the 50 and 200 day moving average
        priceData['200_ma'] = priceData.groupby('Symbol')['Close'].transform(lambda x: x.rolling(200).mean())
        priceData['50_ma'] = priceData.groupby('Symbol')['Close'].transform(lambda x: x.rolling(50).mean())

        # Calculate Close to 200DMA and 50DMA to 200DMA Ratio.
        priceData['200_ma_ratio'] = priceData['Close'] / priceData['200_ma']
        priceData['50_200_ratio'] = priceData['50_ma'] / priceData['200_ma']

        # Compute the RSI
        priceData['rsi'] = priceData.groupby('Symbol')['Close'].transform(lambda x: rsi(x, 14))

        ## Compute the rank for each of the sub-factors
        priceData['200_ma_ratio_score'] = priceData.groupby('Date')['200_ma_ratio'].rank(ascending=False, pct=True)
        priceData['50_200_score'] = priceData.groupby('Date')['50_200_ratio'].rank(ascending=False, pct=True)
        priceData['rsi_score'] = priceData.groupby('Date')['rsi'].rank(ascending=False, pct=True)

        # Calculate final scores and sor the dataframe
        priceData['finalScore'] = priceData[['200_ma_ratio_score', '50_200_score', 'rsi_score']].sum(axis = 1, skipna = False)

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Rank the top500 companies and filter the necessary columns
        priceData['AM_New'] = priceData.groupby('Date')['finalScore'].rank(ascending=False, pct=True)
        priceData = priceData.filter(items= ['Date','Symbol','AM_New'])

        ## Filter the data after year 2006 and sor the dataframe.
        priceData = priceData[priceData["Date"] >= "2006-01-01"].copy()
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
    
        ## Convert the long from Dataframe to wide form dataframe
        priceData = priceData.pivot_table(index='Date',columns='Symbol',values='AM_New').reset_index()
        ## Shifting the Date column
        priceData["Date"] = priceData["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        priceData.iloc[-1, priceData.columns.get_loc("Date")] = pd.to_datetime(date.today())
        ## Re-converting the wide form dataframe to long form dataframe
        priceData = priceData.melt(id_vars='Date', value_name= "EM")
        ## Drop na and reset the index
        priceData = priceData.dropna().reset_index(drop = True)

        ## Store the Factor Data in dictionary.
        self.factorData["EM"] = priceData.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["EM"] = priceData[priceData["Date"] == priceData["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### EM COMPLETE ###")

        del priceData

    def generate_Dividend(self, valueYieldCall = False):

        def previousQuarter(currDate):
            if pd.Period(currDate,freq = "Q").end_time.date() == currDate.date():
                return currDate
            else:
                return currDate - pd.offsets.QuarterEnd()
            
        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossGrowth is None:
            self.__readProfitLossGrowth() 
    
        if self.stockPriceData is None:
            self.__readPriceData()
            
        # Fetch the rebalance dates for the last 100 years from the "GrowthDate" table in the database
        # rebalanceDates = self.__dataCursor.fetch_data_from_database(table_name = "GrowthDate", no_of_years=100)
        rebalanceDates = pd.read_csv('growth_date_latest.csv', parse_dates=['Date'])

        # Create a copy of the strategy universe (likely a dataframe or similar structure) to work with for company composition
        composition  = self.strategyUniverse.copy()

        # Extract the quarter and date values from the rebalanceDates dataframe
        rebalanceQtr = list(rebalanceDates['Quarter'])
        rebalanceDate = list(rebalanceDates['Date'])

        # Create a dictionary mapping each date to its corresponding quarter and vice versa
        qtrDateDict = dict(zip(rebalanceDate, rebalanceQtr))
        dateQtrDict = dict(zip(rebalanceQtr, rebalanceDate))

        # Add a new column 'Quarter' to the composition dataframe by mapping the 'Date' column to the respective quarter using qtrDateDict
        composition['Quarter'] = composition['Date'].map(qtrDateDict)

        # Filter the necessary columns ('FINCODE', 'Date_End', 'Dividend payout ratio', 'PAT') from the profitLossGrowth dataframe
        div_qtr = self.profitLossGrowth.filter(items=["FINCODE", "Date_End", "Dividend payout ratio", "PAT"])

        # Merge div_qtr with companyMaster on the 'FINCODE' column to include company information
        div_qtr = pd.merge(div_qtr, self.companyMaster, on="FINCODE", how="inner")

        # Convert the 'Date_End' column to datetime format
        div_qtr["Date_End"] = pd.to_datetime(div_qtr['Date_End'], format="%Y%m")

        # Shift 'Date_End' to the respective month-end date
        div_qtr["MonthEnd"] = div_qtr["Date_End"] + pd.offsets.MonthEnd()

        # Create a new column 'QuarterEnd' to calculate the end of the financial quarter based on the 'MonthEnd'
        div_qtr["QuarterEnd"] = div_qtr["MonthEnd"].apply(previousQuarter) 

        # Convert the 'QuarterEnd' to the Indian Financial Year format, which ends in March
        div_qtr["YearQuarter"] = div_qtr["QuarterEnd"].dt.to_period("Q-MAR")

        # Extract the quarter and year from 'YearQuarter' and create a new 'Quarter' column
        div_qtr["Quarter"] = div_qtr["YearQuarter"].astype(str).str[-2:] + div_qtr["YearQuarter"].astype(str).str[:4]

        # Extract the year from 'YearQuarter' and create a 'Year' column
        div_qtr["Year"] = div_qtr["YearQuarter"].astype(str).str[:4].astype(int)

        # Sort the dataframe by 'YearQuarter' in ascending order for chronological order
        div_qtr.sort_values("YearQuarter", ascending=True, inplace=True)
        div_qtr.reset_index(drop=True, inplace=True)

        # Calculate the dividend by multiplying 'PAT' (Profit After Tax) by the 'Dividend payout ratio'
        div_qtr['dividend'] = div_qtr['PAT'] * div_qtr['Dividend payout ratio']

        # Sort by 'Symbol' and 'MonthEnd' for proper chronological order within each company
        div_qtr.sort_values(['Symbol', 'MonthEnd'], inplace=True)

        # Calculate the rolling dividend over the last 4 quarters for each company
        div_qtr['Rollingdividend'] = div_qtr.groupby('Symbol')['dividend'].transform(lambda x: x.rolling(4).sum())

        # Sort the dataframe again by 'Symbol' and 'YearQuarter', and reset the index
        div_qtr.sort_values(["Symbol", "YearQuarter"], inplace=True)
        div_qtr.reset_index(drop=True, inplace=True)

        # Map the 'Quarter' to its respective 'Date' using the dateQtrDict for final data
        div_qtr['Date'] = div_qtr['Quarter'].map(dateQtrDict)

        # Merge the 'div_qtr' with the 'composition' dataframe on 'Symbol', 'Date', and 'Quarter' to get the final composition
        div_qtr = pd.merge(div_qtr, composition, on=['Symbol', 'Date', 'Quarter'])

        # Extract the year from the 'Date' column and assign it to a new 'Year' column
        div_qtr['Year'] = div_qtr['Date'].dt.year

        # Sort the dataframe by 'Date' and 'Symbol' for easier analysis and reset the index
        div_qtr = div_qtr.sort_values(["Date", "Symbol"]).reset_index(drop=True)

        # Create a dataframe 'divRank' containing only relevant columns like dividend, PAT, and Rollingdividend
        divRank = div_qtr.filter(items=["Date", "Symbol", "dividend", "PAT", 'Dividend payout ratio', 'Rollingdividend'])

        # Map the 'divRank' to the 'composition' dataframe on 'Date' and 'Symbol' to align dividends with composition details
        divRank = pd.merge(composition, divRank, on=['Date', 'Symbol'], how='left')

        # Sort the 'divRank' dataframe by 'Symbol' and 'Date' for proper chronological order
        divRank = divRank.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        # Forward fill missing values in 'dividend', 'PAT', 'Dividend payout ratio', and 'Rollingdividend' within each 'Symbol'
        divRank[['dividend', 'PAT', 'Dividend payout ratio', 'Rollingdividend']] = divRank.groupby('Symbol')[['dividend', 'PAT', 'Dividend payout ratio', 'Rollingdividend']].ffill()

        # Calculate the dividend yield for each company as (Rollingdividend / Market Capitalization) * 100
        divRank['yield'] = (divRank['Rollingdividend'] / divRank['Mcap']) * 100

        # Calculate the 4-quarter rolling average of the dividend yield for each company
        divRank["yield"] = divRank.groupby("Symbol")["yield"].transform(lambda x: x.rolling(4).mean())

        # Final dataframe 'divRankFinal' contains 'Date', 'Symbol', and 'Dividend' columns, with no missing values
        divRankFinal = divRank[['Date', 'Symbol', 'yield']].dropna()

        # Rename the 'yield' column to 'Dividend' for better clarity
        divRankFinal.rename(columns={'yield': 'Dividend'}, inplace=True)

        if valueYieldCall:
            return divRankFinal

        ## Convert the long from Dataframe to wide form dataframe
        divRankFinal = divRankFinal.pivot_table(index='Date',columns='Symbol',values='Dividend').reset_index()

        ## Shifting the Date column
        divRankFinal["Date"] = divRankFinal["Date"].shift(-1)
        
        ## Fill NaN value of date with todays date.
        divRankFinal.iloc[-1, divRankFinal.columns.get_loc("Date")] = pd.to_datetime(date.today())
        
        ## Re-converting the wide form dataframe to long form dataframe
        divRankFinal = divRankFinal.melt(id_vars='Date', value_name= "Dividend")

        ## Drop na and reset the index
        divRankFinal = divRankFinal.dropna().reset_index(drop = True)
        divRankFinal = divRankFinal[divRankFinal["Date"] >= "2006-01-01"].reset_index(drop = True).copy()
        divRankFinal['Dividend']=divRankFinal.groupby('Date')['Dividend'].rank(pct=True)

        ## Store the Factor Data in dictionary.
        self.factorData["Dividend"] = divRankFinal.copy()
        
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["Dividend"] = divRankFinal[divRankFinal["Date"] == divRankFinal["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### DIVIDEND COMPLETE ###")

        del div_qtr, divRank, divRankFinal

    def generate_Growth(self,valueYieldCall = False):

        ## FUnction to get the previous Quarter Date
        def previousQuarter(currDate):
            if pd.Period(currDate,freq = "Q").end_time.date() == currDate.date():
                return currDate
            else:
                return currDate - pd.offsets.QuarterEnd()

        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossGrowth is None:
            self.__readProfitLossGrowth()

        if self.cashFlow is None:
            self.__readCashFlow()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()


        ######################
        ## DATA PREPARATION ##
        ######################
        ## List of the Rebalance Dates
        # rebalanceDates = self.__dataCursor.fetch_data_from_database(table_name = "GrowthDate", no_of_years=100)
        rebalanceDates = pd.read_csv('growth_date_latest.csv', parse_dates=['Date'])
        
        # Identify Companies which have been in Top-500 historically
        composition  = self.strategyUniverse.copy()
        
        ## List of Quarter and Dates
        rebalanceQtr=list(rebalanceDates['Quarter'])
        rebalanceDate=list(rebalanceDates['Date'])
        
        ## Mappinf Dictionary of Date to Quarter and vice versa
        qtrDateDict=dict(zip(rebalanceDate,rebalanceQtr))
        dateQtrDict=dict(zip(rebalanceQtr,rebalanceDate))

        ## Quarter column in PriceData
        composition['Quarter']=composition['Date'].map(qtrDateDict)

        ############################
        ## OPERATION ON CASH DATA ##
        ############################
        
        ## Import the Cash data
        cf_yearly = self.cashFlow.filter(items = ["FINCODE", "Year_end", "Cash_from_Operation"])
        ## Map the Company Names
        cf_yearly = pd.merge(cf_yearly, self.companyMaster, how = "inner", on = "FINCODE")
        ## Convert the string into Datetime
        cf_yearly["Year_end"] = pd.to_datetime(cf_yearly['Year_end'],format="%Y%m")
        cf_yearly["Year"] = cf_yearly["Year_end"].dt.year
        ## Shift the Date Year
        cf_yearly['Year'] = np.where(cf_yearly['Year_end'].dt.month == 12, 
                                    (cf_yearly['Year'] + 1), 
                                    (cf_yearly['Year']))
        cf_yearly["Year"] = cf_yearly["Year"] + 1
        ## Sort the dataframe
        cf_yearly.sort_values('Year',inplace=True)
        cf_yearly.reset_index(drop = True, inplace = True)

        self.cf_yearly = cf_yearly.copy()

        ###################################
        ## OPERATION ON PROFIT/LOSS DATA ##
        ###################################
        ## Read the Fundamental data of the comapnies - Growth Related
        pl_yearly = self.profitLossGrowth.drop(columns = [ 'Debt/Equity Ratio', 'Adj_eps_abs', 'Dividend payout ratio']).copy()
        ## Convert the string into Datetime
        pl_yearly["Date_End"] = pd.to_datetime(pl_yearly['Date_End'],format="%Y%m")

        ## Merge the Fundamental data with master list of the companies 
        pl_yearly = pd.merge(pl_yearly, self.companyMaster, on = "FINCODE", how = "inner")

        ## Calculate the Operating_Margin and Gross_Margin
        pl_yearly["Operating_Margin"] = pl_yearly["OPERATING_PROFIT"].div(pl_yearly["NET_SALES"])
        pl_yearly["Gross_Margin"] = pl_yearly["GROSS_PROFIT"].div(pl_yearly["NET_SALES"])

        # ## Shift Date to respective Month End date.
        pl_yearly["MonthEnd"] = pl_yearly["Date_End"] + pd.offsets.MonthEnd()
        pl_yearly["QuarterEnd"] = pl_yearly["MonthEnd"].apply(previousQuarter) 

        ## Convert the Dates as per Indian Financial Quarter
        pl_yearly["YearQuarter"] = pl_yearly["QuarterEnd"].dt.to_period("Q-MAR")
        pl_yearly["Quarter"] = pl_yearly["YearQuarter"].astype(str).str[-2:] + pl_yearly["YearQuarter"].astype(str).str[:4]
        pl_yearly["Year"] =pl_yearly["YearQuarter"].astype(str).str[:4].astype(int)

        ## sort the Data Frame based on Quarter
        pl_yearly.sort_values("YearQuarter", ascending = True, inplace = True)
        pl_yearly.reset_index(drop = True, inplace = True)

        self.pl_yearly = pl_yearly.copy()

        ###############
        ## COMBINING ##
        ###############
        ## Combine the Profit loss and Cash data
        data = pd.merge(pl_yearly, cf_yearly, on = ['FINCODE',"Symbol", "Year"])

        ## List of Factors
        factorUniverse = ['PAT','OPERATING_PROFIT','GROSS_PROFIT','NET_SALES','EPS_DILUTED','PBT','Operating_Margin','Gross_Margin','Cash_from_Operation']
        ## SUbste the columns
        data = data[["Year", "YearQuarter", "Quarter", "Symbol", "Date_End"] + factorUniverse]

        ## Sort the DataFrame
        data.sort_values(["Symbol", "YearQuarter"], inplace = True)
        data.reset_index(drop = True, inplace = True)

        ## Calculating the Change in factors over a year
        year_chg = lambda ser: (ser - ser.shift(4))/ser.abs().shift(4)
        data[[f"{c}_change" for c in factorUniverse]] = data.groupby("Symbol", group_keys=False)[factorUniverse].apply(year_chg)
    
        ## Mapping the Date column to Quarter Date
        data['Date']=data['Quarter'].map(dateQtrDict)
        
        data=pd.merge(data,composition,on=['Symbol','Date','Quarter'])
        data['Year']=data['Date'].dt.year

        ## Compute the Absolute / Peer and Historical Rank
        data = data.sort_values(["Date", "Symbol"]).reset_index(drop = True)

        if valueYieldCall:
            return data

        data.drop(columns = ["Date_End"], inplace = True)

        factorUniverse = [f"{col}_change" for col in factorUniverse]
        data["AbsRank"] = data.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        data["PeerRank"] = data.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        data = data.sort_values(["Symbol", "Date"]).reset_index(drop = True)
        data["HistRank"] = data.groupby("Symbol")[factorUniverse].rolling(window = 12, min_periods = 4).rank(pct = True).mean(axis = 1).reset_index(drop = True)
        growthRank = data.filter(items = ["Date", "Symbol", "AbsRank", "PeerRank", "HistRank"])
        growthRank["Rank"] = growthRank[[ "AbsRank", "PeerRank", "HistRank"]].mean(axis = 1)
        
        ## Mapping the Growth score to each date
        growthRank = pd.merge(composition,growthRank,on=['Date','Symbol'],how='left')
        growthRank = growthRank.sort_values(["Symbol",  "Date"]).reset_index(drop = True)

        ## Forward fill the growth score, as growth is for each quarter
        growthRank[['AbsRank','PeerRank','HistRank','FinalRank']]=growthRank.groupby('Symbol')[['AbsRank','PeerRank','HistRank','Rank']].ffill()
        growthRank['Growth'] = growthRank['PeerRank']*0.95 + growthRank['AbsRank']*0.05

        ## Convert the long from Dataframe to wide form dataframe
        growthRank = growthRank.pivot_table(index='Date',columns='Symbol',values='Growth').reset_index()
        ## Shifting the Date column
        growthRank["Date"] = growthRank["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        growthRank.iloc[-1, growthRank.columns.get_loc("Date")] = pd.to_datetime(date.today())
        ## Re-converting the wide form dataframe to long form dataframe
        growthRank = growthRank.melt(id_vars='Date', value_name= "Growth")
        ## Drop na and reset the index
        growthRank = growthRank.dropna().reset_index(drop = True)
        growthRank = growthRank[growthRank["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["Growth"] = growthRank.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["Growth"] = growthRank[growthRank["Date"] == growthRank["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### GROWTH COMPLETE ###")

    def generate_ValueYield(self):

        ## Fetch the Stock Value Data
        if self.stockValueData is None:
            self.__readValueData()

        ## Fetch the 
        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        
        ### VALUE ###
        # Filter the rows for Top-500 companies historically
        valueData = self.stockValueData[self.stockValueData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].copy()

        ## List of Fundamental Factor Value.
        fundamentalValueFactor = ["EV_EBITDA", "PS", "PB", "PE"]
        ## Replace -ve value with NaN value for all the fundamental value factor
        for factor in fundamentalValueFactor:
            valueData[factor] = np.where(valueData[factor] <= 0, np.nan, valueData[factor])
            valueData[factor] = 1/valueData[factor]

        ## Sort and reset the index
        valueData.sort_values(["Symbol", "Date"], inplace = True)
        valueData.reset_index(drop = True, inplace = True)


        ### Dividend ###
        dividendData = self.generate_Dividend(valueYieldCall = True)

        ## EPS ###
        epsData = self.generate_Growth(valueYieldCall = True)

        ## ROE
        roe = self.generate_QualityQuarter(valueYieldCall=True)
        roe["Date_End"] = pd.to_datetime(roe['Date_End'],format="%Y%m")

        growth = pd.merge(epsData[["Date", "Date_End", "Symbol","EPS_DILUTED_change"]], 
                          roe[["Date_End", "Symbol", "ROE_ttm"]], on = ["Date_End", "Symbol"], how = "left")

        final=pd.merge(valueData,dividendData,on=['Date','Symbol'])
        final=pd.merge(final,growth,on=['Date','Symbol'],how='left')

        final = final.sort_values(["Symbol","Date"]).reset_index(drop = True)
        final[['ROE_ttm', "EPS_DILUTED_change"]]=final.groupby('Symbol')[['ROE_ttm', "EPS_DILUTED_change"]].ffill()
        final['PEG']=final['PE']/final['EPS_DILUTED_change']

        factorUniverse = ['EV_EBITDA', 'PS','PB', 'PE', 'Dividend','PEG','ROE_ttm']

        final = pd.merge(self.strategyUniverse, final[["Date", "Symbol"] + factorUniverse], on = ["Date","Symbol"], how = "left")

        final["AbsRank"] = final.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        final["PeerRank"] = final.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        final['ValueYield'] = final['PeerRank']*0.95 + final['AbsRank']*0.05
        final['ValueABS'] = final['PeerRank']*0.05 + final['AbsRank']*0.95        

        result = list()

        # Iterate over each volatility-related column
        for col in ["ValueYield", "ValueABS"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = final.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))


        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        ## Store the Factor Data in dictionary.
        self.factorData["ValueYield"] = scores.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ValueYield"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### ValueYield COMPLETE ###")

    def generate_QualityAnnual(self,):

        ## Function to calculate the volatility of the ratio
        def calculateVol(series):
            _max = series.rolling(window = 5, min_periods = 3).max()
            _min = series.rolling(window = 5, min_periods = 3).min()
            _mean = series.rolling(window = 5, min_periods = 3).mean() 
            return (_max - _min)/(_mean)

        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossQuality is None:
            self.__readProfitLossQuality()

        if self.financialRatio is None:
            self.__readFinancialRatio()

        if self.cashFlow is None:
            self.__readCashFlow()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        ## Read the Rebalance Dates
        # rebalanceDates = self.__dataCursor.fetch_data_from_database(table_name = "QualityDate", no_of_years = 100)
        rebalanceDates = pd.read_csv('QualityDate.csv')
        rebalanceDates["Date"] = pd.to_datetime(rebalanceDates["Date"])
        rebalances = rebalanceDates["Date"].tolist()

        ## Mapping the Data Receive Year
        priceData = pd.merge(self.strategyUniverse, rebalanceDates, on = "Date", how = "left")
        priceData = priceData.sort_values(["Symbol", "Date"]).reset_index(drop = True)
        
        ## Filter the dataframe for the rebalance dates.
        priceDataTop500Rebal = priceData[priceData["Date"].isin(rebalances)]

        ############################
        ### PROFIT and LOSS DATA ###
        ############################
        ## Mapping Company Symbol to Cash Flow Data
        profitLoss = pd.merge(self.profitLossQuality, self.companyMaster, on = ["FINCODE"], how = "inner")

        ## Calculating the Operating Margin and Gross Margin
        profitLoss["OperatingMargin"] = profitLoss["Operating_profit"].div(profitLoss["Net_sales"])
        profitLoss["GrossMargin"] = profitLoss["Gross_profits"].div(profitLoss["Net_sales"])

        ## Convert the Date String to python datetime
        profitLoss["Year_end"] = pd.to_datetime(profitLoss["Year_end"], format = "%Y%m")
        profitLoss["Year"]  = profitLoss["Year_end"].dt.year
        profitLoss["Year"] = np.where(profitLoss["Year_end"].dt.month == 12, profitLoss["Year"]+1, profitLoss["Year"])

        ## Drop the duplicates and Keep the last record
        profitLoss = profitLoss.sort_values(["Year_end", "Symbol"])#.drop_duplicates(["Year", "Symbol"], keep = "last")

        ## Sort the dataframe by Year column and reset the dataframe
        profitLoss.sort_values("Year", inplace = True)
        profitLoss.reset_index(drop = True, inplace = True)
        
        ############################
        ### FINANCIAL RATIO DATA ###
        ############################
        ## Replace NaN value with 0
        financialRatio = self.financialRatio.copy()
        financialRatio["Payable_days"] = financialRatio["Payable_days"].fillna(0)
        ## Calculating the Working Capital Days
        financialRatio["WC_Days"] = financialRatio["Inventory_Days"] + financialRatio["Receivable_days"] - financialRatio["Payable_days"]
        ## Mapping COmpany Symbol to Financial Ratio data
        financialRatio = pd.merge(financialRatio, self.companyMaster, on = ["FINCODE"], how = "inner")

        ## Convert the Date string to python datetime
        financialRatio["Year_end"] = pd.to_datetime(financialRatio.Year_end, format = "%Y%m")
        financialRatio["Year"] = financialRatio["Year_end"].dt.year
        financialRatio["Year"] = np.where(financialRatio["Year_end"].dt.month == 12, financialRatio["Year"] + 1, financialRatio["Year"])

        ## Drop the duplicates and Keep the last record
        financialRatio = financialRatio.sort_values(["Year_end", "Symbol"])#.drop_duplicates(["Year", "Symbol"], keep = "last")

        ## Sort the dataframe by Year column and reset the dataframe
        financialRatio.sort_values("Year", inplace = True)
        financialRatio.reset_index(drop = True, inplace = True)

        ######################
        ### CASH FLOW DATA ###
        ######################
        ## Mapping Company Symbol to Cash Flow Data
        cashFlow = self.cashFlow[["FINCODE", "Year_end","Cash_from_Operation"]].copy()
        cashFlow = pd.merge(cashFlow, self.companyMaster, on = ["FINCODE"], how = "inner")

        ## Convert the Date String to python datetime
        cashFlow["Year_end"] = pd.to_datetime(cashFlow["Year_end"], format = "%Y%m")
        cashFlow["Year"]  = cashFlow["Year_end"].dt.year
        cashFlow["Year"] = np.where(cashFlow["Year_end"].dt.month == 12, cashFlow["Year"]+1, cashFlow["Year"])

        ## Drop the duplicates and Keep the last record
        cashFlow = cashFlow.sort_values(["Year_end", "Symbol"])#.drop_duplicates(["Year", "Symbol"], keep = "last")
        ## Sort the dataframe by Year column and reset the dataframe
        cashFlow.sort_values("Year", inplace = True)
        cashFlow.reset_index(drop = True, inplace = True)

        ## Merge the dataframes of Profit and Loss, Cash Flow and Financial Ratios into one dataframe
        data = ft.reduce(lambda left, right: pd.merge(left, right, on=['Symbol','Year','FINCODE']), [profitLoss, financialRatio, cashFlow])
        ## Calculating Cash Flow Operation (CFO) BY EBITDA  and Free cash Flow (FCF) COnversion ratio.
        data["CFO_By_EBITDA"] = data["Cash_from_Operation"].div(data["Operating_profit"])
        data["FCF_Conversion"] = data["FCF_Share"].div(data["Adj_Eps"])

        ## Sort the dataframe and reset the index
        data.sort_values(["Symbol", "Year"], inplace = True)
        data.reset_index(drop = True, inplace = True)

        ## Calculating the Volatility of Selected Financial ratio ("ROE", "Adj_EPS", "CEPS", "OperatingMargin")
        tempFactors = ['ROE', 'Adj_Eps', 'CEPS', 'OperatingMargin']
        data[[f"Vol_{value}" for value in tempFactors]] = data.groupby("Symbol", group_keys = False)[tempFactors].apply(calculateVol)

        ## Calculating the Percentage of Selected Financila Ratio ("Net_Sales", "Oprating_profit", "Adj_Eps", "Cash_from_operation", "FCF_Share")
        tempFactors = ["Net_sales", "Operating_profit", "Adj_Eps", "Cash_from_Operation", "FCF_Share"]
        data[[f"{value}_Growth" for value in tempFactors]] = data.groupby("Symbol", group_keys = False)[tempFactors].apply(lambda ser: ser.pct_change())

        ## Consolidated List of individual factors to create the Consolidated Quality Factor
        factorUniverse = ["FCF_Share", "OperatingMargin", "GrossMargin", "CFO_By_EBITDA", "ROE", "ROCE",
                        "Total_Debt_Equity", "Interest_Cover", "ROA", "Inventory_Days", "Receivable_days",
                        "WC_Days", "FCF_Conversion","Vol_ROE", "Vol_Adj_Eps", "Vol_CEPS", "Vol_OperatingMargin",
                        "Dividend_Payout_Per"]

        # "Growth"
        ## Indivodual facrtor in universe are divided in two list, one to be ranked in ascending order and other list to be ranked in descending order.
        ascendingFactorUniverse = ["CFO_By_EBITDA", "Dividend_Payout_Per", 
                                "FCF_Conversion",  "FCF_Share", "GrossMargin", "Interest_Cover", "OperatingMargin", "ROA",
                                "ROCE", "ROE"]

        descendingFactorUniverse = ["Inventory_Days", "Receivable_days", "WC_Days", "Total_Debt_Equity", 
                                    "Vol_ROE", "Vol_Adj_Eps", "Vol_CEPS", "Vol_OperatingMargin"]
        
        ## Filter the Necessary columns
        data = data.filter(items = ["Symbol", 'Year'] + factorUniverse)
        
        ## Mapping the Factor to Stock price Data.
        rank = pd.merge(priceDataTop500Rebal, data, on = ["Symbol", "Year"])

        ## Sort the Dataframe and reset the index
        rank.sort_values("Date", inplace = True)
        rank.reset_index(drop = True, inplace = True)

        ## Individual Absolute rank for Ascending and Descending Factor
        absAscRank = rank.groupby("Year")[ascendingFactorUniverse].rank(ascending = True, pct = True)
        absDescRank = rank.groupby("Year")[descendingFactorUniverse].rank(ascending = False, pct = True)
        ## Take the mean of individual factor to get Aboslute rank for each stock
        rank["AbsoluteRank"] = pd.concat([absAscRank, absDescRank],axis = 1).mean(axis = 1)

        ## Individual Peer rank for Ascending and Descending Factor
        peerAscRank = rank.groupby(["Year", "Sector"])[ascendingFactorUniverse].rank(ascending = True, pct = True)
        peerDescRank = rank.groupby(["Year", "Sector"])[descendingFactorUniverse].rank(ascending = False, pct = True)
        ## Take the mean of individual factor to get Peer rank for each stock
        rank["PeerRank"] = pd.concat([peerAscRank, peerDescRank],axis = 1).mean(axis = 1)

        ## Mapping the Rank against The dates of each stock.
        quality = pd.merge(priceData, rank[["Date", "Symbol", "AbsoluteRank", "PeerRank"]], on = ["Date", "Symbol"], how = "left")

         ## Sort the dataframe and reset the index
        quality.sort_values(["Symbol", "Date"], inplace = True)
        quality.reset_index(drop = True, inplace = True)
    
        ## Forward fill the Rank
        quality[["AbsoluteRank", "PeerRank"]] = quality.groupby("Symbol", group_keys = False)[["AbsoluteRank", "PeerRank"]].ffill()
    
        ## Selecting the relevant columns
        quality = quality.filter(["Date", "Symbol", "AbsoluteRank", "PeerRank"])

        ## Calculate the Weighted Quality Factor Score
        quality["QualityFactor"] = quality["AbsoluteRank"] * 0.05 + quality["PeerRank"] * 0.95
        ## Selecting the relevant columns
        quality = quality.filter(["Date", "Symbol", "QualityFactor"])

        ## Convert the long from Dataframe to wide form dataframe
        quality = quality.pivot_table(index='Date',columns='Symbol',values='QualityFactor').reset_index()
        ## Shifting the Date column
        quality["Date"] = quality["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        quality.iloc[-1, quality.columns.get_loc("Date")] = pd.to_datetime(date.today())
        ## Re-converting the wide form dataframe to long form dataframe
        quality = quality.melt(id_vars='Date', value_name= "QualityAnnual")
        ## Drop na and reset the index
        quality = quality.dropna().reset_index(drop = True)
        quality = quality[quality["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

        ## Store the Factor Data in dictionary.
        self.factorData["QualityAnnual"] = quality.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["QualityAnnual"] = quality[quality["Date"] == quality["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys() ], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### QUALITY ANNUAL COMPLETE ###")
            
    def generate_QualityQuarter(self, valueYieldCall = False):

        # Quarterly Networth Calculation
        def fill_networth(row, prev_networth):
            if pd.isna(row['NetWorth']):
                return prev_networth + row['PAT']
            else:
                return row['NetWorth']

        def fill_networth_column(df):
            df = df.sort_values(by=['FINCODE', 'Date_End']) 
            df['NetWorth_Filled'] = np.nan
            
            for fincode in df['FINCODE'].unique():
                prev_networth = None
                for i, row in df[df['FINCODE'] == fincode].iterrows():
                    if prev_networth is None:
                        prev_networth = row['NetWorth']
                    else:
                        df.at[i, 'NetWorth_Filled'] = fill_networth(row, prev_networth)
                        prev_networth = df.at[i, 'NetWorth_Filled']
            
            return df
        
        #  EPS Growth Calculation
        def calculate_yoy_eps_growth(eps):
            """Calculates YoY EPS Growth based on the provided rules."""
            growth = []
            for i in range(len(eps)):
                if i < 4:
                    growth.append(np.nan) 
                else:
                    prev_eps = eps.iloc[i - 4]  
                    curr_eps = eps.iloc[i]      
                    if prev_eps > 0:
                        growth.append((curr_eps - prev_eps) / prev_eps)
                    elif prev_eps < 0:
                        growth.append(-(curr_eps - prev_eps) / prev_eps)
                    else:
                        growth.append(np.nan)  
            return pd.Series(growth, index=eps.index)  

        # Calculate 5-year mean and std deviation for each quarter separately
        def calc_quarterly_stats(group):
            group = group.sort_values('Date_End')
            group['Mean_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=5, min_periods=1).mean()
            group['Std_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=5, min_periods=1).std()
            return group
        
        # Weighted Average Z Quality Score
        def calculate_weighted_avg_z(row):
            if row['Sector'] == 'Bank':
                return (1/2) * row['Z_ROE_ttm_fin'] - (1/2) * abs(row['Z_EPS_Growth_Variability_fin'])
            else:
                return (1/3) * row['Z_ROE_ttm'] - (1/3) * abs(row['Z_DE']) - (1/3) * abs(row['Z_EPS_Growth_Variability'])
        
        # Define a function to check for negatives in the last 16 quarters
        def has_negative_in_last_4_years(series):
            return series.rolling(window=16, min_periods=16).apply(lambda x: (x < 0).any(), raw=True)

        #########################
        ## COMPANY MASTER DATA ##
        #########################
        if self.companyMaster is None:
            self.__readCompanyMaster()

        if self.profitLossGrowth is None:
            self.__readProfitLossGrowth()

        if self.financeBalanceSheet is None:
            self.__readFinanceBalanceSheet()

        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()


        price_data = self.stockPriceData.copy()
        top_500 = self.strategyUniverse.copy()

        # rebalDates = self.__dataCursor.fetch_data_from_database(table_name="GrowthDate", no_of_years=50)
        rebalDates = pd.read_csv('growth_date_latest.csv', parse_dates=['Date'])
        Rebalance_Qtr=list(rebalDates['Quarter'])
        Rebalance_Date=list(rebalDates['Date'])
        Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))
        date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))

        top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)

        ## Financial Balance Sheet Modfication
        financialBalanceSheet = self.financeBalanceSheet[["Year_end", "FINCODE", "Share_Capital", "Reserve"]]
        financialBalanceSheet["NetWorth"] = financialBalanceSheet[["Share_Capital", "Reserve"]].sum(axis = 1)
        financialBalanceSheet  = pd.merge(financialBalanceSheet, self.companyMaster[["FINCODE", "Symbol"]], on = "FINCODE")
        financialBalanceSheet = financialBalanceSheet[["Year_end", "FINCODE", "Symbol", "NetWorth"]]

        ## PAT Modification
        pat = self.profitLossGrowth[["Date_End", "FINCODE", "PAT"]]
        pat["PAT"] /= 10

        patBS = pd.merge(pat, financialBalanceSheet, left_on = ["Date_End", "FINCODE"], 
                right_on = ["Year_end", "FINCODE"], how = "left")
        patBS['Symbol'] = patBS.groupby('FINCODE', group_keys=False)['Symbol'].apply(lambda x: x.fillna(method='ffill'))
        patBS.drop(columns = ["Year_end"], inplace = True)

        # Apply the function
        patBS = fill_networth_column(patBS)
        # Fill any remaining NaN in the original NetWorth column with the filled values
        patBS['NetWorth'] = patBS['NetWorth'].combine_first(patBS['NetWorth_Filled'])
        patBS.drop(columns=['NetWorth_Filled'], inplace=True)  
        patBS = patBS.dropna()

        # Calculating ROE from scratch
        patBS['ROE'] = patBS.groupby('Symbol', group_keys=False).apply(lambda x: x['PAT']/x['NetWorth'])
        patBS['ROE_ttm'] = patBS.groupby('Symbol', group_keys=False)['ROE'].apply(lambda x: x.rolling(window=4).sum())

        self.patBS = patBS.copy()

        roe_ttm = patBS[['FINCODE', 'Date_End', 'ROE_ttm']].copy()

        if valueYieldCall:
            return  patBS[['FINCODE', "Symbol", 'Date_End', 'ROE_ttm']]

        quality = self.profitLossGrowth[['FINCODE','Date_End','Debt/Equity Ratio', 'Adj_eps_abs']]
        quality = pd.merge(quality, roe_ttm, on=['FINCODE', 'Date_End'])
        pl_quality = pd.merge(quality, self.companyMaster, on = "FINCODE", how='inner')

        self.pl_quality = pl_quality.copy()
    
        # Getting required Quality ratios
        pl_quality = pl_quality[['FINCODE', "Symbol", 'Date_End', 'Debt/Equity Ratio', 'ROE_ttm', 'Adj_eps_abs']].reset_index(drop=True)

        # Removing the rows with NaN Symbol Names
        pl_quality = pl_quality[pl_quality["Symbol"].notna()]

        # Filtering Stocks which have been historically in top 500 universe only.
        pl_quality = pl_quality[pl_quality["Symbol"].isin(top_500.Symbol.unique())]

        # Calculate Adjusted EPS TTM
        pl_quality['Adj_eps_abs_TTM'] = pl_quality.groupby('Symbol')['Adj_eps_abs'].transform(lambda x: x.rolling(4).sum())

        # Apply the function to create a flag for exclusion
        pl_quality['exclude_flag'] = pl_quality.groupby('Symbol')['Adj_eps_abs_TTM'].transform(has_negative_in_last_4_years)

        # Remove symbols with fewer than 16 quarters of data
        # Count the number of quarters for each symbol
        symbol_quarter_counts = pl_quality.groupby('Symbol').size()

        # Create a list of symbols with at least 16 quarters
        valid_symbols = symbol_quarter_counts[symbol_quarter_counts >= 16].index

        # Filter the DataFrame to keep only rows with valid symbols
        pl_quality = pl_quality[pl_quality['Symbol'].isin(valid_symbols)]

        # Filter out rows based on the exclude flag
        b_filtered = pl_quality[pl_quality['exclude_flag'] != 1].reset_index(drop=True)

        # Drop the intermediate column used for filtering
        b_filtered = b_filtered.drop(columns=['exclude_flag'])

        # Mapping GICS to each Symbol
        df = pd.merge(b_filtered, self.sectorData[['Symbol', 'Sector']], on='Symbol', how='left')

        # Calculating ROE for TTM for the past 4 Years years
        df['ROE_ttm'] = df.groupby('Symbol')['ROE_ttm'].transform(lambda x : x.rolling(16).mean())

        # Create a new column for quarter information
        df['Quarter'] = pd.to_datetime(df['Date_End'], format='%Y%m').dt.quarter

        # Calculating YoY EPS Growth for each quarter
        df['YoY_EPS_Growth'] = df.groupby('Symbol')['Adj_eps_abs'].transform(calculate_yoy_eps_growth)

        # Apply function to each SYMBOL and Quarter group
        df = df.groupby(['Symbol', 'Quarter'], group_keys=False).apply(calc_quarterly_stats)

        # Dropping Quarter column after calculation to keep the original dataframe structure
        df.drop(columns=['Quarter'], inplace=True)

        # Calculating rolling 5-year mean and std deviation for EPS growth
        df['Mean_YoY_EPS_Growth'] = df.groupby('Symbol')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=20).mean())
        df['Std_YoY_EPS_Growth'] = df.groupby('Symbol')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=20).std())

        # Calulating EPS Growth Variability
        df['EPS_Growth_Variability'] = df['Mean_YoY_EPS_Growth'].div(df['Std_YoY_EPS_Growth'])

        # Split data into financial and non-financial sectors
        financials_df = df[df['Sector'] == 'Bank']
        non_financials_df = df[df['Sector'] != 'Bank']

        # Calculate mean and std for financials
        financials_df['mean_roe_fin'] = financials_df.groupby('Date_End')['ROE_ttm'].transform('mean')
        financials_df['std_roe_fin'] = financials_df.groupby('Date_End')['ROE_ttm'].transform('std')

        financials_df['mean_eps_growth_variability_fin'] = financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
        financials_df['std_eps_growth_variability_fin'] = financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

        # Calculate mean and std for non-financials
        non_financials_df['mean_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('mean')
        non_financials_df['std_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('std')

        non_financials_df['mean_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('mean')
        non_financials_df['std_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('std')

        non_financials_df['mean_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
        non_financials_df['std_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

        # Combine the financials and non-financials dataframes back together
        df = pd.concat([financials_df, non_financials_df]).reset_index(drop=True)

        # Z-Score Calculation
        df['Z_ROE_ttm'] = (df['ROE_ttm'] - df['mean_roe']) / df['std_roe']
        df['Z_ROE_ttm_fin'] = (df['ROE_ttm'] - df['mean_roe_fin']) / df['std_roe_fin']
        df['Z_DE'] = (df['Debt/Equity Ratio'] - df['mean_de']) / df['std_de']
        df['Z_EPS_Growth_Variability'] = (df['EPS_Growth_Variability'] - df['mean_eps_growth_variability']) / df['std_eps_growth_variability']
        df['Z_EPS_Growth_Variability_fin'] = (df['EPS_Growth_Variability'] - df['mean_eps_growth_variability_fin']) / df['std_eps_growth_variability_fin']

        df['WeightedAvgZ'] = df.apply(calculate_weighted_avg_z, axis=1)

        # Calculate Normalized Quality Score
        df['Quality_Score'] = np.where(df['WeightedAvgZ'] >= 0, 
                                    1 + df['WeightedAvgZ'], 
                                    (1 - df['WeightedAvgZ'])**-1)

        df = df[['Date_End', 'Symbol', 'Sector','Quality_Score']].dropna().reset_index(drop=True).dropna()

        df['Quality_pct_rank'] = df.groupby('Date_End', group_keys=False)['Quality_Score'].apply(lambda x : x.rank(pct=True))
        # GrowthDate = self.__dataCursor.fetch_data_from_database(table_name='GrowthDate', no_of_years=25)[['Date', 'Qtr']]
        GrowthDate = pd.read_csv('growth_date_latest.csv', parse_dates=['Date'])[['Date', 'Qtr']]
        GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')

        # GrowthDate Mapping  
        final_df =  pd.merge(df, GrowthDate, left_on='Date_End', right_on='Qtr').drop(columns=['Date_End', 'Qtr'])
        final_df = final_df[['Date', 'Symbol', 'Sector', 'Quality_Score', 'Quality_pct_rank']].sort_values(by='Date').reset_index(drop=True)
        
        price_data['Date'] = pd.to_datetime(price_data['Date'])
        # final_df['Date'] = pd.to_datetime(final_df['Date'])
        merged_df = pd.merge(final_df[['Date', 'Symbol', 'Quality_pct_rank']], price_data, on=['Date', 'Symbol'], how='outer')

        merged_df['Quality_pct_rank'] = merged_df.groupby('Symbol', group_keys=False)['Quality_pct_rank'].apply(lambda x: x.fillna(method='ffill'))

        merged_df = merged_df.groupby("Date").apply(lambda x: x.sort_values("Mcap", ascending = False).head(500)).reset_index(drop = True)

        merged_df["Quality_pct_rank"] = merged_df.groupby("Date")["Quality_pct_rank"].rank(pct = True)

        qualityfinal = merged_df[['Date','Symbol','Quality_pct_rank']]

        top500 = qualityfinal.filter(["Symbol", "Date", "Quality_pct_rank"])

        ## Convert the long from Dataframe to wide form dataframe
        top500 = top500.pivot_table(index='Date',columns='Symbol',values='Quality_pct_rank').reset_index()
        ## Shifting the Date column
        top500["Date"] = top500["Date"].shift(-1)
        ## Fill NaN value of date with todays date.
        top500.iloc[-1, top500.columns.get_loc("Date")] = pd.to_datetime(date.today())

        ## Re-converting the wide form dataframe to long form dataframe
        top500 = top500.melt(id_vars='Date', value_name = "QualityQuarter")
        ## Drop na and reset the index
        top500 = top500.dropna().reset_index(drop = True)
        top500 = top500[top500["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

         ## Store the Factor Data in dictionary.
        self.factorData["QualityQuarter"] = top500.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["QualityQuarter"] = top500[top500["Date"] == top500["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].drop_duplicates(["Date","Symbol"]).set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### QUALITY QUARTER COMPLETE ###")

    def generate_ValueYieldNoPeg(self):

        ## Fetch the Stock Value Data
        if self.stockValueData is None:
            self.__readValueData()

        ## Fetch the 
        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        
        ### VALUE ###
        # Filter the rows for Top-500 companies historically
        valueData = self.stockValueData[self.stockValueData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].copy()

        ## List of Fundamental Factor Value.
        fundamentalValueFactor = ["EV_EBITDA", "PS", "PB", "PE"]
        ## Replace -ve value with NaN value for all the fundamental value factor
        for factor in fundamentalValueFactor:
            valueData[factor] = np.where(valueData[factor] <= 0, np.nan, valueData[factor])
            valueData[factor] = 1/valueData[factor]

        ## Sort and reset the index
        valueData.sort_values(["Symbol", "Date"], inplace = True)
        valueData.reset_index(drop = True, inplace = True)


        ### Dividend ###
        dividendData = self.generate_Dividend(valueYieldCall = True)

        ## EPS ###
        epsData = self.generate_Growth(valueYieldCall = True)

        ## ROE
        roe = self.generate_QualityQuarter(valueYieldCall=True)
        roe["Date_End"] = pd.to_datetime(roe['Date_End'],format="%Y%m")

        growth = pd.merge(epsData[["Date", "Date_End", "Symbol","EPS_DILUTED_change"]], 
                          roe[["Date_End", "Symbol", "ROE_ttm"]], on = ["Date_End", "Symbol"], how = "left")

        final=pd.merge(valueData,dividendData,on=['Date','Symbol'])
        final=pd.merge(final,growth,on=['Date','Symbol'],how='left')

        final = final.sort_values(["Symbol","Date"]).reset_index(drop = True)
        final[['ROE_ttm', "EPS_DILUTED_change"]]=final.groupby('Symbol')[['ROE_ttm', "EPS_DILUTED_change"]].ffill()
        final['PEG']=final['PE']/final['EPS_DILUTED_change']

        factorUniverse = ['EV_EBITDA', 'PS','PB', 'PE', 'Dividend','ROE_ttm']

        final = pd.merge(self.strategyUniverse, final[["Date", "Symbol"] + factorUniverse], on = ["Date","Symbol"], how = "left")

        final["AbsRank"] = final.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        final["PeerRank"] = final.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        final['ValueYieldNoPeg'] = final['PeerRank']*0.95 + final['AbsRank']*0.05
        final['ValueABSNoPeg'] = final['PeerRank']*0.05 + final['AbsRank']*0.95        

        result = list()

        # Iterate over each volatility-related column
        for col in ["ValueYieldNoPeg", "ValueABSNoPeg"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = final.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))


        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        ## Store the Factor Data in dictionary.
        self.factorData["ValueYieldNoPeg"] = scores.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ValueYieldNoPeg"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### ValueYieldNoPEG COMPLETE ###")

    def generate_ValueYieldExDiv(self):

        ## Fetch the Stock Value Data
        if self.stockValueData is None:
            self.__readValueData()

        ## Fetch the 
        if self.stockPriceData is None:
            self.__readPriceData()

        if self.sectorData is None:
            self.__readSectorData()

        
        ### VALUE ###
        # Filter the rows for Top-500 companies historically
        valueData = self.stockValueData[self.stockValueData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].copy()

        ## List of Fundamental Factor Value.
        fundamentalValueFactor = ["EV_EBITDA", "PS", "PB", "PE"]
        ## Replace -ve value with NaN value for all the fundamental value factor
        for factor in fundamentalValueFactor:
            valueData[factor] = np.where(valueData[factor] <= 0, np.nan, valueData[factor])
            valueData[factor] = 1/valueData[factor]

        ## Sort and reset the index
        valueData.sort_values(["Symbol", "Date"], inplace = True)
        valueData.reset_index(drop = True, inplace = True)


        ### Dividend ###
        dividendData = self.generate_Dividend(valueYieldCall = True)

        ## EPS ###
        epsData = self.generate_Growth(valueYieldCall = True)

        ## ROE
        roe = self.generate_QualityQuarter(valueYieldCall=True)
        roe["Date_End"] = pd.to_datetime(roe['Date_End'],format="%Y%m")

        growth = pd.merge(epsData[["Date", "Date_End", "Symbol","EPS_DILUTED_change"]], 
                          roe[["Date_End", "Symbol", "ROE_ttm"]], on = ["Date_End", "Symbol"], how = "left")

        final=pd.merge(valueData,dividendData,on=['Date','Symbol'])
        final=pd.merge(final,growth,on=['Date','Symbol'],how='left')

        final = final.sort_values(["Symbol","Date"]).reset_index(drop = True)
        final[['ROE_ttm', "EPS_DILUTED_change"]]=final.groupby('Symbol')[['ROE_ttm', "EPS_DILUTED_change"]].ffill()
        final['PEG']=final['PE']/final['EPS_DILUTED_change']

        factorUniverse = ['EV_EBITDA', 'PS','PB', 'PE','PEG','ROE_ttm']

        final = pd.merge(self.strategyUniverse, final[["Date", "Symbol"] + factorUniverse], on = ["Date","Symbol"], how = "left")

        final["AbsRank"] = final.groupby("Date")[factorUniverse].rank(pct = True).mean(axis = 1)
        final["PeerRank"] = final.groupby(["Date", "Sector"])[factorUniverse].rank(pct = True).mean(axis = 1)

        final['ValueYieldExDiv'] = final['PeerRank']*0.95 + final['AbsRank']*0.05
        final['ValueABSExDiv'] = final['PeerRank']*0.05 + final['AbsRank']*0.95        

        result = list()

        # Iterate over each volatility-related column
        for col in ["ValueYieldExDiv", "ValueABSExDiv"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = final.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))


        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()

        ## Store the Factor Data in dictionary.
        self.factorData["ValueYieldExDiv"] = scores.copy()
        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ValueYieldExDiv"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### ValueYieldExDiv COMPLETE ###")

    def generate_ShortAM(self,):

        # Function to calculate log returns
        def calculate_log_returns(df):
            df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
            return df.dropna()

        # Function to calculate annualized standard deviation
        def calculate_annualized_std(df, window=252):
            return df['LogReturn'].rolling(window).std() * np.sqrt(window)

        # Function to calculate momentum ratios
        def calculate_momentum_ratios(series, period):
            return series / series.shift(period) - 1

        ################
        ## PRICE DATA ##
        ################
        # Check if stock price data is loaded; if not, read it
        if self.stockPriceData is None:
            self.__readPriceData()

        ## Drop the Unnecessary COlumns
        priceData = self.stockPriceData.drop(columns = ["Open", "High", "Low", "Volume", "Mcap"]).copy()
        
        ## Filter the Symbol which are present in strategy universe
        priceData = priceData[priceData["Symbol"].isin(self.strategyUniverse["Symbol"].unique())].reset_index(drop = True)

        # Define periods for momentum ratios (in trading days)
        periods = {
            'MR1': 21,   # 1 month
            'MR2': 42,   # 2 months
            'MR3': 63,   # 3 months
        }

        # Apply log return calculation
        priceData = priceData.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
    
        # Calculate momentum ratios for each period
        for label, period in periods.items():
            priceData[label] = priceData.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

        # Calculate annualized standard deviation
        priceData['AnnualizedStd'] = priceData.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)

        # Normalize the momentum ratios by dividing by the annualized standard deviation
        for label in periods.keys():
            priceData[label] /= priceData['AnnualizedStd']

        priceData = priceData.sort_values(["Date", "Symbol"]).reset_index(drop = True)

        # Calculate the mean and std deviation of each momentum ratio across the universe
        for label in periods.keys():
            priceData[f'mu_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.mean())
            priceData[f'sigma_{label}'] = priceData.groupby('Date')[label].transform(lambda x: x.std())

        # Calculate Z-scores for each period
        for label in periods.keys():
            priceData[f'Z_{label}'] = (priceData[label] - priceData[f'mu_{label}']) / priceData[f'sigma_{label}']

        # Define specific combinations for which to calculate the final Z-scores
        metrics = ['Z_MR1', 'Z_MR2', 'Z_MR3']

        # Weighted average Z-score
        priceData["WtdZScore"] =priceData[metrics].mean(axis= 1)

        # Normalized momentum score
        priceData[f'Momentum'] = np.where(priceData[f'WtdZScore'] >= 0,
                                                1 + priceData[f'WtdZScore'],
                                                (1 - priceData[f'WtdZScore']) ** -1)

        ## Filter the Required Columns  
        priceData = priceData.filter(items = ["Date", "Symbol", "Momentum"])

        ## Merge the Computed Metrics against the symbol on each day's universe
        priceData = pd.merge(self.strategyUniverse, priceData, on = ["Date", "Symbol"], how = "left")

        ## Computing the Percentile Score of the Symbols on each Date
        priceData["Momentum"] = priceData.groupby(["Date"])["Momentum"].rank(ascending = True, pct = True)

        ## Computing the Aggreate rank of Peer (Theme / Sector / GICS) on each date using Symbol
        priceData["PeerMomentum"] = priceData.groupby(["Date", "Peer"])["Momentum"].transform(lambda x: x.mean())

        ## Re-Ranki The Agg.Score of Peer (Theme / Sector / GICS) on each Date.
        priceData["PeerMomentum"] = priceData.groupby(["Date"])["PeerMomentum"].rank(ascending = True, pct = True)

        ## Filter the data after year 2006
        priceData = priceData[priceData["Date"] >= "2006-01-01"].reset_index(drop = True)
         
        # Initialize an empty list to store the transformed data for each volatility column
        result = list()

        # Iterate over each volatility-related column
        for col in ["Momentum", "PeerMomentum"]:

            # Filter the dataframe to keep only the "Symbol", "Date", and current column
            temp = priceData.filter(["Symbol", "Date", col])

            # Pivot the table to have dates as rows and symbols as columns, with values from the current column
            temp = temp.pivot_table(index='Date', columns='Symbol', values=col).reset_index()

            # Shift the "Date" column up by one row to align data with the next date
            temp["Date"] = temp["Date"].shift(-1)

            # Set the last row's "Date" to today's date to capture current data
            temp.iloc[-1, temp.columns.get_loc("Date")] = pd.to_datetime(date.today())

            # Unpivot the table to return it to long format with "Date", "Symbol", and current column values
            temp = temp.melt(id_vars='Date', value_name=col)

            # Drop rows with missing values and reset the index
            temp = temp.dropna().reset_index(drop=True)

            # Filter data to include only records from January 1, 2006, onwards and reset the index
            temp = temp[temp["Date"] >= "2006-01-01"].reset_index(drop=True).copy()

            # Set the index to "Date" and "Symbol" for each transformed column and add to the result list
            result.append(temp.set_index(["Date", "Symbol"]))

        
        # Concatenate all transformed columns along the columns axis to create a combined dataframe
        scores = pd.concat(result, axis=1).reset_index()
        scores.columns = scores.columns.str.replace("Momentum", "AM").str.replace("Peer", self.peer)
        scores.rename(columns = {"AM" : "ShortAM", f"{self.peer}AM" : f"Short{self.peer}AM"},inplace=True)
        self.x = scores.copy()

        # Sort the final dataframe by "Symbol" and "Date" columns for organized viewing and reset the index
        scores = scores.sort_values(["Symbol", "Date"]).reset_index(drop=True)

        ## Store the Factor Data in dictionary.
        self.factorData["ShortAM"] = scores.copy()

        ## Filter the data for latest update or values.
        self.recentFactorUpdate["ShortAM"] = scores[scores["Date"] == scores["Date"].max()].reset_index(drop = True)
        
        ## Combining the Factors in one Dataframe
        self.stockFactors = pd.concat([self.recentFactorUpdate[key].set_index(["Date", "Symbol"]) for key in self.recentFactorUpdate.keys()], axis = 1)
        self.stockFactors.reset_index(inplace = True)

        print("### SHORT MOMENTUM COMPLETE ###")

        del priceData, result, temp, scores
    
    def generate_AllFactors(self,save_appender = False):
        ## FUnction call for LTM Factor
        # self.generate_LTM()
        # ## Function call for Momentum Factor
        # self.generate_Momentum()
        # ## Function call for Theme Factor
        # self.generate_Theme()
        # ## Function call for Volatility Factor
        # self.generate_Volatility()
        ## Function call for Value Factor
        # self.generate_ValueYield()
        ## Function call for Growth Factor
        self.generate_Growth()
        ## Function call for Quality Factor
        self.generate_QualityAnnual()
        ## Function call for Quality Factor
        self.generate_QualityQuarter()
        ## Function call for EM Factor
        # self.generate_EM()
        ## Function call for Dividend Factor
        # self.generate_Dividend()
        ## Function call for LTMA Factor
        # self.generate_LTMA()
        ## Function call for Low Vol Factor
        # self.generate_LowVol()
        ## Function call for AM Factor
        # self.generate_AM()
        ## Function call for Short AM Factor
        # self.generate_ShortAM()
        ## Function call for AM Factor
        # self.generate_ValueYieldNoPeg()
        ## Function call for AM Factor
        # self.generate_ValueYieldExDiv()

        # Extract unique dates from the 'Date' column of the strategyUniverse DataFrame.
        # Drop duplicates and reset the index to create a clean, unique list of dates.
        dates = self.strategyUniverse[["Date"]].drop_duplicates().reset_index(drop=True)

        # Create a new column 'DateO' which is the next date (shifted upwards by one row).
        # This maps each date to the following date in the dataset.
        dates["DateO"] = dates["Date"].shift(-1)

        # Create a dictionary to map each date to its corresponding next date ('DateO').
        dateMapper = dict(zip(dates["Date"], dates["DateO"]))

        # Make a copy of the original strategyUniverse DataFrame to avoid modifying it directly.
        strategyUniverse = self.strategyUniverse.copy()

        # Map the 'DateO' column in the copied DataFrame using the dateMapper.
        strategyUniverse["DateO"] = strategyUniverse["Date"].replace(dateMapper)

        # Fill any missing values in 'DateO' (last row) with today's date.
        strategyUniverse["DateO"] = strategyUniverse["DateO"].fillna(pd.to_datetime(date.today()))

        # Drop the original 'Date' column as 'DateO' will now replace it.
        strategyUniverse.drop(columns=["Date"], inplace=True)

        # Rename the 'DateO' column back to 'Date' to maintain consistency in column names.
        strategyUniverse.rename(columns={'DateO': "Date"}, inplace=True)

        # Reorder the columns so that the 'Date' column is the last one in the DataFrame.
        strategyUniverse = strategyUniverse[list(strategyUniverse.columns[-1:]) + list(strategyUniverse.columns[:-2])]

        ## Generating the Appender file only when all the factors are generated.
        self.appender = pd.DataFrame()
        for key in self.factorData.keys():
            if len(self.appender) == 0:
                self.appender = pd.merge(strategyUniverse,  self.factorData[key], on = ["Date", "Symbol"], how = "left")
            else:
                self.appender = pd.merge(self.appender, self.factorData[key], on = ["Date", "Symbol"], how= "left")

        ## Save the appender file to csv only if the flag is True
        if save_appender:
            ## If there is no directory then create the directory
            if not os.path.isdir("./DailyAppender"):
                os.mkdir("./DailyAppender")

            # self.appender.to_csv(self.uniqueFileName(f"./DailyAppender/Appender_{date.today().strftime('%d%b%Y')}.csv"), index = False)
            self.appender.to_csv(f"./DailyAppender/Appender_Cons_latest.csv", index = False)

        print("### ALL THE FACTORS ARE GENERATED ###")    


In [4]:
## Import the Libraries
import pandas as pd
import numpy as np
import functools as ft
import datetime
import os
import itertools
import logging
import smtplib
import sys
import itertools
import zipfile
import shutil
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from email.message import EmailMessage

# from rebalancedate import rebalancedates

# from AQUA import AQUA
# from getNav import navCalculator



# A function to get Yesterday's Date
def Yesterdays_Date():
    todays_date = datetime.date.today()
    yesterdays_date = todays_date - datetime.timedelta(days=1)
    return yesterdays_date

# set logger
def set_logger(log_folderpath: str):
    
    # Setting up the logger
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s %(message)s')

    #date
    date_str = Yesterdays_Date().strftime('%d%m%Y')
    today_date = datetime.date.today().strftime('%Y-%m-%d')

    # Creating log folder datewise
    if not os.path.exists(log_folderpath):
        os.makedirs(log_folderpath)

    # Setting log file name as the date for which updation is taking place
    # file_handler = logging.FileHandler(fr"{log_folderpath_date}\Factor_Sheet_log_{date_str}.log", mode='a')
    file_handler = logging.FileHandler(fr"{log_folderpath}\Factor_Sheet_log_{today_date}.log", mode='a')
    file_handler.setFormatter(formatter)
    if (logger.hasHandlers()):
        logger.handlers.clear()
    logger.addHandler(file_handler)

    # Logging
    logger.debug(f"Date:{today_date}")
    logger.debug('logger set')

    return logger

# send mail
def send_mail(from_email: str, recipients: list,  file_path_ls: list = None, cc_email: list = None, subject: str = None, plain_body_text: str = None, html_body_text: str = None):

    logger.debug('sending_email')
    logger.debug(f'from_email = {from_email}')
    logger.debug(f'recipients = {recipients}')
    logger.debug(f'cc_email = {cc_email}')
    logger.debug(f'file_path_ls = {file_path_ls}')
    logger.debug(f'subject = {subject}')
    logger.debug(f'plain_body_text = {plain_body_text}')
    logger.debug(f'html_body_text = {html_body_text}')

    msg = EmailMessage()

    #recipients
    recipients_emails_str = ', '.join(recipients)

    #cc_email
    if cc_email != None:
        cc_email_str = ', '.join(cc_email)

    # Set plain text (fallback)
    if plain_body_text != None:
        msg.set_content(plain_body_text)

    # Set HTML content
    if html_body_text != None:
        msg.add_alternative(html_body_text, subtype='html')

    #add attachment:
    if file_path_ls != None:
        for file_path, maintype, subtype, filename in file_path_ls:
            try:
                with open(file_path, 'rb') as fp:
                    file_data = fp.read()
                    msg.add_attachment(file_data, maintype=maintype, subtype=subtype, filename=filename)
            except FileNotFoundError as e:
                logger.debug(f'error = {e}')


    #mail subject, from, to
    if subject != None:
        msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = recipients_emails_str
    if cc_email != None:
        msg['Cc'] = cc_email_str

    # Send email
    with smtplib.SMTP(host='192.168.11.18', port=25) as s:
        s.send_message(msg)
        logger.debug('mail send')

# Folder Path where log files will be stored
today_date = datetime.date.today().strftime('%Y-%m-%d')
# log_folderpath = fr"C:\Users\ronaksinghsahni\OneDrive - PRABHUDAS LILLADHER PVT. LTD\Documents\sagar_appender\output\{today_date}"
# log_folderpath = fr"C:\Users\Administrator\PycharmProjects\Projects\sagar_appender\akshat_appender\output\{today_date}_akshat"
log_folderpath = fr"output\{today_date}_akshat"
# log_folderpath = fr"C:\Users\Administrator\PycharmProjects\Projects\sagar_appender\output\{today_date}\Factor_Sheet_log_{today_date}.txt"

logger = set_logger(log_folderpath)

# try:
factor = EquityFactors(noOfYears = 30, correct = True, peer = "Sector")
factor.generate_AllFactors(save_appender=True)
appender = factor.appender.copy()
logger.debug('EquityFactors completed appender generated')
full_appender = True
# appender = pd.read_csv('DailyAppender/Appender_latest.csv', parse_dates=['Date'])
# except Exception as e:
#     logger.debug(f'error occurred in function EquityFactors (appender): {e}')


#read consol factor csv files: appender, stockPriceData, benchmark
# class ReadCSVConsolFactor:

#     def __init__(self):
#         ## Initialize the variables
#         self.appender = pd.read_csv('DailyAppender/Appender_Cons_latest.csv', parse_dates=['Date'])
#         self.stockPriceData = pd.read_csv('DailyAppender/stockPriceData.csv', parse_dates=['Date'])
#         self.benchmark = pd.read_csv('DailyAppender/benchmark.csv', parse_dates=['Date'])

# consol_factor = ReadCSVConsolFactor()
# consol_factor_df = consol_factor.appender.copy()

# try:
consol_factor = EquityFactorsConsol(noOfYears = 30, correct = True, peer = "Sector")
consol_factor.generate_AllFactors(save_appender=False)
consol_factor_df = consol_factor.appender.copy()
# consol_factor_df.to_csv('DailyAppender/Appender_Cons_latest.csv', index=False)
logger.debug('EquityFactorsConsol completed appender_consol generated')
consol_factor_df.rename(columns={'Growth': 'GrowthConsol', 'QualityAnnual': 'QualityAnnualConsol', 'QualityQuarter': 'QualityQuarterConsol'}, inplace=True)
consol_factor_df = consol_factor_df[['Date', 'Symbol','GrowthConsol', 'QualityAnnualConsol', 'QualityQuarterConsol']]
# except Exception as e:
#     logger.debug(f'error occurred in function EquityFactorsConsol (appender_consol): {e}')

#generate output csv of stockPriceData and benchmark
# consol_factor.stockPriceData.to_csv('DailyAppender/stockPriceData.csv', index=False)
# consol_factor.benchmark[['Date', 'BenchmarkPrice']].to_csv('DailyAppender/benchmark.csv', index=False)
# logger.debug('stockPriceData.csv and benchmark.csv generated')

# shift date by -1 and shift nan date to todays date
def date_shift(df):
    df['date_shift'] = df.groupby(['Symbol'])['Date'].shift(-1)
    todays_date = datetime.datetime.today()
    todays_date = pd.to_datetime(todays_date.strftime('%Y-%m-%d'))
    # df['date_shift'].fillna(pd.to_datetime(todays_date), inplace=True)
    df['date_shift'] = df['date_shift'].fillna(pd.to_datetime(todays_date))
    df['Date'] = df['date_shift']
    df.drop(columns={'date_shift'}, inplace=True)
    return df

appender_final = appender.copy()

def calculate_distance52Low(df):  
    df['52W_Low'] = df.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=252, min_periods=1).min())
    df['percentage_change_52WLow'] = ((df['52W_Low'] / df['Close']) - 1).mul(100)
    # df = df[df['Date'] == df['Date'].iloc[-1]].reset_index(drop=True)
    df = df[['Date', 'Symbol', 'percentage_change_52WLow']]
    return df

def calculate_append_value_price(appender_final, price_data):
    # 52 week low
    # price_data = consol_factor.stockPriceData.copy()
    price_data.rename(columns={'Price': 'Close'}, inplace=True)
    price_data = price_data[['Date', 'Symbol', 'Close']].reset_index(drop=True)
    latest_52WLow = calculate_distance52Low(price_data.copy(deep=True))
    latest_52WLow = date_shift(latest_52WLow)
    # ValuePrice
    appender_final = pd.merge(appender_final, latest_52WLow, how='left', on=['Date', 'Symbol'])
    appender_final['ValuePrice'] = appender_final.groupby('Date')['percentage_change_52WLow'].rank(pct=True, ascending=True)
    # appender_final['ValuePrice'] = appender_final['percentage_change_52WLow'].rank(pct=True, ascending=True)
    appender_final.drop(columns={'percentage_change_52WLow'}, inplace=True)
    return appender_final

try:
    appender_final = calculate_append_value_price(appender_final, consol_factor.stockPriceData.copy())
    logger.debug('calculate_append_value_price completed')
except Exception as e:
    logger.debug(f'error occurred in function calculate_append_value_price: {e}')

#append factor sector, theme
def append_factor_sector_theme(appender_final, factor_ls, classification_type_ls):
    for classification in classification_type_ls:
        for factor_name in factor_ls:
            factor_sector = appender_final.groupby(['Date', classification])[factor_name].mean().rank(pct=True, ascending=True).reset_index()
            factor_sector.rename(columns={factor_name: f'{classification}{factor_name}'}, inplace=True)
            appender_final = pd.merge(appender_final, factor_sector, how='left', on=['Date', classification])
    return appender_final

# try:
# consol factors
appender_final = pd.merge(appender_final, consol_factor_df, how='left', on=['Date', 'Symbol'])

append_factor_sector_theme_ls = ['HighBeta', 'LowBeta', 'ValueYield', 'ValueABS', 'ValuePrice', 'Growth', 'GrowthConsol', 'TrendMR', 'AntiTrendMR']
appender_final = append_factor_sector_theme(appender_final, factor_ls=append_factor_sector_theme_ls, classification_type_ls=['Sector', 'Theme'])

append_factor_theme_ls = ['AM', 'UltraShortAM', 'ShortAM', 'MidAM', 'LongAM', 'LTM', 'LTMA', 'EM', 'LowVol', 'DownVol', 'AvgVol']
appender_final = append_factor_sector_theme(appender_final, factor_ls=append_factor_theme_ls, classification_type_ls=['Theme'])

logger.debug('append_factor_sector_theme completed')
# except Exception as e:
#     logger.debug(f'error occurred in function append_factor_sector_theme: {e}')

#full appender generate
if full_appender:
    appender_final.to_csv('DailyAppender/Full_appender.csv', index=False)
    print('full appender generated')
    logger.debug('full appender generated')
    # sys.exit()


### LTM COMPLETE ###
### ValueYield COMPLETE ###
### GROWTH COMPLETE ###
### QUALITY ANNUAL COMPLETE ###
### QUALITY QUARTER COMPLETE ###
### EM COMPLETE ###
### DIVIDEND COMPLETE ###
### LTMA COMPLETE ###
### LOWVOL COMPLETE ###
### MOMENTUM COMPLETE ###
### ULTRA SHORT MOMENTUM COMPLETE ###
### SHORT MOMENTUM COMPLETE ###
### ValueYieldNoPEG COMPLETE ###
### ValueYieldExDiv COMPLETE ###
### MID MOMENTUM COMPLETE ###
### LONG MOMENTUM COMPLETE ###
### MID MOMENTUM COMPLETE ###
### BETA COMPLETE ###
### TREND COMPLETE ###
### SHIFTED AM COMPLETE ###
### 3D QUALITY COMPLETE ###
### 3D VALUE COMPLETE ###
### ALL THE FACTORS ARE GENERATED ###
### GROWTH COMPLETE ###
### QUALITY ANNUAL COMPLETE ###
### QUALITY QUARTER COMPLETE ###
### ALL THE FACTORS ARE GENERATED ###
full appender generated


,Date,Symbol,Mcap,MCAP_Type,Sector,Theme,GICS,LTM,ValueYield,ValueABS,...,ThemeUltraShortAM,ThemeShortAM,ThemeMidAM,ThemeLongAM,ThemeLTM,ThemeLTMA,ThemeEM,ThemeLowVol,ThemeDownVol,ThemeAvgVol
0,2006-01-03,3IINFOLTD,10308.833182,Small,IT,IT - Software,InformationTechnology,NaN,NaN,NaN,...,0.411617,0.657857,0.574197,0.427606,0.006074,0.463739,0.457505,0.583383,0.523855,0.562052
1,2006-01-03,3MINDIA,9662.050539,Small,Diversified,Diversified,Industrials,NaN,NaN,NaN,...,0.385273,0.336062,0.505973,0.729573,0.210003,0.664657,0.420483,0.530650,0.549679,0.535854
2,2006-01-03,63MOONS,58362.450476,Large,IT,IT - Software,InformationTechnology,NaN,NaN,NaN,...,0.411617,0.657857,0.574197,0.427606,0.006074,0.463739,0.457505,0.583383,0.523855,0.562052
3,2006-01-03,AARTIIND,5455.246093,Small,Chemicals,Chemicals,Chemicals,NaN,NaN,NaN,...,0.637129,0.427392,0.471549,0.552877,0.064611,0.556062,0.551893,0.200178,0.206188,0.193764
4,2006-01-03,ABAN,21307.541602,Mid,Crude Oil,Oil Exploration,Energy,NaN,NaN,NaN,...,0.879100,0.748131,0.409333,0.625954,NaN,0.645305,0.394864,0.561185,0.690176,0.601453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412495,2025-06-24,ZENSARTECH,192371.773014,Small,IT,IT - Software,InformationTechnology,0.706377,0.661956,0.610128,...,0.812606,0.793572,0.403107,0.459416,0.601718,0.444828,0.493615,0.512314,0.468275,0.495354
2412496,2025-06-24,ZENTEC,171530.668333,Small,IT,IT - Software,InformationTechnology,0.924433,0.291073,0.288632,...,0.812606,0.793572,0.403107,0.459416,0.601718,0.444828,0.493615,0.512314,0.468275,0.495354
2412497,2025-06-24,ZFCVINDIA,250523.849472,Small,Automobile & Ancillaries,Auto Ancillary,Automobile & Ancillaries,0.303555,0.320072,0.411037,...,0.350545,0.607686,0.523744,0.365520,0.598362,0.505113,0.408815,0.442001,0.440641,0.447037
2412498,2025-06-24,ZYDUSLIFE,963066.551829,Mid,Healthcare,Pharmaceuticals & Drugs,Healthcare,0.392913,0.806724,0.694777,...,0.433340,0.394800,0.356062,0.652217,0.304999,0.548429,0.469953,0.569681,0.592292,0.578240
